In [ ]:
#| hide

# This JavaScript cell adjusts the notebook width in case the default is too narrow
# Please, note that it will only work in vanilla Jupyter Notebook, not Jupyter Lab
# In any case, Jupyter Notebook is also recommended for the IFC.js viz to work properly

from IPython.display import Javascript
Javascript('document.getElementById("notebook-container").style.width = "45vw";')  # adapt width as preferred

<IPython.core.display.Javascript object>

In [ ]:
#| default_exp core

# Parametric house in IFC - Walkthrough

> Recreation of the original IfcOpenHouse with the IfcOpenShell Python API 

In [ ]:
#| hide

#  If you're seeing this, you're seeing the Jupyter Notebook version.
#  Feel free to pass by the static website if you don't need to run the code:
#  https://cvillagrasa.github.io/IfcOpenHouse/

#  You can ignore the next cell, which is targeted at the website version.
#  You may now start to explore the notebook, just run each cell (SHIFT + ENTER) and enjoy.

```{=html}
<!-- This is where a canvas with an IFC.js visualisation of the model
    will be placed within the Quarto website -->
<div id="ifcjs-container">
</div>
```

\

Welcome to the static website version of this Jupyter Notebook. If you wish to run the code, the notebook can be found [here](https://github.com/cvillagrasa/IfcOpenHouse/blob/master/nbs/00_generation.ipynb). In addition, you may follow the instructions on the [home page](index.ipynb) in order to clone [the repository](https://github.com/cvillagrasa/IfcOpenHouse) and install the required dependencies.

If you prefer pure Python 🐍, note that an autogenerated script of this notebook can also be found [here](https://github.com/cvillagrasa/IfcOpenHouse/blob/master/IfcOpenHouse/core.py).

If in doubt, just stick to the website version first.

## Necessary imports

Run the next cell to import the necessary libraries, as well as an [`ios_utils`](https://github.com/cvillagrasa/IfcOpenHouse/blob/master/IfcOpenHouse/ios_utils.py) module which has been placed in a separate \*.py file so as to focus on the important bits. Feel free to also consult its content, if desired.

In [ ]:
#| exports

import sys
from pathlib import Path
from collections import defaultdict
import numpy as np
import ifcopenshell
import ifcopenshell.api
import ifcopenshell.api.owner
import ifcopenshell.api.owner.settings
import ifcopenshell.api.material
import ifcopenshell.api.geometry
import ifcopenshell.validate

from IfcOpenHouse.ios_utils import (
    IfcOpenShellPythonAPI, placement_matrix, clipping_matrix, Size2D, Size3D, ColourRGB, 
    TerrainBuildMethod, build_native_bspline_terrain, build_tesselated_occ_terrain,
    ios_entity_overwrite_hook
)

## Definition of project data

Note that project data defined here can be modified conveniently to parametrically build our IFC model.

In [ ]:
#| exports

# Data definition
project_name = 'IFC Open House'
author_details = {'given_name': 'Carlos', 'family_name': 'V', 'identification': 'CV'}
author_role = 'CIVILENGINEER'
organization_details = {'name': 'OSArch', 'identification': 'OSArch'}
site_name, building_name, storey_name = 'OSArch Land', 'Open house', 'Ground floor'

# All dimensions in meters
storey_size = Size3D(10., 5., 3.)  # Utility class to call "storey_size.x" and improve readability
wall_thickness = 0.36
footing_ledge = 0.05
footing_size = Size3D(
    storey_size.x + 2 * (wall_thickness + footing_ledge),
    storey_size.y + 2 * (wall_thickness + footing_ledge),
    2.
)
roof_ledge = Size2D(0.1, 0.22)
roof_thickness = 0.36
roof_angle = 45. # degrees
roof_angle_sin = float(np.sin(roof_angle * np.pi/180))
roof_angle_cos = float(np.cos(roof_angle * np.pi/180))
roof_height = float(
    (storey_size.y / 2 + wall_thickness + roof_ledge.y) * np.tan(roof_angle * np.pi / 180)
)
roof_size = Size3D(
    storey_size.x + 2 * (wall_thickness + roof_ledge.x),
    storey_size.y + 2 * (wall_thickness + roof_ledge.y),
    roof_height
)
door_horizontal_offset = 1.6
window_base_height = 0.4
right_window_horizontal_offset = 2.
stair_width = 1.2

# Colours for surface styles
wall_colour = ColourRGB(.75, 0.73, 0.68)
footing_colour = ColourRGB(.38, 0.4, 0.42)
roof_colour = ColourRGB(.24, 0.08, 0.04)
terrain_colour = ColourRGB(.15, 0.25, 0.05)
door_colour = ColourRGB(.8, .8, .8)
window_colour = ColourRGB(.5, 0.4, 0.3, transparency=0.8)

# Choice of terrain building method. Use NONE if unsure about the viewer capabilities.   
terrain_build_method = TerrainBuildMethod.TESSELATE_OCC_SHAPE

# Door and window geometric info is defined in a separate file due to its complexity
from IfcOpenHouse.opening_data import door_params, single_window_params, triple_window_params


## Setting up a project

Without further ado, let's get down to business:

In [ ]:
#| exports

# Little trickery to ease the use of the ifcopenshell.api when scripting
ios = IfcOpenShellPythonAPI()  #q1: thoughts about a data-scientish "import ifcopenshell.api as ios"?

# standard use      -> ifcopenshell.api.run('root.create_entity', file, ifc_class='IfcWall')
# with the trickery -> ios.root.create_entity(file, ifc_class='IfcWall')

# It will reduce the overall string overhead, as well as the length of the API calls
# However, it will not help with static typing autocomplete and help
# Bear in mind that currently, this is not a canonical use of IfcOpenShell


# Setting up the project
file = ios.project.create_file(version='IFC4X3')  # We're using 4x3, because why not? it has better docs
project = ios.root.create_entity(file, ifc_class='IfcProject', name=project_name)
ios.project.assign_declaration(file, definition=project, relating_context=project)  #q2: from my ignorance, is this necessary?
ios.unit.assign_unit(
    file, length={'is_metric': True, 'raw': 'METERS'}, area={'is_metric': True, 'raw': 'METERS'},
    volume={'is_metric': True, 'raw': 'METERS'}
)
ctx = ios.context.add_context(file, context_type='Model')
body = ios.context.add_context(  #q3: isn't this screaming for "context.add_subcontext"? also, context_type may be redundant
    file, context_type='Model', context_identifier='Body', target_view='MODEL_VIEW', parent=ctx
)

# We allow for live overwriting of IfcOpenShell entities within the notebook environment
# Only use this sorcery when experimenting in Jupyter Notebooks, never in production
sys.addaudithook(
    ios_entity_overwrite_hook(file, sys.modules[__name__], do_not_delete=[project, ctx, body])
)

Ok, we just got started! The `file` object we just created will be the main binding agent for the rest of the notebook. As it can be seen, most API calls are self-explanatory. In order to look up for the most updated
documentation, check out the following links:

[https://blenderbim.org/docs-python/autoapi/ifcopenshell/api/index.html](https://blenderbim.org/docs-python/autoapi/ifcopenshell/api/index.html)

[https://blenderbim.org/docs-python/ifcopenshell-python/code_examples.html](https://blenderbim.org/docs-python/ifcopenshell-python/code_examples.html)

In case you might also want to have a look at the Python source code, this is the place to go:

[https://github.com/IfcOpenShell/IfcOpenShell/tree/v0.7.0/src/ifcopenshell-python/ifcopenshell/api](https://github.com/IfcOpenShell/IfcOpenShell/tree/v0.7.0/src/ifcopenshell-python/ifcopenshell/api)

## Optional definition of owner data

IFCs can hold data from stakeholders, like a person or an organisation. With the last two lines, we're assigning the user here defined to the rest of the IfcOpenShell API actions taken from now on.

In [ ]:
#| exports

application = ios.owner.add_application(file)
person = ios.owner.add_person(file, **author_details)
organisation = ios.owner.add_organisation(file, **organization_details)
user = ios.owner.add_person_and_organisation(file, person=person, organisation=organisation)
ios.owner.add_role(file, assigned_object=organisation, role=author_role)
actor = ios.owner.add_actor(file, actor=user)
ifcopenshell.api.owner.settings.get_user = lambda x: user
ifcopenshell.api.owner.settings.get_application = lambda x: application

## Setting up the basic project outline

That's how we define a site, containing a building, which in turn contains a building storey:

In [ ]:
#| exports

site = ios.root.create_entity(file, ifc_class='IfcSite', name=site_name)
ios.aggregate.assign_object(file, product=site, relating_object=project)

building = ios.root.create_entity(file, ifc_class='IfcBuilding', name=building_name)
ios.aggregate.assign_object(file, product=building, relating_object=site)

storey = ios.root.create_entity(file, ifc_class='IfcBuildingStorey', name=storey_name)
ios.aggregate.assign_object(file, product=storey, relating_object=building);

Property Sets can be easily added with `pset.add_pset` and `pset_edit_pset`. Here, we set the total area of the site to 20x20 = 400 m2.

In [ ]:
#| exports

pset_site_common = ios.pset.add_pset(file, product=site, name='Pset_SiteCommon')
ios.pset.edit_pset(file, pset=pset_site_common, properties={'TotalArea': 400.})

## Creating our first wall

The following statement will create a brand new wall entity within our IFC file:

In [ ]:
#| exports

south_wall = ios.root.create_entity(file, ifc_class='IfcWall', name='South wall', predefined_type='SOLIDWALL')

As expected, IfcOpenShell is tightly coupled to the underlying IFC Schema. Hence, opening a browser tab with the [IFC official documentation](http://ifc43-docs.standards.buildingsmart.org/) is not at all a bad idea when using it.

In order to better illustrate it with this wall, let's consider the following. If we want to know among which predefined types we can choose for a certain IfcWall, going to [its documentation page](https://ifc43-docs.standards.buildingsmart.org/IFC/RELEASE/IFC4x3/HTML/lexical/IfcWall.htm), scrolling to "Predefined Type", and clicking into [IfcWallTypeEnum](https://ifc43-docs.standards.buildingsmart.org/IFC/RELEASE/IFC4x3/HTML/lexical/IfcWallTypeEnum.htm) will give us the appropriate list: MOVABLE, PARAPET, PARTITIONING, PLUMBINGWALL, RETAININGWALL, SHEAR, SOLIDWALL and WAVEWALL.

Clearly, our traditional structural wall for the house needs to be classified as a SOLIDWALL. As always in IFC, a predefined type could also be set as USERDEFINED if none of the enumeration options fits our use case, in which case we could monkey-patch the type as `south_wall.ObjectType = 'CUSTOM TYPE'`.

The previous kind of reasoning and consulting of the documentation is to be followed for the rest of this notebook. In case that at some point we needed to know which IFC class a certain IfcOpenShell entity corresponds to (so as to know where in the IFC documentation to look at), the `is_a()` method will give us the answer:

In [ ]:
#| exports

south_wall.is_a()

'IfcWall'

This notebook environment highly encourages experimentation, like in the statement above. Feel free to add a new cell at any point to further explore the newly created variables. Another very useful method is `get_info()`, which returns a dictionary with all the entity attributes:

In [ ]:
#| exports

south_wall.get_info()

{'id': 41,
 'type': 'IfcWall',
 'GlobalId': '0hjXlpMtn1KPhS1825wdB9',
 'OwnerHistory': #40=IfcOwnerHistory(#19,#16,.READWRITE.,.ADDED.,1682984215,#19,#16,1682984215),
 'Name': 'South wall',
 'Description': None,
 'ObjectType': None,
 'ObjectPlacement': None,
 'Representation': None,
 'Tag': None,
 'PredefinedType': 'SOLIDWALL'}

As seen previously with user defined types, entity attributes can be easily [monkey-patched](https://en.wikipedia.org/wiki/Monkey_patch) 🐵 (we're in Python after all!). It will work if the attribute exists in the corresponding IFC Schema. For instance, if we wanted to set the "Description" of our wall, we would do it as follows:

In [ ]:
#| exports

south_wall.Description = 'This is my first wall with the IfcOpenShell Python API!'

In any case, the IfcOpenShell Python API takes care of most low level tasks. Actions like the generation of global IDs, the assignment of owner history to entities or the setup of entity relationships for geometric representations or placements are already dealt with automatically.

Now, let's assign the wall to the ground floor storey by means of `spatial.assign_container`:

In [ ]:
#| exports

ios.spatial.assign_container(file, product=south_wall, relating_structure=storey);

It's important to keep in mind the IFC distinction between geometry and semantics. Up to now, we've created an IfcWall entity, only containing a name and a predefined type. However, the geometric representations of the actual wall will exist within their own IfcShapeRepresentation entity. That is to say, one thing is the creation of the IfcWall, but then its geometry needs to be created as a separate IFC class, which in turn will be linked to the former.

The API call `geometry.add_wall_representation` generates a swept solid (IfcExtrudedAreaSolid), an implicit geometry representation which requires less memory than explicit BREPS or tesselated representations. You should know that IFC supports all of them, among other fancy paradigms like Constructive Solid Geometry or BSplines. All in all, for the typical walls/slabs or beams/columns typically found in the AEC field, an IfcExtrudedAreaSolid is difficult to beat, and that's why the IfcOpenShell API defaults to it. You don't necessarily need to be aware of the low level definition of a wall geometry, but just let the library do the job for you.

In [ ]:
#| exports

south_wall_representation = ios.geometry.add_wall_representation(
    file, context=body, length=storey_size.x + 2 * wall_thickness, height=storey_size.z, 
    thickness=wall_thickness
)
ios.geometry.assign_representation(file, product=south_wall, representation=south_wall_representation)
ios.geometry.edit_object_placement(
    file, product=south_wall, matrix=placement_matrix(
        [-storey_size.x / 2 - wall_thickness, -wall_thickness / 2, 0.]
    )
);  #q4: why a matrix if Y is going to be ignored? why not just pass the placement coords + optionals x_local, z_local and scale?

Placements are assigned to a product (IfcProduct), which in IFC is anything existing within the physical space, like a wall, but also an annotation or a spatial element.

Now, let's add a surface style in order to assign a colour to the wall representation. If eager to learn more about styles and colour in IFC, there's an in-depth explanation by Dion Moult [in this link](https://community.osarch.org/discussion/comment/13744/#Comment_13744), highly recommended.

In [ ]:
#| exports

south_wall_style = ios.style.add_style(file)
ios.style.add_surface_style(
    file, style=south_wall_style, ifc_class='IfcSurfaceStyleShading', attributes=wall_colour.info
)
ios.style.assign_representation_styles(
    file, shape_representation=south_wall_representation, styles=[south_wall_style]
);

## Creating a footing

Let's repeat the same process again to create the structural footing. Creation of the element entity (an IfcElement is just a tangible IfcProduct, like a wall or a footing, but not an annotation or an IfcBuildingStorey), assignation of the storey, creation of the swept solid representation, linkage to the element, setting of the placement and addition of a surface style:

In [ ]:
#| exports

footing = ios.root.create_entity(file, ifc_class='IfcFooting', name='Footing', predefined_type='STRIP_FOOTING')
ios.spatial.assign_container(file, product=footing, relating_structure=storey)
footing_representation = ios.geometry.add_wall_representation(
    file, context=body, length=footing_size.x, height=footing_size.z, thickness=footing_size.y
)
ios.geometry.assign_representation(file, product=footing, representation=footing_representation)
ios.geometry.edit_object_placement(
    file, product=footing, matrix=placement_matrix(
        [-footing_size.x/2, -wall_thickness/2 - footing_ledge, -footing_size.z]
    )
)
footing_style = ios.style.add_style(file)
ios.style.add_surface_style(
    file, style=footing_style, ifc_class='IfcSurfaceStyleShading', attributes=footing_colour.info
)
ios.style.assign_representation_styles(
    file, shape_representation=footing_representation, styles=[footing_style]
);

## Voiding the walls with window openings

The IfcOpenShell API also provides utilities to void elements. An IFC representation is needed, which will correspond to an opening that will void a certain element. It is important to note that in accordance with the schema specs, one opening can only void one element.

When defining the geometric representation of the voids, it is advisable to make them deeper than the element they're voiding (along the perpendicular axis). If they measured exactly the same, computer intrinsic floating point error could leave a thin layer without voiding.

In [ ]:
#| exports

west_void_margin = 0.5
west_opening = ios.root.create_entity(file, ifc_class='IfcOpeningElement')
west_opening_width = 2 * single_window_params['overall_width']
wo_representation = ios.geometry.add_wall_representation(
    file, context=body, 
    length=triple_window_params['overall_width'] + west_void_margin, 
    height=triple_window_params['overall_height'],
    thickness=west_opening_width
)
ios.geometry.assign_representation(file, product=west_opening, representation=wo_representation)
west_opening_coords = [
    (
        -storey_size.x / 2 - wall_thickness - west_void_margin 
        + single_window_params['lining_properties']['LiningOffset']
    ), 
    (
        -west_opening_width / 2 - wall_thickness / 3
        + triple_window_params['lining_properties']['LiningOffset'] 
        + triple_window_params['lining_properties']['LiningDepth']
    ), 
    window_base_height
]
ios.geometry.edit_object_placement(
    file, product=west_opening, matrix=placement_matrix(west_opening_coords)
)
ios.void.add_opening(file, opening=west_opening, element=south_wall)

south_opening = ios.root.create_entity(file, ifc_class='IfcOpeningElement')
south_opening_width = 3.
so_representation = ios.geometry.add_wall_representation(
    file, context=body, length=single_window_params['overall_width'], 
    height=single_window_params['overall_height'], thickness=south_opening_width
)
ios.geometry.assign_representation(file, product=south_opening, representation=so_representation)
ios.geometry.edit_object_placement(
    file, product=south_opening, matrix=placement_matrix(
        [right_window_horizontal_offset, -south_opening_width / 2, window_base_height]
    )
)
ios.void.add_opening(file, opening=south_opening, element=south_wall);

## Creating the roof

The gable roof consists of two slabs, which will be aggregated into the roof entity. Since both slabs are identical, except for rotation and mirroring operations, their IfcSlab entities will point to identical representations, though with different placements. These representations will be again an IfcExtrudedAreaSolid, created by `add_wall_representation`, but in this case the extruded direction will be at a certain angle. That way, the slab prism won't have its faces perpendicular to each other, which better suits a gable roof geometry.

Local axes are set by the calls to `geometry.edit_object_placement`, with a local Z equivalent to global Y. That effectively rotates the representation by 90º so that its previously horizontal "base" and "top" now become vertical. Lastly, north-side slab is also mirrored along the X axis, so that both slabs are facing each other and the gable roof is successfully constructed.

In [ ]:
#| exports

roof = ios.root.create_entity(file, ifc_class='IfcRoof', name='Roof', predefined_type='GABLE_ROOF')
ios.spatial.assign_container(file, product=roof, relating_structure=storey)
roof_representation_south = ios.geometry.add_wall_representation(
    file, context=body, length=roof_size.x, height=roof_size.y / 2, thickness=roof_thickness, 
    x_angle=(roof_angle) * np.pi / 180
)
roof_representation_north = ifcopenshell.util.element.copy_deep(file, roof_representation_south)
#q5: add_slab_representation doesn't accept width and depth? isn't it strange calling it add_wall
#if we're using it for everything? do "add_wall_representation", "add_slab_representation" and 
#"add_profile_representation" add all of them an IfcExtrudedAreaSolid? Would it make any sense to 
#add a single "add_extruded_representation" instead? As a higher level API call than shape_builder

roof_downward_offset = (roof_ledge.y + wall_thickness / 2) * np.tan(roof_angle * np.pi / 180)

south_roof = ios.root.create_entity(file, ifc_class='IfcSlab', name='South roof', predefined_type='ROOF')
ios.geometry.assign_representation(file, product=south_roof, representation=roof_representation_south)
ios.geometry.edit_object_placement(
    file, product=south_roof, matrix=placement_matrix(
        [roof_size.x / 2, -roof_ledge.y - wall_thickness / 2, storey_size.z - roof_downward_offset], 
        x_local=[-1., 0., 0.], z_local=[0., 1., 0.]
    )
)

north_roof = ios.root.create_entity(file, ifc_class='IfcSlab', name='North roof', predefined_type='ROOF')
ios.geometry.assign_representation(file, product=north_roof, representation=roof_representation_north)
ios.geometry.edit_object_placement(
    file, product=north_roof, matrix=placement_matrix(
        [
            -roof_size.x / 2, 
            (storey_size.y + wall_thickness) / 2, 
            storey_size.z - roof_downward_offset + roof_size.z + roof_thickness / roof_angle_cos
        ], x_local=[1., 0., 0.], z_local=[0., 1., 0.]
    )
)

ios.aggregate.assign_object(file, product=south_roof, relating_object=roof)
ios.aggregate.assign_object(file, product=north_roof, relating_object=roof)

roof_style = ios.style.add_style(file)
ios.style.add_surface_style(
    file, style=roof_style, ifc_class='IfcSurfaceStyleShading', attributes=roof_colour.info
)
ios.style.assign_representation_styles(
    file, shape_representation=roof_representation_south, styles=[roof_style]
)
ios.style.assign_representation_styles(
    file, shape_representation=roof_representation_north, styles=[roof_style]
);

## Creating the remaining walls

Since both geometries are identical, we can duplicate the existing south wall representation into a new northern side wall. Again, the placement will position the new representation in the relevant coordinates.

In [ ]:
#| exports

north_wall_representation = ifcopenshell.util.element.copy_deep(file, south_wall_representation)
north_wall = ios.root.create_entity(file, ifc_class='IfcWall', name='North wall', predefined_type='SOLIDWALL')
ios.spatial.assign_container(file, product=north_wall, relating_structure=storey)
ios.geometry.assign_representation(file, product=north_wall, representation=north_wall_representation)
ios.geometry.edit_object_placement(
    file, product=north_wall, matrix=placement_matrix(
        [-storey_size.x/2 - wall_thickness, storey_size.y + wall_thickness / 2, 0.]
    )
);

A new east wall entity is added, along with its representation. This time, though, the geometry is not just an extruded rectangle, but due to the gable roof shape, a pair of clipping operations need to be performed on top. The IfcOpenShell API easily allows us to do so by passing some planes with the `clippings` argument to `add_wall_representation`.

In [ ]:
#| exports

east_wall = ios.root.create_entity(
    file, ifc_class='IfcWall', name='East wall', predefined_type='SOLIDWALL'
)
ios.spatial.assign_container(file, product=east_wall, relating_structure=storey)

south_roof_clipping_matrix = clipping_matrix(
    [0., wall_thickness / 2, storey_size.z], x_dir=[1., 0., 0.], 
    z_dir=[0., -roof_angle_sin, roof_angle_cos]
)
north_roof_clipping_matrix = clipping_matrix(
    [0., storey_size.y + 3 / 2 * wall_thickness, storey_size.z], x_dir=[1., 0., 0.], 
    z_dir=[0., roof_angle_sin, roof_angle_cos]
)

roof_clippings = [
    {
        'type': 'IfcBooleanClippingResult', 'operand_type': 'IfcHalfSpaceSolid', 'matrix': matrix
    } for matrix in [south_roof_clipping_matrix, north_roof_clipping_matrix]
]

east_wall_representation = ios.geometry.add_wall_representation(
    file, context=body, length=wall_thickness, height=storey_size.z + roof_size.z, 
    thickness=storey_size.y + 2 * wall_thickness, clippings=roof_clippings
)

ios.geometry.assign_representation(file, product=east_wall, representation=east_wall_representation)
ios.geometry.edit_object_placement(
    file, product=east_wall, matrix=placement_matrix([storey_size.x / 2, -wall_thickness / 2, 0.])
);

The methodology of duplicating a representation is is once more time followed for the remaining wall, and an opening is defined on it. Analogously, the opening is exactly equal to the previously defined `west_opening`. However, since in IFC, an opening can only be used to create a single void within a single element, a new identical opening entity is created.

In [ ]:
#| exports

west_wall = ios.root.create_entity(
    file, ifc_class='IfcWall', name='West wall', predefined_type='SOLIDWALL'
)
ios.spatial.assign_container(file, product=west_wall, relating_structure=storey)

west_wall_representation = ifcopenshell.util.element.copy_deep(file, east_wall_representation)
ios.geometry.assign_representation(file, product=west_wall, representation=west_wall_representation)
ios.geometry.edit_object_placement(
    file, product=west_wall, matrix=placement_matrix(
        [-storey_size.x / 2 - wall_thickness, -wall_thickness / 2, 0.]
    )
)

west_opening_copy = ifcopenshell.util.element.copy_deep(file, west_opening)
ios.geometry.edit_object_placement(
    file, product=west_opening_copy, matrix=placement_matrix(west_opening_coords)
)
ios.void.add_opening(file, opening=west_opening_copy, element=west_wall);

We also indicate that walls are connected between each other by means of `geometry.connect_path`.

In [ ]:
#| exports

connection_args = {'relating_connection': 'ATSTART', 'related_connection': 'ATEND'}

rel_connect_paths = [
    ios.geometry.connect_path(
        file, relating_element=south_wall, related_element=east_wall, **connection_args
    ),
    ios.geometry.connect_path(
        file, relating_element=east_wall, related_element=north_wall, **connection_args
    ),
    ios.geometry.connect_path(
        file, relating_element=north_wall, related_element=west_wall, **connection_args
    ),
    ios.geometry.connect_path(
        file, relating_element=west_wall, related_element=south_wall, **connection_args
    )
]

#q7: do IfcRelConnectsPathElements with ConnectionGeometry work in any viewer? is this done like this?
# Original IfcOpenHouse had half a wall thickness less of extension per wall end, I bet it's better for
# qto's, but how is the way to properly make connections and to have a proper viz of them in IFC?
point_list = file.create_entity('IfcCartesianPointList2D', CoordList = [[-1., -1.], [1., 1.]])
curve_on_relating = file.create_entity('IfcIndexedPolyCurve', Points=point_list)
connection_curve = file.create_entity(
    'IfcConnectionCurveGeometry', CurveOnRelatingElement=curve_on_relating
)

for path in rel_connect_paths:
    path.ConnectionGeometry = connection_curve

## Creating the terrain as a NURBS surface

This section refers to the original terrain of the IfcOpenHouse, defined by a NURBS surface. The variable `terrain_build_method` can be set to one of three enum values:
- `TerrainBuildMethod.NATIVE_BSPLINE`: Use the native IfcBSplineSurfaceWithKnots entity, existing from IFC4 onwards.
- `TerrainBuildMethod.TESSELATE_OCC_SHAPE`: Construct the shape in Open Cascade and tesselate it with the help of IfcOpenShell.
- `TerrainBuildMethod.NONE`: You may also safely skip the terrain part.

The terrain with a fancy NURBS surface may be challenging to be correctly displayed within most IFC viewers. On the other hand, the tesselation method was followed on the original IfcOpenHouse for IFC2x3, but note that it requires the installation of Open Cascade (`pythonocc-core` into your Python environment).

You may just choose to ignore the fancy terrain part, since it's the least connected with real use cases in the industry.

In [ ]:
#| exports

terrain_control_points = [  # obtained from the original IfcOpenHouse
    [( -10., -10., -4.13), ( -10., -4.33, -4.13), (-10., 0., -5.13), ( -10., 4.33, -7.13), ( -10., 10., -7.13)],
    [(-3.33, -10., -5.13), (-7.67, -3.67,    5.), ( -9., 0.,    1.), (-7.67, 7.67,    6.), (-3.33, 10., -4.13)],
    [(   0., -10., -5.53), (   0., -3.67,    3.), (  0., 0.,  -12.), (   0., 7.67,   1.5), (   0., 10., -4.13)],
    [( 3.33, -10., -6.13), ( 7.67, -3.67,    6.), (  9., 0.,    5.), ( 7.67,   9.,    7.), ( 3.33, 10., -4.13)],
    [(  10., -10., -6.13), (  10., -4.33, -5.13), ( 10., 0., -4.13), (  10., 4.33, -4.13), (  10., 10., -8.13)]
]

degree, multiplicity = 4, 5

if terrain_build_method == TerrainBuildMethod.NATIVE_BSPLINE:
    # https://ifc43-docs.standards.buildingsmart.org/IFC/RELEASE/IFC4x3/HTML/lexical/IfcAdvancedBrep.htm
    # Advanced BREPs are complex and there is no high level API for now, so feel free to skip
    
    #q8: where can I check if I have done this right? any viewer showing IfcBSplineSurfaceWithKnots?
    
    terrain_representation = build_native_bspline_terrain(
        file, body, terrain_control_points, degree, multiplicity
    )
    
elif terrain_build_method == TerrainBuildMethod.TESSELATE_OCC_SHAPE:
    # Open Cascade is also complex, feel free to skip as well
    
    terrain_representation = build_tesselated_occ_terrain(
        file, body, terrain_control_points, degree, multiplicity
    )

if terrain_build_method != TerrainBuildMethod.NONE:
    # If we have produced an IfcShapeRepresentation, by any of the previous methods, we assign it
    # to the IfcSite and also assign it a style.
    
    ios.geometry.assign_representation(file, product=site, representation=terrain_representation)
    terrain_style = ios.style.add_style(file)
    ios.style.add_surface_style(
        file, style=terrain_style, ifc_class='IfcSurfaceStyleShading', attributes=terrain_colour.info
    )
    ios.style.assign_representation_styles(
        file, shape_representation=terrain_representation, styles=[terrain_style]
    );

## Creation of material layer sets and usages

Firstly, an IfcMaterialLayerSet is added, with a unique brick layer consisting of a newly defined brick material for the entire wall thickness. If there were walls consisting of more layers, they could be simply added with the help of `material.add_layer`.

Then, a new IfcMaterialLayerSetUsage is defined on each wall. This usage would potentially allow to set different attributes, even if the underlying layer set was the same. [More info](https://ifc43-docs.standards.buildingsmart.org/IFC/RELEASE/IFC4x3/HTML/concepts/Object_Association/Material_Association/Material_Layer_Set_Usage/content.html).

In [ ]:
#| exports

brick = ios.material.add_material(file, name='Brick', category='brick')
wall_layerset = ios.material.add_material_set(file, name='Wall', set_type='IfcMaterialLayerSet')
brick_layer = ios.material.add_layer(file, layer_set=wall_layerset, material=brick)
ios.material.edit_layer(file, layer=brick_layer, attributes={'LayerThickness': wall_thickness})

#q9: This is a copy from the original IfcOpenHouse, but I guess in current IFC usages without types are 
# not a good idea, right? Do I add types?

for wall in [south_wall, north_wall, east_wall, west_wall]:
    rel_usage_south_wall = ios.material.assign_material(file, product=wall, type='IfcMaterialLayerSetUsage')
    ios.material.edit_layer_usage(
        file, usage=rel_usage_south_wall.RelatingMaterial, attributes={
            'ForLayerSet': wall_layerset, 'OffsetFromReferenceLine': 0.
        }
    )

## Creation of a stair flight

Similarly to the previously created entities, a new IfcStairFlight is created, in this case with an extruded profile representation.

In [ ]:
#| exports

stair_flight_params = {'NumberOfRisers': 2, 'NumberOfTreads': 2, 'RiserHeight': 0.2, 'TreadLength': 0.25}
stair_flight = ios.root.create_entity(file, ifc_class='IfcStairFlight', predefined_type='STRAIGHT')
ios.spatial.assign_container(file, product=stair_flight, relating_structure=storey)
for attr, value in stair_flight_params.items():
    setattr(stair_flight, attr, value)  # the entity is monkey-patched with its IFC compliant attributes

stair_points = []  # Staircase cross-sectional profile 2D points
for step in range(stair_flight_params['NumberOfRisers'] + 1):
    next_step = 0 if step == stair_flight_params['NumberOfRisers'] else step + 1
    stair_points.extend([
        (step * stair_flight_params['TreadLength'], step * stair_flight_params['RiserHeight']),
        (next_step * stair_flight_params['TreadLength'], step * stair_flight_params['RiserHeight']),
    ])

stair_flight_curve = file.create_entity(
    'IfcIndexedPolyCurve', file.create_entity('IfcCartesianPointList2D', stair_points), None, False
)
stair_flight_profile = file.create_entity('IfcArbitraryClosedProfileDef', 'AREA', None, stair_flight_curve)
stair_flight_representation = ios.geometry.add_profile_representation(
    file, context=body, profile=stair_flight_profile, depth=stair_width
)

ios.geometry.assign_representation(file, product=stair_flight, representation=stair_flight_representation)
ios.geometry.edit_object_placement(
    file, product=stair_flight, matrix=placement_matrix(
        [footing_size.x / 2, door_horizontal_offset + (door_params['overall_width'] - stair_width) / 2, 0.],
        z_local=[0., 1., 0.]
    )
)

ios.style.assign_representation_styles(  # same style as the footing
    file, shape_representation=stair_flight_representation, styles=[footing_style]
);

## The house needs a door...

For the door creation, we can rely on `geometry.add_door_representation`, which will produce a high quality representation and is highly customizable.

Note that when this API function is used outside of Blender, it requires the `mathutils` package to be installed within the Python environment.

In [ ]:
#| exports

door = ios.root.create_entity(file, ifc_class='IfcDoor', name='Main door', predefined_type='DOOR')
ios.spatial.assign_container(file, product=door, relating_structure=storey)

door_opening = ios.root.create_entity(file, ifc_class='IfcOpeningElement')
door_opening_representation = ios.geometry.add_wall_representation(
    file, context=body, length=door_params['overall_width'], height=door_params['overall_height'], 
    thickness=door_params['overall_width']
)
ios.geometry.assign_representation(
    file, product=door_opening, representation=door_opening_representation
)
ios.geometry.edit_object_placement(
    file, product=door_opening, matrix=placement_matrix(
        [storey_size.x / 2 - door_params['overall_width'] / 2 , door_horizontal_offset, 0.]
    )
)
ios.void.add_opening(file, opening=door_opening, element=east_wall)

door_representation = ios.geometry.add_door_representation(  # requires mathutils
    file, context=body, **door_params
)
ios.geometry.edit_object_placement(
    file, product=door, matrix=placement_matrix(
        [storey_size.x / 2 + wall_thickness / 4, door_horizontal_offset, 0.],
        x_local=[0., 1., 0.]  # door is rotated into the east wall
    )
)
ios.geometry.assign_representation(file, product=door, representation=door_representation)
ios.void.add_filling(file, opening=door_opening, element=door)

door_style = ios.style.add_style(file)
ios.style.add_surface_style(
    file, style=door_style, ifc_class='IfcSurfaceStyleShading', attributes=door_colour.info
)
ios.style.assign_representation_styles(
    file, shape_representation=door_representation, styles=[door_style]
);

## ... and some windows

Window representations rely on the `geometry.add_window_representation` call, which is also highly customizable. It also depend on the `mathutils` package.

In [ ]:
#| exports

window_right = ios.root.create_entity(
    file, ifc_class='IfcWindow', name='Right window', predefined_type='WINDOW'
)
window_right.PartitioningType = single_window_params['partition_type']  #q6: couldn't this fit into the previous call?
window_right_representation = ios.geometry.add_window_representation(  # requires mathutils
    file, context=body, **single_window_params
)
ios.spatial.assign_container(file, product=window_right, relating_structure=storey)
ios.geometry.edit_object_placement(
    file, product=window_right, matrix=placement_matrix(
        [right_window_horizontal_offset, -wall_thickness / 3, window_base_height]
    )
)
ios.geometry.assign_representation(
    file, product=window_right, representation=window_right_representation
)
ios.void.add_filling(file, opening=south_opening, element=window_right)

window_west = ios.root.create_entity(
    file, ifc_class='IfcWindow', name='West window', predefined_type='WINDOW'
)
window_west.PartitioningType = single_window_params['partition_type']
window_west_representation = ios.geometry.add_window_representation(
    file, context=body, **single_window_params
)
ios.spatial.assign_container(file, product=window_west, relating_structure=storey)
ios.geometry.edit_object_placement(
    file, product=window_west, matrix=placement_matrix(
        [
            -storey_size.x / 2 - wall_thickness,
            (
                + single_window_params['overall_width'] - wall_thickness / 3 
                + triple_window_params['lining_properties']['LiningOffset'] 
                + triple_window_params['lining_properties']['LiningDepth']
            ), 
            window_base_height
        ], x_local=[0., -1., 0.]
    )
)
ios.geometry.assign_representation(
    file, product=window_west, representation=window_west_representation
)
ios.void.add_filling(file, opening=west_opening_copy, element=window_west)

window_left = ios.root.create_entity(
    file, ifc_class='IfcWindow', name='Left Window', predefined_type='WINDOW'
)
window_left.PartitioningType = triple_window_params['partition_type']
window_left_representation = ios.geometry.add_window_representation(
    file, context=body, **triple_window_params
)
ios.spatial.assign_container(file, product=window_left, relating_structure=storey)
ios.geometry.edit_object_placement(
    file, product=window_left, matrix=placement_matrix(
        [
            (
                -storey_size.x / 2 - wall_thickness
                + single_window_params['lining_properties']['LiningOffset']
            ),
            -wall_thickness / 3,
            window_base_height]
    )
)
ios.geometry.assign_representation(
    file, product=window_left, representation=window_left_representation
)
ios.void.add_filling(file, opening=west_opening, element=window_left)

window_style = ios.style.add_style(file)
ios.style.add_surface_style(
    file, style=window_style, ifc_class='IfcSurfaceStyleShading', attributes=window_colour.info
)
ios.style.assign_representation_styles( #q10: Will it be possible to assign different styles to the panel and the lining?
    file, shape_representation=window_right_representation, styles=[window_style]
)
ios.style.assign_representation_styles(
    file, shape_representation=window_west_representation, styles=[window_style]
)
ios.style.assign_representation_styles(
    file, shape_representation=window_left_representation, styles=[window_style]
);

## Checking the validity of our file

After all the previous steps, we have programmatically defined an IFC file. But, is it valid? have we messed up at some point? IfcOpenShell provides the `validate` method to validate a certain file against its corresponding schema:

In [ ]:
#| exports

json_logger = ifcopenshell.validate.json_logger()
ifcopenshell.validate.validate(file, json_logger)

# Showing only first 3 here
json_logger.statements[:min(3, len(json_logger.statements))]

[{'level': 'error',
  'message': 'Attribute not optional',
  'type': 'schema',
  'instance': #3075=IfcIndexedPolyCurve(#3076,(IfcLineIndex((1,2)),IfcLineIndex((2,3)),IfcLineIndex((3,4)),IfcLineIndex((4,1))),$),
  'attribute': 'IfcIndexedPolyCurve.SelfIntersect'},
 {'level': 'error',
  'message': 'Attribute not optional',
  'type': 'schema',
  'instance': #3067=IfcIndexedPolyCurve(#3066,(IfcLineIndex((1,2)),IfcLineIndex((2,3)),IfcLineIndex((3,4)),IfcLineIndex((4,1))),$),
  'attribute': 'IfcIndexedPolyCurve.SelfIntersect'},
 {'level': 'error',
  'message': 'Attribute not optional',
  'type': 'schema',
  'instance': #3065=IfcIndexedPolyCurve(#3064,(IfcLineIndex((1,2)),IfcLineIndex((2,3)),IfcLineIndex((3,4)),IfcLineIndex((4,1))),$),
  'attribute': 'IfcIndexedPolyCurve.SelfIntersect'}]

Let's check how many different validation issues we get:

In [ ]:
#| exports

set([issue['attribute'] for issue in json_logger.statements])

{'IfcIndexedPolyCurve.SelfIntersect', 'IfcMaterialLayerSet.MaterialLayers'}

Refer to the notebook version, if you wish, for a debugging example using `json_logger.statements`.

In [ ]:
#| hide
# Example of a debugging process to traverse the file in search of products which ultimately 
# have a representation with an IfcIndexedPolyCurve without a defined SelfIntersect

def find_inverses(file, entity, ifc_class, attr, relation=lambda a, b: a == b):
    return [e for e in file.by_type(ifc_class) if relation(entity, getattr(e, attr))]

pcurve_results = defaultdict(list)

for issue in json_logger.statements:
    if issue['attribute'] == 'IfcIndexedPolyCurve.SelfIntersect':
        indexed_polycurve = issue['instance']
        txt = f'--> {indexed_polycurve.is_a()} #{indexed_polycurve.id()}: '
        profile_with_voids = find_inverses(
            file, indexed_polycurve, 'IfcArbitraryProfileDefWithVoids', 'InnerCurves', relation=lambda a, b: a in b
        )
        if len(profile_with_voids) == 0:
            profile_with_voids = find_inverses(
                file, indexed_polycurve, 'IfcArbitraryClosedProfileDef', 'OuterCurve'
            )
        if len(profile_with_voids) == 0:        
            print(f'{txt}No links found')
            continue
        swept_area = find_inverses(file, profile_with_voids[0], 'IfcSweptAreaSolid', 'SweptArea')
        if len(swept_area) == 0:
            print(f'{txt}No swept area found for {profile_with_voids[0].is_a()} #{profile_with_voids[0].id()}')
            continue
        shape_representation = find_inverses(
            file, swept_area[0], 'IfcShapeRepresentation', 'Items', relation=lambda a, b: a in b
        )
        if len(shape_representation) == 0:
            print(f'{txt}No shape representation found for {swept_area[0].is_a()} #{swept_area[0].id()}')
            continue
        product_representation = shape_representation[0].OfProductRepresentation
        if len(product_representation) == 0:
            print(f'{txt}No product representation found for {shape_representation[0].is_a()} #{shape_representation[0].id()}')
            continue
        product = product_representation[0].ShapeOfProduct
        if len(product) == 0:
            print(f'{txt}No product found for {product_representation[0].is_a()} #{product_representation[0].id()}')
            continue
        pcurve_results[product[0]].append(indexed_polycurve)

for product, polycurves in pcurve_results.items():
    pcurve_txt = ', '.join([f'{p.is_a()} #{p.id()}' for p in polycurves])
    print(f'--> Product {product.is_a()} #{product.id()} is pointed to by {pcurve_txt}.')
                    

--> IfcIndexedPolyCurve #349: No links found
--> Product IfcWindow #2957 is pointed to by IfcIndexedPolyCurve #3075, IfcIndexedPolyCurve #3067, IfcIndexedPolyCurve #3065, IfcIndexedPolyCurve #3056, IfcIndexedPolyCurve #3054, IfcIndexedPolyCurve #3045, IfcIndexedPolyCurve #3043, IfcIndexedPolyCurve #3033, IfcIndexedPolyCurve #3025, IfcIndexedPolyCurve #3023, IfcIndexedPolyCurve #3014, IfcIndexedPolyCurve #3012, IfcIndexedPolyCurve #3003, IfcIndexedPolyCurve #3001, IfcIndexedPolyCurve #2991, IfcIndexedPolyCurve #2983, IfcIndexedPolyCurve #2981, IfcIndexedPolyCurve #2972, IfcIndexedPolyCurve #2970, IfcIndexedPolyCurve #2961, IfcIndexedPolyCurve #2959.
--> Product IfcWindow #2905 is pointed to by IfcIndexedPolyCurve #2939, IfcIndexedPolyCurve #2931, IfcIndexedPolyCurve #2929, IfcIndexedPolyCurve #2920, IfcIndexedPolyCurve #2918, IfcIndexedPolyCurve #2909, IfcIndexedPolyCurve #2907.
--> Product IfcWindow #2853 is pointed to by IfcIndexedPolyCurve #2887, IfcIndexedPolyCurve #2879, IfcIndexed

## Visualization with IFC.js

It is worth noting that a Jupyter Notebook runs in the browser, and is therefore subject to the laws of web development. Moreover, a couple of handy "magic methods" `%%html` and `%%js` can be placed at the start of a code cell to run HTML or JavaScript. Thanks to it, we can create a new `div` and use a simple prebundled script using [IFC.js](https://ifcjs.github.io/info/) to render our brand new IFC on it. You may have a look at the simple JS script [here](https://github.com/cvillagrasa/IfcOpenHouse/blob/master/viz/nb_viz.js).

In [ ]:
%%html
<!--Run this cell only once, otherwise the bundle will attempt to redeclarate objects-->
<div id="ifcjs-container"></div>
<script src="https://cdn.jsdelivr.net/gh/cvillagrasa/IfcOpenHouse@latest/viz/bundle_nb.js" />

In [ ]:
%%js
//# Run this cell to load the IfcOpenShell file with IFC.js in the div above
//# Rerun cells as needed and run this cell again to reload viz
//# Do not rerun the cell above, or the bundle will attempt to redeclarate objects

async function reloadIfc(out){
    let ifcStr = out.content.data["text/plain"];
    console.log(ifcStr);
    loadIfc(null, ifcStr);
}

let callbacks = {iopub: {output: reloadIfc}};
IPython.notebook.kernel.execute("file.wrapped_data.to_string()", callbacks, {silent: false});
//# IPython.notebook.kernel.execute("test_ifc_str", callbacks, {silent: false});  # only for debugging

<IPython.core.display.Javascript object>

In [ ]:
#| hide
# Example IFC handily stored as a string, for debugging purposes

test_ifc_str = """ISO-10303-21;\nHEADER;\nFILE_DESCRIPTION(('ViewDefinition[DesignTransferView]'),'2;1');\nFILE_NAME('/dev/null','2023-04-28T12:24:05+02:00',(),(),'IfcOpenShell 0.7.0','IfcOpenShell 0.7.0','Nobody');\nFILE_SCHEMA(('IFC4X3'));\nENDSEC;\nDATA;\n#1=IFCPROJECT('3rB1uMZhz90eXZFMLwVnN6',$,'IFC Open House',$,$,$,$,(#11),#6);\n#2=IFCRELDECLARES('2bYTIxAVvBVxk5aQOyIs3U',$,$,$,#1,(#1));\n#3=IFCSIUNIT(*,.LENGTHUNIT.,$,.METRE.);\n#4=IFCSIUNIT(*,.AREAUNIT.,$,.SQUARE_METRE.);\n#5=IFCSIUNIT(*,.VOLUMEUNIT.,$,.CUBIC_METRE.);\n#6=IFCUNITASSIGNMENT((#3,#4,#5));\n#7=IFCCARTESIANPOINT((0.,0.,0.));\n#8=IFCDIRECTION((0.,0.,1.));\n#9=IFCDIRECTION((1.,0.,0.));\n#10=IFCAXIS2PLACEMENT3D(#7,#8,#9);\n#11=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#10,$);\n#12=IFCGEOMETRICREPRESENTATIONSUBCONTEXT('Body','Model',*,*,*,*,#11,$,.MODEL_VIEW.,$);\n#13=IFCACTORROLE(.USERDEFINED.,'CONTRIBUTOR',$);\n#14=IFCTELECOMADDRESS(.USERDEFINED.,$,'WEBPAGE',$,$,$,$,'https://ifcopenshell.org',$);\n#15=IFCORGANIZATION('IfcOpenShell','IfcOpenShell','IfcOpenShell is an open source software library that helps users and software developers to work with IFC data.',(#13),(#14));\n#16=IFCAPPLICATION(#15,'0.7.0','IfcOpenShell','IfcOpenShell');\n#17=IFCPERSON('CV','V','Carlos',$,$,$,$,$);\n#18=IFCORGANIZATION('OSArch','OSArch',$,(#20),$);\n#19=IFCPERSONANDORGANIZATION(#17,#18,$);\n#20=IFCACTORROLE(.CIVILENGINEER.,$,$);\n#21=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#22=IFCACTOR('3eQcYDc7L6P9H8RN8l5kc4',#21,$,$,$,#19);\n#23=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677454,#19,#16,1682677446);\n#24=IFCSITE('0QL5wOz8b7efNJs28yCa1q',#23,'OSArch Land',$,$,$,#2695,$,$,$,$,$,$,$);\n#25=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#26=IFCRELAGGREGATES('0uEbvcwqf31OUlhhk7I_Kr',#25,$,$,#1,(#24));\n#27=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#28=IFCBUILDING('22YD9CmDb3v8PIDualZGM7',#27,'Open house',$,$,$,$,$,$,$,$,$);\n#29=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#30=IFCRELAGGREGATES('3_BBjTl1z9rv8k3tWIxuWe',#29,$,$,#24,(#28));\n#31=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#32=IFCBUILDINGSTOREY('3GUbQdBgr9DRLyyHybHsQC',#31,'Ground floor',$,$,$,$,$,$,$);\n#33=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#34=IFCRELAGGREGATES('3XCMiChmL9FfMaSiQ0gwOl',#33,$,$,#28,(#32));\n#35=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#36=IFCPROPERTYSET('3C9ZkbU$X858kBy08Os$cB',#35,'Pset_SiteCommon',$,(#39));\n#37=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677446,#19,#16,1682677446);\n#38=IFCRELDEFINESBYPROPERTIES('0llQofb_9BGO1UTTkkGH11',#37,$,$,(#24),#36);\n#39=IFCPROPERTYSINGLEVALUE('TotalArea',$,IFCAREAMEASURE(400.),$);\n#40=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677449,#19,#16,1682677447);\n#41=IFCWALL('391nfOIzT8xgudcHu7uw31',#40,'South wall','This is my first wall with the IfcOpenShell Python API!',$,#59,#54,$,.SOLIDWALL.);\n#42=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677457,#19,#16,1682677449);\n#43=IFCRELCONTAINEDINSPATIALSTRUCTURE('0uXnjI0_fDz8TKCoTnPgoM',#42,$,$,(#41,#65,#137,#214,#222,#254,#2716,#2735,#2853,#2905,#2957),#32);\n#44=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,0.36),(10.72,0.36),(10.72,0.),(0.,0.)),$);\n#45=IFCINDEXEDPOLYCURVE(#44,$,.F.);\n#46=IFCDIRECTION((0.,0.,1.));\n#47=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#45);\n#48=IFCCARTESIANPOINT((0.,0.,0.));\n#49=IFCDIRECTION((0.,0.,1.));\n#50=IFCDIRECTION((1.,0.,0.));\n#51=IFCAXIS2PLACEMENT3D(#48,#49,#50);\n#52=IFCEXTRUDEDAREASOLID(#47,#51,#46,3.);\n#53=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#52));\n#54=IFCPRODUCTDEFINITIONSHAPE($,$,(#53));\n#55=IFCCARTESIANPOINT((-5.36,-0.18,0.));\n#56=IFCDIRECTION((0.,0.,1.));\n#57=IFCDIRECTION((1.,0.,0.));\n#58=IFCAXIS2PLACEMENT3D(#55,#56,#57);\n#59=IFCLOCALPLACEMENT($,#58);\n#60=IFCSURFACESTYLE($,.BOTH.,(#61));\n#61=IFCSURFACESTYLESHADING(#62,0.);\n#62=IFCCOLOURRGB($,0.75,0.73,0.68);\n#63=IFCSTYLEDITEM(#52,(#60),$);\n#64=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677450,#19,#16,1682677450);\n#65=IFCFOOTING('2eTnUhS752mvpl$6479YXY',#64,'Footing',$,$,#81,#76,$,.STRIP_FOOTING.);\n#66=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,5.82),(10.82,5.82),(10.82,0.),(0.,0.)),$);\n#67=IFCINDEXEDPOLYCURVE(#66,$,.F.);\n#68=IFCDIRECTION((0.,0.,1.));\n#69=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#67);\n#70=IFCCARTESIANPOINT((0.,0.,0.));\n#71=IFCDIRECTION((0.,0.,1.));\n#72=IFCDIRECTION((1.,0.,0.));\n#73=IFCAXIS2PLACEMENT3D(#70,#71,#72);\n#74=IFCEXTRUDEDAREASOLID(#69,#73,#68,2.);\n#75=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#74));\n#76=IFCPRODUCTDEFINITIONSHAPE($,$,(#75));\n#77=IFCCARTESIANPOINT((-5.41,-0.23,-2.));\n#78=IFCDIRECTION((0.,0.,1.));\n#79=IFCDIRECTION((1.,0.,0.));\n#80=IFCAXIS2PLACEMENT3D(#77,#78,#79);\n#81=IFCLOCALPLACEMENT($,#80);\n#82=IFCSURFACESTYLE($,.BOTH.,(#83));\n#83=IFCSURFACESTYLESHADING(#84,0.);\n#84=IFCCOLOURRGB($,0.38,0.4,0.42);\n#85=IFCSTYLEDITEM(#74,(#82),$);\n#86=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677451,#19,#16,1682677451);\n#87=IFCOPENINGELEMENT('3bFVCMh45E2xGS_Abnot5S',#86,$,$,$,#110,#98,$,$);\n#88=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,3.72),(6.,3.72),(6.,0.),(0.,0.)),$);\n#89=IFCINDEXEDPOLYCURVE(#88,$,.F.);\n#90=IFCDIRECTION((0.,0.,1.));\n#91=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#89);\n#92=IFCCARTESIANPOINT((0.,0.,0.));\n#93=IFCDIRECTION((0.,0.,1.));\n#94=IFCDIRECTION((1.,0.,0.));\n#95=IFCAXIS2PLACEMENT3D(#92,#93,#94);\n#96=IFCEXTRUDEDAREASOLID(#91,#95,#90,1.6);\n#97=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#96));\n#98=IFCPRODUCTDEFINITIONSHAPE($,$,(#97));\n#104=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677451,#19,#16,1682677451);\n#105=IFCRELVOIDSELEMENT('1wCd63KXT4oOOK2EWOv7JK',#104,$,$,#41,#87);\n#106=IFCCARTESIANPOINT((-0.45,-1.7,0.4));\n#107=IFCDIRECTION((0.,0.,1.));\n#108=IFCDIRECTION((1.,0.,0.));\n#109=IFCAXIS2PLACEMENT3D(#106,#107,#108);\n#110=IFCLOCALPLACEMENT(#59,#109);\n#111=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677451,#19,#16,1682677451);\n#112=IFCOPENINGELEMENT('0CSXIJZW15sPk5pvSNHlIU',#111,$,$,$,#135,#123,$,$);\n#113=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,3.),(1.86,3.),(1.86,0.),(0.,0.)),$);\n#114=IFCINDEXEDPOLYCURVE(#113,$,.F.);\n#115=IFCDIRECTION((0.,0.,1.));\n#116=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#114);\n#117=IFCCARTESIANPOINT((0.,0.,0.));\n#118=IFCDIRECTION((0.,0.,1.));\n#119=IFCDIRECTION((1.,0.,0.));\n#120=IFCAXIS2PLACEMENT3D(#117,#118,#119);\n#121=IFCEXTRUDEDAREASOLID(#116,#120,#115,1.6);\n#122=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#121));\n#123=IFCPRODUCTDEFINITIONSHAPE($,$,(#122));\n#129=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677451,#19,#16,1682677451);\n#130=IFCRELVOIDSELEMENT('3dgj6sYmL32AcHgYK$lolp',#129,$,$,#41,#112);\n#131=IFCCARTESIANPOINT((7.36,-1.32,0.4));\n#132=IFCDIRECTION((0.,0.,1.));\n#133=IFCDIRECTION((1.,0.,0.));\n#134=IFCAXIS2PLACEMENT3D(#131,#132,#133);\n#135=IFCLOCALPLACEMENT(#59,#134);\n#136=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677451,#19,#16,1682677451);\n#137=IFCROOF('0KaQ5N4sT5hfZ3FohF9zs8',#136,'Roof',$,$,$,$,$,.GABLE_ROOF.);\n#138=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,0.439478851954124),(10.92,0.439478851954124),(10.92,0.),(0.,0.)),$);\n#139=IFCINDEXEDPOLYCURVE(#138,$,.F.);\n#140=IFCDIRECTION((0.,0.573576436351046,0.819152044288992));\n#141=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#139);\n#142=IFCCARTESIANPOINT((0.,0.,0.));\n#143=IFCDIRECTION((0.,0.,1.));\n#144=IFCDIRECTION((1.,0.,0.));\n#145=IFCAXIS2PLACEMENT3D(#142,#143,#144);\n#146=IFCEXTRUDEDAREASOLID(#141,#145,#140,3.75998573338528);\n#147=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#146));\n#148=IFCSHAPEREPRESENTATION(#149,'Body','SweptSolid',(#155));\n#149=IFCGEOMETRICREPRESENTATIONSUBCONTEXT('Body','Model',*,*,*,*,#150,$,.MODEL_VIEW.,$);\n#150=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#151,$);\n#151=IFCAXIS2PLACEMENT3D(#152,#153,#154);\n#152=IFCCARTESIANPOINT((0.,0.,0.));\n#153=IFCDIRECTION((0.,0.,1.));\n#154=IFCDIRECTION((1.,0.,0.));\n#155=IFCEXTRUDEDAREASOLID(#156,#159,#163,3.75998573338528);\n#156=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#157);\n#157=IFCINDEXEDPOLYCURVE(#158,$,.F.);\n#158=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,0.439478851954124),(10.92,0.439478851954124),(10.92,0.),(0.,0.)),$);\n#159=IFCAXIS2PLACEMENT3D(#160,#161,#162);\n#160=IFCCARTESIANPOINT((0.,0.,0.));\n#161=IFCDIRECTION((0.,0.,1.));\n#162=IFCDIRECTION((1.,0.,0.));\n#163=IFCDIRECTION((0.,0.573576436351046,0.819152044288992));\n#164=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677451,#19,#16,1682677451);\n#165=IFCSLAB('182uPRLlzFDwL_pUdKO3_p',#164,'South roof',$,$,#186,#166,$,.ROOF.);\n#166=IFCPRODUCTDEFINITIONSHAPE($,$,(#147));\n#172=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677451,#19,#16,1682677451);\n#173=IFCSLAB('1wVoOo52zFaQm_yUxX5bGt',#172,'North roof',$,$,#191,#174,$,.ROOF.);\n#174=IFCPRODUCTDEFINITIONSHAPE($,$,(#148));\n#180=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677451,#19,#16,1682677451);\n#181=IFCRELAGGREGATES('2zJvkoNR91mRYraUcLAazG',#180,$,$,#137,(#173,#165));\n#182=IFCCARTESIANPOINT((5.46,-0.4,2.71991698471612));\n#183=IFCDIRECTION((0.,1.,0.));\n#184=IFCDIRECTION((-1.,0.,0.));\n#185=IFCAXIS2PLACEMENT3D(#182,#183,#184);\n#186=IFCLOCALPLACEMENT($,#185);\n#187=IFCCARTESIANPOINT((-5.46,2.68,5.31603505435615));\n#188=IFCDIRECTION((0.,1.,0.));\n#189=IFCDIRECTION((1.,0.,0.));\n#190=IFCAXIS2PLACEMENT3D(#187,#188,#189);\n#191=IFCLOCALPLACEMENT($,#190);\n#192=IFCSURFACESTYLE($,.BOTH.,(#193));\n#193=IFCSURFACESTYLESHADING(#194,0.);\n#194=IFCCOLOURRGB($,0.24,0.08,0.04);\n#195=IFCSTYLEDITEM(#146,(#192),$);\n#196=IFCSTYLEDITEM(#155,(#192),$);\n#197=IFCSHAPEREPRESENTATION(#198,'Body','SweptSolid',(#204));\n#198=IFCGEOMETRICREPRESENTATIONSUBCONTEXT('Body','Model',*,*,*,*,#199,$,.MODEL_VIEW.,$);\n#199=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#200,$);\n#200=IFCAXIS2PLACEMENT3D(#201,#202,#203);\n#201=IFCCARTESIANPOINT((0.,0.,0.));\n#202=IFCDIRECTION((0.,0.,1.));\n#203=IFCDIRECTION((1.,0.,0.));\n#204=IFCEXTRUDEDAREASOLID(#205,#208,#212,3.);\n#205=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#206);\n#206=IFCINDEXEDPOLYCURVE(#207,$,.F.);\n#207=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,0.36),(10.72,0.36),(10.72,0.),(0.,0.)),$);\n#208=IFCAXIS2PLACEMENT3D(#209,#210,#211);\n#209=IFCCARTESIANPOINT((0.,0.,0.));\n#210=IFCDIRECTION((0.,0.,1.));\n#211=IFCDIRECTION((1.,0.,0.));\n#212=IFCDIRECTION((0.,0.,1.));\n#213=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677452,#19,#16,1682677452);\n#214=IFCWALL('17tQPzBCHF8AW_kGpfo2iN',#213,'North wall',$,$,#220,#215,$,.SOLIDWALL.);\n#215=IFCPRODUCTDEFINITIONSHAPE($,$,(#197));\n#216=IFCCARTESIANPOINT((-5.36,5.18,0.));\n#217=IFCDIRECTION((0.,0.,1.));\n#218=IFCDIRECTION((1.,0.,0.));\n#219=IFCAXIS2PLACEMENT3D(#216,#217,#218);\n#220=IFCLOCALPLACEMENT($,#219);\n#221=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677452,#19,#16,1682677452);\n#222=IFCWALL('2xyOSTZozCexEvo4DtnsoB',#221,'East wall',$,$,#252,#247,$,.SOLIDWALL.);\n#223=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,5.72),(0.36,5.72),(0.36,0.),(0.,0.)),$);\n#224=IFCINDEXEDPOLYCURVE(#223,$,.F.);\n#225=IFCDIRECTION((0.,0.,1.));\n#226=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#224);\n#227=IFCCARTESIANPOINT((0.,0.,0.));\n#228=IFCDIRECTION((0.,0.,1.));\n#229=IFCDIRECTION((1.,0.,0.));\n#230=IFCAXIS2PLACEMENT3D(#227,#228,#229);\n#231=IFCEXTRUDEDAREASOLID(#226,#230,#225,5.15663921768591);\n#232=IFCCARTESIANPOINT((0.,5.54,3.));\n#233=IFCDIRECTION((0.,0.573576436351046,0.819152044288992));\n#234=IFCDIRECTION((1.,0.,0.));\n#235=IFCAXIS2PLACEMENT3D(#232,#233,#234);\n#236=IFCPLANE(#235);\n#237=IFCHALFSPACESOLID(#236,.F.);\n#238=IFCBOOLEANCLIPPINGRESULT(.DIFFERENCE.,#231,#237);\n#239=IFCCARTESIANPOINT((0.,0.18,3.));\n#240=IFCDIRECTION((0.,-0.573576436351046,0.819152044288992));\n#241=IFCDIRECTION((1.,0.,0.));\n#242=IFCAXIS2PLACEMENT3D(#239,#240,#241);\n#243=IFCPLANE(#242);\n#244=IFCHALFSPACESOLID(#243,.F.);\n#245=IFCBOOLEANCLIPPINGRESULT(.DIFFERENCE.,#238,#244);\n#246=IFCSHAPEREPRESENTATION(#12,'Body','Clipping',(#245));\n#247=IFCPRODUCTDEFINITIONSHAPE($,$,(#246));\n#248=IFCCARTESIANPOINT((5.,-0.18,0.));\n#249=IFCDIRECTION((0.,0.,1.));\n#250=IFCDIRECTION((1.,0.,0.));\n#251=IFCAXIS2PLACEMENT3D(#248,#249,#250);\n#252=IFCLOCALPLACEMENT($,#251);\n#253=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677453,#19,#16,1682677453);\n#254=IFCWALL('3wihAbPKfDhOZU8IhBYd9M',#253,'West wall',$,$,#290,#285,$,.SOLIDWALL.);\n#255=IFCSHAPEREPRESENTATION(#256,'Body','Clipping',(#262));\n#256=IFCGEOMETRICREPRESENTATIONSUBCONTEXT('Body','Model',*,*,*,*,#257,$,.MODEL_VIEW.,$);\n#257=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#258,$);\n#258=IFCAXIS2PLACEMENT3D(#259,#260,#261);\n#259=IFCCARTESIANPOINT((0.,0.,0.));\n#260=IFCDIRECTION((0.,0.,1.));\n#261=IFCDIRECTION((1.,0.,0.));\n#262=IFCBOOLEANCLIPPINGRESULT(.DIFFERENCE.,#263,#279);\n#263=IFCBOOLEANCLIPPINGRESULT(.DIFFERENCE.,#264,#273);\n#264=IFCEXTRUDEDAREASOLID(#265,#268,#272,5.15663921768591);\n#265=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#266);\n#266=IFCINDEXEDPOLYCURVE(#267,$,.F.);\n#267=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,5.72),(0.36,5.72),(0.36,0.),(0.,0.)),$);\n#268=IFCAXIS2PLACEMENT3D(#269,#270,#271);\n#269=IFCCARTESIANPOINT((0.,0.,0.));\n#270=IFCDIRECTION((0.,0.,1.));\n#271=IFCDIRECTION((1.,0.,0.));\n#272=IFCDIRECTION((0.,0.,1.));\n#273=IFCHALFSPACESOLID(#274,.F.);\n#274=IFCPLANE(#275);\n#275=IFCAXIS2PLACEMENT3D(#276,#277,#278);\n#276=IFCCARTESIANPOINT((0.,5.54,3.));\n#277=IFCDIRECTION((0.,0.573576436351046,0.819152044288992));\n#278=IFCDIRECTION((1.,0.,0.));\n#279=IFCHALFSPACESOLID(#280,.F.);\n#280=IFCPLANE(#281);\n#281=IFCAXIS2PLACEMENT3D(#282,#283,#284);\n#282=IFCCARTESIANPOINT((0.,0.18,3.));\n#283=IFCDIRECTION((0.,-0.573576436351046,0.819152044288992));\n#284=IFCDIRECTION((1.,0.,0.));\n#285=IFCPRODUCTDEFINITIONSHAPE($,$,(#255));\n#286=IFCCARTESIANPOINT((-5.36,-0.18,0.));\n#287=IFCDIRECTION((0.,0.,1.));\n#288=IFCDIRECTION((1.,0.,0.));\n#289=IFCAXIS2PLACEMENT3D(#286,#287,#288);\n#290=IFCLOCALPLACEMENT($,#289);\n#291=IFCOPENINGELEMENT('0UdNmfXXrBmOLRm4WxH6x4',#292,$,$,$,#339,#311,$,$);\n#292=IFCOWNERHISTORY(#293,#297,.READWRITE.,.MODIFIED.,1682677453,#19,#16,1682677451);\n#293=IFCPERSONANDORGANIZATION(#294,#295,$);\n#294=IFCPERSON('CV','V','Carlos',$,$,$,$,$);\n#295=IFCORGANIZATION('OSArch','OSArch',$,(#296),$);\n#296=IFCACTORROLE(.CIVILENGINEER.,$,$);\n#297=IFCAPPLICATION(#298,'0.7.0','IfcOpenShell','IfcOpenShell');\n#298=IFCORGANIZATION('IfcOpenShell','IfcOpenShell','IfcOpenShell is an open source software library that helps users and software developers to work with IFC data.',(#299),(#300));\n#299=IFCACTORROLE(.USERDEFINED.,'CONTRIBUTOR',$);\n#300=IFCTELECOMADDRESS(.USERDEFINED.,$,'WEBPAGE',$,$,$,$,'https://ifcopenshell.org',$);\n#302=IFCLOCALPLACEMENT($,#303);\n#303=IFCAXIS2PLACEMENT3D(#304,#305,#306);\n#304=IFCCARTESIANPOINT((-5.36,-0.18,0.));\n#305=IFCDIRECTION((0.,0.,1.));\n#306=IFCDIRECTION((1.,0.,0.));\n#311=IFCPRODUCTDEFINITIONSHAPE($,$,(#312));\n#312=IFCSHAPEREPRESENTATION(#313,'Body','SweptSolid',(#319));\n#313=IFCGEOMETRICREPRESENTATIONSUBCONTEXT('Body','Model',*,*,*,*,#314,$,.MODEL_VIEW.,$);\n#314=IFCGEOMETRICREPRESENTATIONCONTEXT($,'Model',3,1.E-05,#315,$);\n#315=IFCAXIS2PLACEMENT3D(#316,#317,#318);\n#316=IFCCARTESIANPOINT((0.,0.,0.));\n#317=IFCDIRECTION((0.,0.,1.));\n#318=IFCDIRECTION((1.,0.,0.));\n#319=IFCEXTRUDEDAREASOLID(#320,#323,#327,1.6);\n#320=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#321);\n#321=IFCINDEXEDPOLYCURVE(#322,$,.F.);\n#322=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,3.72),(6.,3.72),(6.,0.),(0.,0.)),$);\n#323=IFCAXIS2PLACEMENT3D(#324,#325,#326);\n#324=IFCCARTESIANPOINT((0.,0.,0.));\n#325=IFCDIRECTION((0.,0.,1.));\n#326=IFCDIRECTION((1.,0.,0.));\n#327=IFCDIRECTION((0.,0.,1.));\n#333=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677453,#19,#16,1682677453);\n#334=IFCRELVOIDSELEMENT('1ONsol9YDDewT38eCG5vYw',#333,$,$,#254,#291);\n#335=IFCCARTESIANPOINT((-0.45,-1.7,0.4));\n#336=IFCDIRECTION((0.,0.,1.));\n#337=IFCDIRECTION((1.,0.,0.));\n#338=IFCAXIS2PLACEMENT3D(#335,#336,#337);\n#339=IFCLOCALPLACEMENT(#290,#338);\n#340=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677453,#19,#16,1682677453);\n#341=IFCRELCONNECTSPATHELEMENTS('0VTNds$Jr6teXEN3Wl6Vbr',#340,$,$,#350,#41,#222,(),(),.ATEND.,.ATSTART.);\n#342=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677453,#19,#16,1682677453);\n#343=IFCRELCONNECTSPATHELEMENTS('1kr_SuJFTC9RJG1571QY7U',#342,$,$,#350,#222,#214,(),(),.ATEND.,.ATSTART.);\n#344=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677453,#19,#16,1682677453);\n#345=IFCRELCONNECTSPATHELEMENTS('2KMPmFUIjCDR2OX$CPD9iI',#344,$,$,#350,#214,#254,(),(),.ATEND.,.ATSTART.);\n#346=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677453,#19,#16,1682677453);\n#347=IFCRELCONNECTSPATHELEMENTS('1COzE0gVPEvvpGwB$uWUYH',#346,$,$,#350,#254,#41,(),(),.ATEND.,.ATSTART.);\n#348=IFCCARTESIANPOINTLIST2D(((-1.,-1.),(1.,1.)),$);\n#349=IFCINDEXEDPOLYCURVE(#348,$,$);\n#350=IFCCONNECTIONCURVEGEOMETRY(#349,$);\n#351=IFCCARTESIANPOINT((-4.42354486374086E-16,-1.70235595703125,-0.49668701171875));\n#352=IFCCARTESIANPOINT((-3.05319480895996,-1.75537829399109,-0.768822317123413));\n#353=IFCCARTESIANPOINT((-2.86571655273438,-4.38050888061523,-1.15385635375977));\n#354=IFCPOLYLOOP((#351,#352,#353));\n#355=IFCFACEOUTERBOUND(#354,.T.);\n#356=IFCFACE((#355));\n#357=IFCCARTESIANPOINT((-4.16333634234434E-16,-4.3525390625,-0.9861328125));\n#358=IFCPOLYLOOP((#351,#353,#357));\n#359=IFCFACEOUTERBOUND(#358,.T.);\n#360=IFCFACE((#359));\n#361=IFCCARTESIANPOINT((-4.42354486374086E-16,3.61112060546875,-0.41280029296875));\n#362=IFCCARTESIANPOINT((-2.86571655273438,6.01451919555664,-1.13110549926758));\n#363=IFCCARTESIANPOINT((-3.05319480895996,3.49151425361633,-0.791727666854858));\n#364=IFCPOLYLOOP((#361,#362,#363));\n#365=IFCFACEOUTERBOUND(#364,.T.);\n#366=IFCFACE((#365));\n#367=IFCCARTESIANPOINT((-4.16333634234434E-16,6.1490234375,-0.7556640625));\n#368=IFCPOLYLOOP((#367,#362,#361));\n#369=IFCFACEOUTERBOUND(#368,.T.);\n#370=IFCFACE((#369));\n#371=IFCCARTESIANPOINT((-8.11354149208741,-3.77236751448647,-2.44009447043433));\n#372=IFCCARTESIANPOINT((-8.72551555265276,-4.66816406824812,-3.02801821833143));\n#373=IFCCARTESIANPOINT((-7.88140887078804,-5.44710391507401,-2.64655628155315));\n#374=IFCPOLYLOOP((#371,#372,#373));\n#375=IFCFACEOUTERBOUND(#374,.T.);\n#376=IFCFACE((#375));\n#377=IFCCARTESIANPOINT((-7.43010520805589,-3.13995517113504,-1.9601898093612));\n#378=IFCCARTESIANPOINT((-8.23906950580105,-2.52094679746348,-2.40648133510616));\n#379=IFCPOLYLOOP((#377,#378,#371));\n#380=IFCFACEOUTERBOUND(#379,.T.);\n#381=IFCFACE((#380));\n#382=IFCCARTESIANPOINT((-1.84404849266973,-7.07125774358934,-2.35825401020775));\n#383=IFCCARTESIANPOINT((-3.06201357876754,-6.16740295943602,-1.8580642025017));\n#384=IFCCARTESIANPOINT((-3.1166057848309,-7.08308933036972,-2.40113541446977));\n#385=IFCPOLYLOOP((#382,#383,#384));\n#386=IFCFACEOUTERBOUND(#385,.T.);\n#387=IFCFACE((#386));\n#388=IFCCARTESIANPOINT((-1.29001716675697,-6.15003971673833,-1.77345646663052));\n#389=IFCPOLYLOOP((#388,#353,#383));\n#390=IFCFACEOUTERBOUND(#389,.T.);\n#391=IFCFACE((#390));\n#392=IFCPOLYLOOP((#388,#383,#382));\n#393=IFCFACEOUTERBOUND(#392,.T.);\n#394=IFCFACE((#393));\n#395=IFCCARTESIANPOINT((-0.56776087798863,-7.06528201235338,-2.3599470877244));\n#396=IFCPOLYLOOP((#388,#382,#395));\n#397=IFCFACEOUTERBOUND(#396,.T.);\n#398=IFCFACE((#397));\n#399=IFCPOLYLOOP((#388,#357,#353));\n#400=IFCFACEOUTERBOUND(#399,.T.);\n#401=IFCFACE((#400));\n#402=IFCCARTESIANPOINT((-4.40335392057896,-3.10406802725047,-1.08389494951814));\n#403=IFCPOLYLOOP((#402,#353,#352));\n#404=IFCFACEOUTERBOUND(#403,.T.);\n#405=IFCFACE((#404));\n#406=IFCCARTESIANPOINT((-5.88511169433594,-1.87118148803711,-1.27368362426758));\n#407=IFCPOLYLOOP((#402,#352,#406));\n#408=IFCFACEOUTERBOUND(#407,.T.);\n#409=IFCFACE((#408));\n#410=IFCCARTESIANPOINT((-5.585146484375,-4.45016357421875,-1.56888427734375));\n#411=IFCPOLYLOOP((#402,#406,#410));\n#412=IFCFACEOUTERBOUND(#411,.T.);\n#413=IFCFACE((#412));\n#414=IFCCARTESIANPOINT((0.496576319823033,-6.14550439621651,-1.77312365166719));\n#415=IFCPOLYLOOP((#414,#388,#395));\n#416=IFCFACEOUTERBOUND(#415,.T.);\n#417=IFCFACE((#416));\n#418=IFCPOLYLOOP((#414,#357,#388));\n#419=IFCFACEOUTERBOUND(#418,.T.);\n#420=IFCFACE((#419));\n#421=IFCCARTESIANPOINT((0.709674528237069,-7.06503411303741,-2.40410878869377));\n#422=IFCPOLYLOOP((#414,#395,#421));\n#423=IFCFACEOUTERBOUND(#422,.T.);\n#424=IFCFACE((#423));\n#425=IFCCARTESIANPOINT((-4.10273085633859,-0.0480174397005035,-0.856698093492798));\n#426=IFCPOLYLOOP((#425,#406,#352));\n#427=IFCFACEOUTERBOUND(#426,.T.);\n#428=IFCFACE((#427));\n#429=IFCCARTESIANPOINT((-4.59635009765625,0.794963836669922,-0.952576141357422));\n#430=IFCCARTESIANPOINT((-5.98484375,0.6952734375,-1.2955078125));\n#431=IFCPOLYLOOP((#429,#430,#425));\n#432=IFCFACEOUTERBOUND(#431,.T.);\n#433=IFCFACE((#432));\n#434=IFCCARTESIANPOINT((-5.06079074125286,1.62829164250779,-1.08344771967776));\n#435=IFCPOLYLOOP((#434,#430,#429));\n#436=IFCFACEOUTERBOUND(#435,.T.);\n#437=IFCFACE((#436));\n#438=IFCCARTESIANPOINT((-5.88511169433594,3.25629653930664,-1.4789387512207));\n#439=IFCPOLYLOOP((#434,#363,#438));\n#440=IFCFACEOUTERBOUND(#439,.T.);\n#441=IFCFACE((#440));\n#442=IFCPOLYLOOP((#434,#429,#363));\n#443=IFCFACEOUTERBOUND(#442,.T.);\n#444=IFCFACE((#443));\n#445=IFCCARTESIANPOINT((0.989462362707178,-3.46647381127562,-0.722982941777073));\n#446=IFCPOLYLOOP((#445,#351,#357));\n#447=IFCFACEOUTERBOUND(#446,.T.);\n#448=IFCFACE((#447));\n#449=IFCCARTESIANPOINT((5.60681210712777,-8.05311729790546,-3.98081682385316));\n#450=IFCCARTESIANPOINT((4.1971447441955,-10.,-5.93655990034642));\n#451=IFCCARTESIANPOINT((6.84912868973226,-10.,-6.07602486447226));\n#452=IFCPOLYLOOP((#449,#450,#451));\n#453=IFCFACEOUTERBOUND(#452,.T.);\n#454=IFCFACE((#453));\n#455=IFCCARTESIANPOINT((4.35328798198007,-6.1819094923489,-2.14228833966457));\n#456=IFCCARTESIANPOINT((3.72722625412883,-7.08808115030013,-2.71996672298559));\n#457=IFCCARTESIANPOINT((4.98951010345256,-7.10872610166149,-2.98252669438335));\n#458=IFCPOLYLOOP((#455,#456,#457));\n#459=IFCFACEOUTERBOUND(#458,.T.);\n#460=IFCFACE((#459));\n#461=IFCCARTESIANPOINT((1.49946240603924,-3.02592028509825,-0.615825333558023));\n#462=IFCPOLYLOOP((#461,#351,#445));\n#463=IFCFACEOUTERBOUND(#462,.T.);\n#464=IFCFACE((#463));\n#465=IFCCARTESIANPOINT((3.05319480895996,-1.71428256034851,-0.374357824325561));\n#466=IFCPOLYLOOP((#461,#465,#351));\n#467=IFCFACEOUTERBOUND(#466,.T.);\n#468=IFCFACE((#467));\n#469=IFCCARTESIANPOINT((2.86571655273437,-4.36589706420898,-1.01429946899414));\n#470=IFCPOLYLOOP((#461,#445,#469));\n#471=IFCFACEOUTERBOUND(#470,.T.);\n#472=IFCFACE((#471));\n#473=IFCCARTESIANPOINT((-7.98357004259595,5.54199721009906,-3.54830858512497));\n#474=IFCCARTESIANPOINT((-8.14381885772828,4.31738788859784,-3.23237320134351));\n#475=IFCCARTESIANPOINT((-7.34769495115332,4.81499622727138,-2.6645284737957));\n#476=IFCPOLYLOOP((#473,#474,#475));\n#477=IFCFACEOUTERBOUND(#476,.T.);\n#478=IFCFACE((#477));\n#479=IFCCARTESIANPOINT((-7.79687651546032,6.6554638245303,-3.91626391370564));\n#480=IFCCARTESIANPOINT((-8.60658890183408,6.18221696984924,-4.59827620855966));\n#481=IFCPOLYLOOP((#479,#480,#473));\n#482=IFCFACEOUTERBOUND(#481,.T.);\n#483=IFCFACE((#482));\n#484=IFCCARTESIANPOINT((-1.57315185546875,0.925898895263672,-0.526970672607422));\n#485=IFCPOLYLOOP((#484,#352,#351));\n#486=IFCFACEOUTERBOUND(#485,.T.);\n#487=IFCFACE((#486));\n#488=IFCPOLYLOOP((#484,#425,#352));\n#489=IFCFACEOUTERBOUND(#488,.T.);\n#490=IFCFACE((#489));\n#491=IFCPOLYLOOP((#484,#429,#425));\n#492=IFCFACEOUTERBOUND(#491,.T.);\n#493=IFCFACE((#492));\n#494=IFCPOLYLOOP((#484,#363,#429));\n#495=IFCFACEOUTERBOUND(#494,.T.);\n#496=IFCFACE((#495));\n#497=IFCPOLYLOOP((#484,#361,#363));\n#498=IFCFACEOUTERBOUND(#497,.T.);\n#499=IFCFACE((#498));\n#500=IFCCARTESIANPOINT((6.2878024828953,-7.13798524524916,-3.37572215650807));\n#501=IFCPOLYLOOP((#500,#457,#449));\n#502=IFCFACEOUTERBOUND(#501,.T.);\n#503=IFCFACE((#502));\n#504=IFCCARTESIANPOINT((-4.81629444622917,5.035331929939,-1.41096672388207));\n#505=IFCPOLYLOOP((#504,#438,#363));\n#506=IFCFACEOUTERBOUND(#505,.T.);\n#507=IFCFACE((#506));\n#508=IFCPOLYLOOP((#504,#363,#362));\n#509=IFCFACEOUTERBOUND(#508,.T.);\n#510=IFCFACE((#509));\n#511=IFCCARTESIANPOINT((3.97132144490169,-2.61625971650663,-0.563488797439415));\n#512=IFCPOLYLOOP((#511,#461,#469));\n#513=IFCFACEOUTERBOUND(#512,.T.);\n#514=IFCFACE((#513));\n#515=IFCPOLYLOOP((#511,#465,#461));\n#516=IFCFACEOUTERBOUND(#515,.T.);\n#517=IFCFACE((#516));\n#518=IFCCARTESIANPOINT((4.40335392057896,-3.06809736918658,-0.712334390841425));\n#519=IFCCARTESIANPOINT((5.88511169433594,-1.80542831420898,-0.639146881103516));\n#520=IFCPOLYLOOP((#518,#519,#511));\n#521=IFCFACEOUTERBOUND(#520,.T.);\n#522=IFCFACE((#521));\n#523=IFCPOLYLOOP((#518,#511,#469));\n#524=IFCFACEOUTERBOUND(#523,.T.);\n#525=IFCFACE((#524));\n#526=IFCCARTESIANPOINT((4.81629444622917,-3.52041863321985,-0.902395835204053));\n#527=IFCPOLYLOOP((#526,#518,#469));\n#528=IFCFACEOUTERBOUND(#527,.T.);\n#529=IFCFACE((#528));\n#530=IFCCARTESIANPOINT((2.08743093706017,0.0732206967479838,-0.243809267941489));\n#531=IFCPOLYLOOP((#530,#351,#465));\n#532=IFCFACEOUTERBOUND(#531,.T.);\n#533=IFCFACE((#532));\n#534=IFCCARTESIANPOINT((1.57315185546875,0.966811981201172,-0.244927215576172));\n#535=IFCPOLYLOOP((#534,#361,#484));\n#536=IFCFACEOUTERBOUND(#535,.T.);\n#537=IFCFACE((#536));\n#538=IFCPOLYLOOP((#534,#351,#530));\n#539=IFCFACEOUTERBOUND(#538,.T.);\n#540=IFCFACE((#539));\n#541=IFCPOLYLOOP((#534,#484,#351));\n#542=IFCFACEOUTERBOUND(#541,.T.);\n#543=IFCFACE((#542));\n#544=IFCCARTESIANPOINT((3.05319480895996,3.60566906929016,-0.223984365463257));\n#545=IFCPOLYLOOP((#534,#544,#361));\n#546=IFCFACEOUTERBOUND(#545,.T.);\n#547=IFCFACE((#546));\n#548=IFCCARTESIANPOINT((3.11552734375,0.94940185546875,-0.16685302734375));\n#549=IFCPOLYLOOP((#548,#530,#465));\n#550=IFCFACEOUTERBOUND(#549,.T.);\n#551=IFCFACE((#550));\n#552=IFCPOLYLOOP((#548,#534,#530));\n#553=IFCFACEOUTERBOUND(#552,.T.);\n#554=IFCFACE((#553));\n#555=IFCPOLYLOOP((#548,#544,#534));\n#556=IFCFACEOUTERBOUND(#555,.T.);\n#557=IFCFACE((#556));\n#558=IFCCARTESIANPOINT((-4.093403938572,7.44787379006724,-2.06203074864318));\n#559=IFCPOLYLOOP((#558,#504,#362));\n#560=IFCFACEOUTERBOUND(#559,.T.);\n#561=IFCFACE((#560));\n#562=IFCCARTESIANPOINT((-5.585146484375,5.75380126953125,-1.91190185546875));\n#563=IFCPOLYLOOP((#558,#562,#504));\n#564=IFCFACEOUTERBOUND(#563,.T.);\n#565=IFCFACE((#564));\n#566=IFCCARTESIANPOINT((-5.18048607214884,8.7373551343648,-3.61025754978732));\n#567=IFCCARTESIANPOINT((-6.02256935190325,7.9604352952599,-3.37122642182843));\n#568=IFCCARTESIANPOINT((-4.61127674346795,8.11053917825769,-2.74067387426809));\n#569=IFCPOLYLOOP((#566,#567,#568));\n#570=IFCFACEOUTERBOUND(#569,.T.);\n#571=IFCFACE((#570));\n#572=IFCCARTESIANPOINT((5.06079074125286,-0.00403271089910591,-0.264763826794095));\n#573=IFCPOLYLOOP((#572,#465,#511));\n#574=IFCFACEOUTERBOUND(#573,.T.);\n#575=IFCFACE((#574));\n#576=IFCPOLYLOOP((#572,#511,#519));\n#577=IFCFACEOUTERBOUND(#576,.T.);\n#578=IFCFACE((#577));\n#579=IFCPOLYLOOP((#572,#548,#465));\n#580=IFCFACEOUTERBOUND(#579,.T.);\n#581=IFCFACE((#580));\n#582=IFCCARTESIANPOINT((4.59635009765625,0.902117156982422,-0.181152801513672));\n#583=IFCPOLYLOOP((#582,#548,#572));\n#584=IFCFACEOUTERBOUND(#583,.T.);\n#585=IFCFACE((#584));\n#586=IFCCARTESIANPOINT((-3.34645166850071,8.21550923855824,-2.34967029513584));\n#587=IFCPOLYLOOP((#586,#568,#558));\n#588=IFCFACEOUTERBOUND(#587,.T.);\n#589=IFCFACE((#588));\n#590=IFCCARTESIANPOINT((4.10273085633859,1.80868348479652,-0.151467366648091));\n#591=IFCPOLYLOOP((#590,#544,#548));\n#592=IFCFACEOUTERBOUND(#591,.T.);\n#593=IFCFACE((#592));\n#594=IFCPOLYLOOP((#590,#548,#582));\n#595=IFCFACEOUTERBOUND(#594,.T.);\n#596=IFCFACE((#595));\n#597=IFCCARTESIANPOINT((8.24293261121821,-2.42405393055192,-1.94007047366949));\n#598=IFCCARTESIANPOINT((7.43369437181982,-3.06453672891712,-1.57633998208354));\n#599=IFCCARTESIANPOINT((8.11905706052427,-3.69436230730127,-2.29308647520748));\n#600=IFCPOLYLOOP((#597,#598,#599));\n#601=IFCFACEOUTERBOUND(#600,.T.);\n#602=IFCFACE((#601));\n#603=IFCCARTESIANPOINT((0.989462362707178,5.34153086260241,-0.512167572266553));\n#604=IFCPOLYLOOP((#603,#367,#361));\n#605=IFCFACEOUTERBOUND(#604,.T.);\n#606=IFCFACE((#605));\n#607=IFCCARTESIANPOINT((1.49946240603924,4.92130607832223,-0.417552339397371));\n#608=IFCPOLYLOOP((#607,#603,#361));\n#609=IFCFACEOUTERBOUND(#608,.T.);\n#610=IFCFACE((#609));\n#611=IFCCARTESIANPOINT((2.86571655273437,6.14602554321289,-0.677583770751954));\n#612=IFCPOLYLOOP((#607,#611,#603));\n#613=IFCFACEOUTERBOUND(#612,.T.);\n#614=IFCFACE((#613));\n#615=IFCCARTESIANPOINT((2.01556641518061,4.49126486053955,-0.338393347050755));\n#616=IFCPOLYLOOP((#615,#607,#361));\n#617=IFCFACEOUTERBOUND(#616,.T.);\n#618=IFCFACE((#617));\n#619=IFCPOLYLOOP((#615,#611,#607));\n#620=IFCFACEOUTERBOUND(#619,.T.);\n#621=IFCFACE((#620));\n#622=IFCPOLYLOOP((#615,#361,#544));\n#623=IFCFACEOUTERBOUND(#622,.T.);\n#624=IFCFACE((#623));\n#625=IFCCARTESIANPOINT((5.98484375,0.8199609375,-0.3541015625));\n#626=IFCPOLYLOOP((#625,#582,#572));\n#627=IFCFACEOUTERBOUND(#626,.T.);\n#628=IFCFACE((#627));\n#629=IFCPOLYLOOP((#625,#590,#582));\n#630=IFCFACEOUTERBOUND(#629,.T.);\n#631=IFCFACE((#630));\n#632=IFCCARTESIANPOINT((-1.39419817046714,7.63382734857908,-1.4994362117116));\n#633=IFCPOLYLOOP((#632,#362,#367));\n#634=IFCFACEOUTERBOUND(#633,.T.);\n#635=IFCFACE((#634));\n#636=IFCCARTESIANPOINT((-2.569845595281,8.91114237135098,-2.82327577684157));\n#637=IFCCARTESIANPOINT((-1.9930668212292,8.30012841893651,-2.07119100322777));\n#638=IFCPOLYLOOP((#636,#586,#637));\n#639=IFCFACEOUTERBOUND(#638,.T.);\n#640=IFCFACE((#639));\n#641=IFCCARTESIANPOINT((8.96109880338651,-1.546001934692,-2.51110211584018));\n#642=IFCCARTESIANPOINT((8.33993240833282,-0.738337876200676,-1.64786901175976));\n#643=IFCPOLYLOOP((#641,#642,#597));\n#644=IFCFACEOUTERBOUND(#643,.T.);\n#645=IFCFACE((#644));\n#646=IFCCARTESIANPOINT((-0.717082008794764,8.35469446596244,-1.9169104466114));\n#647=IFCPOLYLOOP((#646,#637,#632));\n#648=IFCFACEOUTERBOUND(#647,.T.);\n#649=IFCFACE((#648));\n#650=IFCCARTESIANPOINT((3.89234076717535,5.28469771662094,-0.503751102930671));\n#651=IFCPOLYLOOP((#650,#611,#615));\n#652=IFCFACEOUTERBOUND(#651,.T.);\n#653=IFCFACE((#652));\n#654=IFCPOLYLOOP((#650,#615,#544));\n#655=IFCFACEOUTERBOUND(#654,.T.);\n#656=IFCFACE((#655));\n#657=IFCCARTESIANPOINT((4.40335392057896,4.83665224481374,-0.452553551755846));\n#658=IFCPOLYLOOP((#657,#650,#544));\n#659=IFCFACEOUTERBOUND(#658,.T.);\n#660=IFCFACE((#659));\n#661=IFCCARTESIANPOINT((4.9080786322588,4.37876389603338,-0.426542674039781));\n#662=IFCCARTESIANPOINT((5.585146484375,5.96421142578125,-1.07572021484375));\n#663=IFCPOLYLOOP((#661,#662,#657));\n#664=IFCFACEOUTERBOUND(#663,.T.);\n#665=IFCFACE((#664));\n#666=IFCPOLYLOOP((#661,#657,#544));\n#667=IFCFACEOUTERBOUND(#666,.T.);\n#668=IFCFACE((#667));\n#669=IFCCARTESIANPOINT((5.88511169433594,3.43894424438477,-0.460124664306642));\n#670=IFCPOLYLOOP((#661,#590,#669));\n#671=IFCFACEOUTERBOUND(#670,.T.);\n#672=IFCFACE((#671));\n#673=IFCPOLYLOOP((#661,#544,#590));\n#674=IFCFACEOUTERBOUND(#673,.T.);\n#675=IFCFACE((#674));\n#676=IFCCARTESIANPOINT((1.29612620623656,7.69948251990711,-1.3368618842077));\n#677=IFCPOLYLOOP((#676,#632,#367));\n#678=IFCFACEOUTERBOUND(#677,.T.);\n#679=IFCFACE((#678));\n#680=IFCCARTESIANPOINT((0.57651532022852,8.38424409766318,-1.85673147716955));\n#681=IFCPOLYLOOP((#680,#632,#676));\n#682=IFCFACEOUTERBOUND(#681,.T.);\n#683=IFCFACE((#682));\n#684=IFCPOLYLOOP((#680,#646,#632));\n#685=IFCFACEOUTERBOUND(#684,.T.);\n#686=IFCFACE((#685));\n#687=IFCCARTESIANPOINT((-0.0885231977945335,8.99806317327125,-2.54766097369416));\n#688=IFCPOLYLOOP((#687,#646,#680));\n#689=IFCFACEOUTERBOUND(#688,.T.);\n#690=IFCFACE((#689));\n#691=IFCCARTESIANPOINT((1.85278615712481,8.38548292522671,-1.89383765773));\n#692=IFCPOLYLOOP((#691,#680,#676));\n#693=IFCFACEOUTERBOUND(#692,.T.);\n#694=IFCFACE((#693));\n#695=IFCCARTESIANPOINT((8.26795330931127,3.14007446089873,-1.72423935556029));\n#696=IFCCARTESIANPOINT((8.33993240833282,1.78797901451588,-1.57214941680432));\n#697=IFCCARTESIANPOINT((8.96972082411329,2.08900140334722,-2.40258977522205));\n#698=IFCPOLYLOOP((#695,#696,#697));\n#699=IFCFACEOUTERBOUND(#698,.T.);\n#700=IFCFACE((#699));\n#701=IFCCARTESIANPOINT((2.32742331614966,9.00044125033705,-2.67502835247898));\n#702=IFCCARTESIANPOINT((3.10622705778103,8.3556492847726,-2.03653611998626));\n#703=IFCPOLYLOOP((#701,#691,#702));\n#704=IFCFACEOUTERBOUND(#703,.T.);\n#705=IFCFACE((#704));\n#706=IFCCARTESIANPOINT((8.155496783392,4.39017486061151,-2.00836986808276));\n#707=IFCCARTESIANPOINT((7.4576630908109,4.11499974812852,-1.30399051875757));\n#708=IFCPOLYLOOP((#706,#707,#695));\n#709=IFCFACEOUTERBOUND(#708,.T.);\n#710=IFCFACE((#709));\n#711=IFCCARTESIANPOINT((4.37253355976419,8.28842325744541,-2.31256132002331));\n#712=IFCCARTESIANPOINT((3.92911887811991,7.62654645302678,-1.57465507762356));\n#713=IFCPOLYLOOP((#711,#702,#712));\n#714=IFCFACEOUTERBOUND(#713,.T.);\n#715=IFCFACE((#714));\n#716=IFCCARTESIANPOINT((8.80786134829556,4.56144818882912,-2.91565348175102));\n#717=IFCCARTESIANPOINT((8.01132616846811,5.53695208613105,-2.41779910002149));\n#718=IFCPOLYLOOP((#716,#717,#706));\n#719=IFCFACEOUTERBOUND(#718,.T.);\n#720=IFCFACE((#719));\n#721=IFCCARTESIANPOINT((4.84054243379101,8.89264070654631,-3.29442621705132));\n#722=IFCCARTESIANPOINT((5.74559621826117,8.16426839435781,-2.80602272120066));\n#723=IFCPOLYLOOP((#721,#711,#722));\n#724=IFCFACEOUTERBOUND(#723,.T.);\n#725=IFCFACE((#724));\n#726=IFCCARTESIANPOINT((8.6456476216862,6.02535228608532,-3.51750923301384));\n#727=IFCCARTESIANPOINT((7.84286750842431,6.58030430470603,-2.94635357937062));\n#728=IFCPOLYLOOP((#726,#727,#717));\n#729=IFCFACEOUTERBOUND(#728,.T.);\n#730=IFCFACE((#729));\n#731=IFCPOLYLOOP((#726,#717,#716));\n#732=IFCFACEOUTERBOUND(#731,.T.);\n#733=IFCFACE((#732));\n#734=IFCCARTESIANPOINT((8.9955292695659,8.61367299958862,-6.14602264915007));\n#735=IFCCARTESIANPOINT((7.91769112324703,9.18276166992915,-5.80172091097046));\n#736=IFCCARTESIANPOINT((8.0156039681587,8.7612273124477,-5.34301930416362));\n#737=IFCPOLYLOOP((#734,#735,#736));\n#738=IFCFACEOUTERBOUND(#737,.T.);\n#739=IFCFACE((#738));\n#740=IFCCARTESIANPOINT((8.9084979589796,9.30800949697651,-6.75543394532811));\n#741=IFCPOLYLOOP((#740,#735,#734));\n#742=IFCFACEOUTERBOUND(#741,.T.);\n#743=IFCFACE((#742));\n#744=IFCCARTESIANPOINT((9.32750282296339,8.55781135326067,-6.4494264755708));\n#745=IFCPOLYLOOP((#740,#734,#744));\n#746=IFCFACEOUTERBOUND(#745,.T.);\n#747=IFCFACE((#746));\n#748=IFCCARTESIANPOINT((7.70561380679685,10.,-6.84967017432755));\n#749=IFCPOLYLOOP((#740,#748,#735));\n#750=IFCFACEOUTERBOUND(#749,.T.);\n#751=IFCFACE((#750));\n#752=IFCCARTESIANPOINT((9.24513771724447,9.51740416458158,-7.22551962234331));\n#753=IFCCARTESIANPOINT((10.,8.43526956500757,-7.11484017135397));\n#754=IFCPOLYLOOP((#752,#744,#753));\n#755=IFCFACEOUTERBOUND(#754,.T.);\n#756=IFCFACE((#755));\n#757=IFCPOLYLOOP((#752,#748,#740));\n#758=IFCFACEOUTERBOUND(#757,.T.);\n#759=IFCFACE((#758));\n#760=IFCCARTESIANPOINT((10.,10.,-8.13));\n#761=IFCPOLYLOOP((#752,#753,#760));\n#762=IFCFACEOUTERBOUND(#761,.T.);\n#763=IFCFACE((#762));\n#764=IFCPOLYLOOP((#752,#760,#748));\n#765=IFCFACEOUTERBOUND(#764,.T.);\n#766=IFCFACE((#765));\n#767=IFCPOLYLOOP((#752,#740,#744));\n#768=IFCFACEOUTERBOUND(#767,.T.);\n#769=IFCFACE((#768));\n#770=IFCCARTESIANPOINT((-7.44749890050135,-8.40168151641437,-3.65170911501098));\n#771=IFCCARTESIANPOINT((-7.57231086730957,-7.17922076225281,-3.11541533470154));\n#772=IFCCARTESIANPOINT((-8.61562958061695,-8.58096974324435,-3.89070422586054));\n#773=IFCPOLYLOOP((#770,#771,#772));\n#774=IFCFACEOUTERBOUND(#773,.T.);\n#775=IFCFACE((#774));\n#776=IFCCARTESIANPOINT((-7.37751580847085,-9.03060295820059,-3.9750960961701));\n#777=IFCPOLYLOOP((#770,#772,#776));\n#778=IFCFACEOUTERBOUND(#777,.T.);\n#779=IFCFACE((#778));\n#780=IFCCARTESIANPOINT((-8.84582820153016,-6.80891467730477,-3.50926544994389));\n#781=IFCPOLYLOOP((#780,#772,#771));\n#782=IFCFACEOUTERBOUND(#781,.T.);\n#783=IFCFACE((#782));\n#784=IFCCARTESIANPOINT((-8.51517405838391,-6.35303742473678,-3.22463932347777));\n#785=IFCPOLYLOOP((#784,#771,#373));\n#786=IFCFACEOUTERBOUND(#785,.T.);\n#787=IFCFACE((#786));\n#788=IFCPOLYLOOP((#784,#373,#372));\n#789=IFCFACEOUTERBOUND(#788,.T.);\n#790=IFCFACE((#789));\n#791=IFCPOLYLOOP((#784,#780,#771));\n#792=IFCFACEOUTERBOUND(#791,.T.);\n#793=IFCFACE((#792));\n#794=IFCCARTESIANPOINT((-10.,-6.46659579617821,-4.289813));\n#795=IFCPOLYLOOP((#784,#794,#780));\n#796=IFCFACEOUTERBOUND(#795,.T.);\n#797=IFCFACE((#796));\n#798=IFCPOLYLOOP((#784,#372,#794));\n#799=IFCFACEOUTERBOUND(#798,.T.);\n#800=IFCFACE((#799));\n#801=IFCCARTESIANPOINT((-6.41402256390939,-7.60388752240756,-3.06053116619859));\n#802=IFCPOLYLOOP((#801,#771,#770));\n#803=IFCFACEOUTERBOUND(#802,.T.);\n#804=IFCFACE((#803));\n#805=IFCCARTESIANPOINT((-6.9333396115739,-6.86581661135016,-2.79652483881474));\n#806=IFCPOLYLOOP((#805,#373,#771));\n#807=IFCFACEOUTERBOUND(#806,.T.);\n#808=IFCFACE((#807));\n#809=IFCPOLYLOOP((#805,#771,#801));\n#810=IFCFACEOUTERBOUND(#809,.T.);\n#811=IFCFACE((#810));\n#812=IFCCARTESIANPOINT((-6.61406425931589,-6.70973537553249,-2.64236339081325));\n#813=IFCCARTESIANPOINT((-5.23739197704209,-8.04817162381488,-3.20803095560588));\n#814=IFCPOLYLOOP((#812,#801,#813));\n#815=IFCFACEOUTERBOUND(#814,.T.);\n#816=IFCFACE((#815));\n#817=IFCPOLYLOOP((#812,#373,#805));\n#818=IFCFACEOUTERBOUND(#817,.T.);\n#819=IFCFACE((#818));\n#820=IFCPOLYLOOP((#812,#805,#801));\n#821=IFCFACEOUTERBOUND(#820,.T.);\n#822=IFCFACE((#821));\n#823=IFCCARTESIANPOINT((-9.43919288537727,-3.92564911367119,-3.71093829120278));\n#824=IFCCARTESIANPOINT((-10.,-3.22467205701655,-4.743008));\n#825=IFCPOLYLOOP((#823,#824,#794));\n#826=IFCFACEOUTERBOUND(#825,.T.);\n#827=IFCFACE((#826));\n#828=IFCPOLYLOOP((#823,#794,#372));\n#829=IFCFACEOUTERBOUND(#828,.T.);\n#830=IFCFACE((#829));\n#831=IFCCARTESIANPOINT((-9.02605601476171,-3.87080847453503,-3.21810959647811));\n#832=IFCPOLYLOOP((#831,#823,#372));\n#833=IFCFACEOUTERBOUND(#832,.T.);\n#834=IFCFACE((#833));\n#835=IFCPOLYLOOP((#831,#372,#371));\n#836=IFCFACEOUTERBOUND(#835,.T.);\n#837=IFCFACE((#836));\n#838=IFCCARTESIANPOINT((-9.16110260866631,-3.47859698591479,-3.34725607728198));\n#839=IFCPOLYLOOP((#838,#824,#823));\n#840=IFCFACEOUTERBOUND(#839,.T.);\n#841=IFCFACE((#840));\n#842=IFCPOLYLOOP((#838,#823,#831));\n#843=IFCFACEOUTERBOUND(#842,.T.);\n#844=IFCFACE((#843));\n#845=IFCCARTESIANPOINT((-6.29455626320458,-5.94938778330111,-2.2321572683907));\n#846=IFCPOLYLOOP((#845,#373,#812));\n#847=IFCFACEOUTERBOUND(#846,.T.);\n#848=IFCFACE((#847));\n#849=IFCCARTESIANPOINT((-9.19944152908606,-2.86546859028085,-3.38119991974034));\n#850=IFCPOLYLOOP((#849,#824,#838));\n#851=IFCFACEOUTERBOUND(#850,.T.);\n#852=IFCFACE((#851));\n#853=IFCCARTESIANPOINT((-8.86657848740686,-3.15989972043125,-3.00388485522699));\n#854=IFCPOLYLOOP((#853,#371,#378));\n#855=IFCFACEOUTERBOUND(#854,.T.);\n#856=IFCFACE((#855));\n#857=IFCPOLYLOOP((#853,#838,#831));\n#858=IFCFACEOUTERBOUND(#857,.T.);\n#859=IFCFACE((#858));\n#860=IFCPOLYLOOP((#853,#831,#371));\n#861=IFCFACEOUTERBOUND(#860,.T.);\n#862=IFCFACE((#861));\n#863=IFCPOLYLOOP((#853,#849,#838));\n#864=IFCFACEOUTERBOUND(#863,.T.);\n#865=IFCFACE((#864));\n#866=IFCCARTESIANPOINT((-5.46102784536132,-6.2142526476494,-2.16396766758088));\n#867=IFCPOLYLOOP((#866,#845,#812));\n#868=IFCFACEOUTERBOUND(#867,.T.);\n#869=IFCFACE((#868));\n#870=IFCPOLYLOOP((#866,#410,#845));\n#871=IFCFACEOUTERBOUND(#870,.T.);\n#872=IFCFACE((#871));\n#873=IFCPOLYLOOP((#866,#812,#813));\n#874=IFCFACEOUTERBOUND(#873,.T.);\n#875=IFCFACE((#874));\n#876=IFCCARTESIANPOINT((-7.24086015842474,-4.54703954530179,-2.11613355444207));\n#877=IFCPOLYLOOP((#876,#373,#845));\n#878=IFCFACEOUTERBOUND(#877,.T.);\n#879=IFCFACE((#878));\n#880=IFCPOLYLOOP((#876,#371,#373));\n#881=IFCFACEOUTERBOUND(#880,.T.);\n#882=IFCFACE((#881));\n#883=IFCCARTESIANPOINT((-4.67762599904899,-6.49697180077741,-2.18314409681883));\n#884=IFCPOLYLOOP((#883,#866,#813));\n#885=IFCFACEOUTERBOUND(#884,.T.);\n#886=IFCFACE((#885));\n#887=IFCCARTESIANPOINT((-9.12659340322939,-2.36776020700005,-3.28706042868335));\n#888=IFCCARTESIANPOINT((-8.97035287849604,-1.37082317931166,-3.11585157043689));\n#889=IFCPOLYLOOP((#887,#378,#888));\n#890=IFCFACEOUTERBOUND(#889,.T.);\n#891=IFCFACE((#890));\n#892=IFCPOLYLOOP((#887,#853,#378));\n#893=IFCFACEOUTERBOUND(#892,.T.);\n#894=IFCFACE((#893));\n#895=IFCPOLYLOOP((#887,#824,#849));\n#896=IFCFACEOUTERBOUND(#895,.T.);\n#897=IFCFACE((#896));\n#898=IFCPOLYLOOP((#887,#849,#853));\n#899=IFCFACEOUTERBOUND(#898,.T.);\n#900=IFCFACE((#899));\n#901=IFCCARTESIANPOINT((-6.9057543226979,-4.09821522810298,-1.88213775575277));\n#902=IFCPOLYLOOP((#901,#876,#845));\n#903=IFCFACEOUTERBOUND(#902,.T.);\n#904=IFCFACE((#903));\n#905=IFCPOLYLOOP((#901,#377,#371));\n#906=IFCFACEOUTERBOUND(#905,.T.);\n#907=IFCFACE((#906));\n#908=IFCPOLYLOOP((#901,#845,#410));\n#909=IFCFACEOUTERBOUND(#908,.T.);\n#910=IFCFACE((#909));\n#911=IFCPOLYLOOP((#901,#371,#876));\n#912=IFCFACEOUTERBOUND(#911,.T.);\n#913=IFCFACE((#912));\n#914=IFCPOLYLOOP((#901,#410,#406));\n#915=IFCFACEOUTERBOUND(#914,.T.);\n#916=IFCFACE((#915));\n#917=IFCCARTESIANPOINT((-1.95275467181647,-8.49533610639887,-3.55904718691874));\n#918=IFCCARTESIANPOINT((-2.9571057900734,-8.99084305043315,-4.01506143372903));\n#919=IFCCARTESIANPOINT((-0.927846364883402,-10.,-5.41516994360616));\n#920=IFCPOLYLOOP((#917,#918,#919));\n#921=IFCFACEOUTERBOUND(#920,.T.);\n#922=IFCFACE((#921));\n#923=IFCCARTESIANPOINT((-3.02152620830207,-8.3406048241549,-3.38514401027404));\n#924=IFCPOLYLOOP((#917,#923,#918));\n#925=IFCFACEOUTERBOUND(#924,.T.);\n#926=IFCFACE((#925));\n#927=IFCCARTESIANPOINT((-1.9242135275594,-8.01310354944917,-3.10735019553297));\n#928=IFCPOLYLOOP((#927,#382,#384));\n#929=IFCFACEOUTERBOUND(#928,.T.);\n#930=IFCFACE((#929));\n#931=IFCPOLYLOOP((#927,#384,#923));\n#932=IFCFACEOUTERBOUND(#931,.T.);\n#933=IFCFACE((#932));\n#934=IFCPOLYLOOP((#927,#923,#917));\n#935=IFCFACEOUTERBOUND(#934,.T.);\n#936=IFCFACE((#935));\n#937=IFCCARTESIANPOINT((-4.35118963123758,-5.73988758280255,-1.78261784384797));\n#938=IFCPOLYLOOP((#937,#410,#866));\n#939=IFCFACEOUTERBOUND(#938,.T.);\n#940=IFCFACE((#939));\n#941=IFCPOLYLOOP((#937,#866,#883));\n#942=IFCFACEOUTERBOUND(#941,.T.);\n#943=IFCFACE((#942));\n#944=IFCCARTESIANPOINT((-1.3648131803368,-8.4929685169818,-3.5829808272175));\n#945=IFCPOLYLOOP((#944,#917,#919));\n#946=IFCFACEOUTERBOUND(#945,.T.);\n#947=IFCFACE((#946));\n#948=IFCPOLYLOOP((#944,#927,#917));\n#949=IFCFACEOUTERBOUND(#948,.T.);\n#950=IFCFACE((#949));\n#951=IFCCARTESIANPOINT((-3.92096327726286,-5.88064375112489,-1.79388820050201));\n#952=IFCPOLYLOOP((#951,#384,#383));\n#953=IFCFACEOUTERBOUND(#952,.T.);\n#954=IFCFACE((#953));\n#955=IFCPOLYLOOP((#951,#383,#353));\n#956=IFCFACEOUTERBOUND(#955,.T.);\n#957=IFCFACE((#956));\n#958=IFCPOLYLOOP((#951,#937,#883));\n#959=IFCFACEOUTERBOUND(#958,.T.);\n#960=IFCFACE((#959));\n#961=IFCCARTESIANPOINT((-4.32568438582491,-5.29397665528581,-1.60350800320523));\n#962=IFCPOLYLOOP((#961,#353,#402));\n#963=IFCFACEOUTERBOUND(#962,.T.);\n#964=IFCFACE((#963));\n#965=IFCPOLYLOOP((#961,#402,#410));\n#966=IFCFACEOUTERBOUND(#965,.T.);\n#967=IFCFACE((#966));\n#968=IFCPOLYLOOP((#961,#410,#937));\n#969=IFCFACEOUTERBOUND(#968,.T.);\n#970=IFCFACE((#969));\n#971=IFCPOLYLOOP((#961,#951,#353));\n#972=IFCFACEOUTERBOUND(#971,.T.);\n#973=IFCFACE((#972));\n#974=IFCPOLYLOOP((#961,#937,#951));\n#975=IFCFACEOUTERBOUND(#974,.T.);\n#976=IFCFACE((#975));\n#977=IFCCARTESIANPOINT((-9.54987564700599,-0.280277795327174,-4.17376671637712));\n#978=IFCCARTESIANPOINT((-10.,0.742018935106771,-5.62334781014795));\n#979=IFCPOLYLOOP((#977,#978,#824));\n#980=IFCFACEOUTERBOUND(#979,.T.);\n#981=IFCFACE((#980));\n#982=IFCPOLYLOOP((#977,#887,#888));\n#983=IFCFACEOUTERBOUND(#982,.T.);\n#984=IFCFACE((#983));\n#985=IFCPOLYLOOP((#977,#824,#887));\n#986=IFCFACEOUTERBOUND(#985,.T.);\n#987=IFCFACE((#986));\n#988=IFCCARTESIANPOINT((-0.779111195846841,-8.49143154588011,-3.61334131678681));\n#989=IFCPOLYLOOP((#988,#944,#919));\n#990=IFCFACEOUTERBOUND(#989,.T.);\n#991=IFCFACE((#990));\n#992=IFCCARTESIANPOINT((-1.11767220524124,-8.00948957297319,-3.12642946453236));\n#993=IFCPOLYLOOP((#992,#395,#382));\n#994=IFCFACEOUTERBOUND(#993,.T.);\n#995=IFCFACE((#994));\n#996=IFCPOLYLOOP((#992,#382,#927));\n#997=IFCFACEOUTERBOUND(#996,.T.);\n#998=IFCFACE((#997));\n#999=IFCPOLYLOOP((#992,#927,#944));\n#1000=IFCFACEOUTERBOUND(#999,.T.);\n#1001=IFCFACE((#1000));\n#1002=IFCPOLYLOOP((#992,#944,#988));\n#1003=IFCFACEOUTERBOUND(#1002,.T.);\n#1004=IFCFACE((#1003));\n#1005=IFCCARTESIANPOINT((-9.19239011911207,-0.31929390808654,-3.51915900422559));\n#1006=IFCPOLYLOOP((#1005,#977,#888));\n#1007=IFCFACEOUTERBOUND(#1006,.T.);\n#1008=IFCFACE((#1007));\n#1009=IFCCARTESIANPOINT((-8.35159604707338,-0.427286326494695,-2.5103989262042));\n#1010=IFCPOLYLOOP((#1005,#888,#1009));\n#1011=IFCFACEOUTERBOUND(#1010,.T.);\n#1012=IFCFACE((#1011));\n#1013=IFCCARTESIANPOINT((-7.12926954022914,-2.18985454588219,-1.73099670857445));\n#1014=IFCPOLYLOOP((#1013,#377,#901));\n#1015=IFCFACEOUTERBOUND(#1014,.T.);\n#1016=IFCFACE((#1015));\n#1017=IFCPOLYLOOP((#1013,#378,#377));\n#1018=IFCFACEOUTERBOUND(#1017,.T.);\n#1019=IFCFACE((#1018));\n#1020=IFCPOLYLOOP((#1013,#901,#406));\n#1021=IFCFACEOUTERBOUND(#1020,.T.);\n#1022=IFCFACE((#1021));\n#1023=IFCCARTESIANPOINT((-0.194688929636722,-8.49071421405792,-3.65006145059807));\n#1024=IFCPOLYLOOP((#1023,#988,#919));\n#1025=IFCFACEOUTERBOUND(#1024,.T.);\n#1026=IFCFACE((#1025));\n#1027=IFCCARTESIANPOINT((-9.2885791961374,0.200070198366522,-3.7530458394066));\n#1028=IFCPOLYLOOP((#1027,#978,#977));\n#1029=IFCFACEOUTERBOUND(#1028,.T.);\n#1030=IFCFACE((#1029));\n#1031=IFCPOLYLOOP((#1027,#977,#1005));\n#1032=IFCFACEOUTERBOUND(#1031,.T.);\n#1033=IFCFACE((#1032));\n#1034=IFCCARTESIANPOINT((-7.56950500192771,-1.5909587177281,-1.92837136235457));\n#1035=IFCPOLYLOOP((#1034,#378,#1013));\n#1036=IFCFACEOUTERBOUND(#1035,.T.);\n#1037=IFCFACE((#1036));\n#1038=IFCPOLYLOOP((#1034,#888,#378));\n#1039=IFCFACEOUTERBOUND(#1038,.T.);\n#1040=IFCFACE((#1039));\n#1041=IFCPOLYLOOP((#1034,#1009,#888));\n#1042=IFCFACEOUTERBOUND(#1041,.T.);\n#1043=IFCFACE((#1042));\n#1044=IFCCARTESIANPOINT((0.431718841053328,-8.98311394745424,-4.25300795111153));\n#1045=IFCPOLYLOOP((#1044,#919,#450));\n#1046=IFCFACEOUTERBOUND(#1045,.T.);\n#1047=IFCFACE((#1046));\n#1048=IFCPOLYLOOP((#1044,#1023,#919));\n#1049=IFCFACEOUTERBOUND(#1048,.T.);\n#1050=IFCFACE((#1049));\n#1051=IFCCARTESIANPOINT((1.99902079423989,-8.01297498818236,-3.34261720311669));\n#1052=IFCPOLYLOOP((#1044,#450,#1051));\n#1053=IFCFACEOUTERBOUND(#1052,.T.);\n#1054=IFCFACE((#1053));\n#1055=IFCCARTESIANPOINT((-0.312808996216534,-8.00773540243184,-3.16011172585437));\n#1056=IFCPOLYLOOP((#1055,#988,#1023));\n#1057=IFCFACEOUTERBOUND(#1056,.T.);\n#1058=IFCFACE((#1057));\n#1059=IFCPOLYLOOP((#1055,#421,#395));\n#1060=IFCFACEOUTERBOUND(#1059,.T.);\n#1061=IFCFACE((#1060));\n#1062=IFCPOLYLOOP((#1055,#395,#992));\n#1063=IFCFACEOUTERBOUND(#1062,.T.);\n#1064=IFCFACE((#1063));\n#1065=IFCPOLYLOOP((#1055,#992,#988));\n#1066=IFCFACEOUTERBOUND(#1065,.T.);\n#1067=IFCFACE((#1066));\n#1068=IFCCARTESIANPOINT((0.520170964594286,-8.32895669251319,-3.53255513094876));\n#1069=IFCPOLYLOOP((#1068,#1055,#1023));\n#1070=IFCFACEOUTERBOUND(#1069,.T.);\n#1071=IFCFACE((#1070));\n#1072=IFCPOLYLOOP((#1068,#1044,#1051));\n#1073=IFCFACEOUTERBOUND(#1072,.T.);\n#1074=IFCFACE((#1073));\n#1075=IFCPOLYLOOP((#1068,#1023,#1044));\n#1076=IFCFACEOUTERBOUND(#1075,.T.);\n#1077=IFCFACE((#1076));\n#1078=IFCPOLYLOOP((#1068,#1051,#421));\n#1079=IFCFACEOUTERBOUND(#1078,.T.);\n#1080=IFCFACE((#1079));\n#1081=IFCPOLYLOOP((#1068,#421,#1055));\n#1082=IFCFACEOUTERBOUND(#1081,.T.);\n#1083=IFCFACE((#1082));\n#1084=IFCCARTESIANPOINT((-7.20471013727267,-1.1226983465214,-1.7311980189908));\n#1085=IFCPOLYLOOP((#1084,#1034,#1013));\n#1086=IFCFACEOUTERBOUND(#1085,.T.);\n#1087=IFCFACE((#1086));\n#1088=IFCPOLYLOOP((#1084,#1013,#406));\n#1089=IFCFACEOUTERBOUND(#1088,.T.);\n#1090=IFCFACE((#1089));\n#1091=IFCPOLYLOOP((#1084,#1009,#1034));\n#1092=IFCFACEOUTERBOUND(#1091,.T.);\n#1093=IFCFACE((#1092));\n#1094=IFCCARTESIANPOINT((-8.99901312167587,0.810029064411353,-3.39688623131833));\n#1095=IFCPOLYLOOP((#1094,#1027,#1005));\n#1096=IFCFACEOUTERBOUND(#1095,.T.);\n#1097=IFCFACE((#1096));\n#1098=IFCCARTESIANPOINT((-7.63824595710067,0.787961486010362,-2.06010547461717));\n#1099=IFCPOLYLOOP((#1094,#1009,#1098));\n#1100=IFCFACEOUTERBOUND(#1099,.T.);\n#1101=IFCFACE((#1100));\n#1102=IFCCARTESIANPOINT((-8.32385558606928,2.0451795995373,-2.80734308873969));\n#1103=IFCPOLYLOOP((#1094,#1098,#1102));\n#1104=IFCFACEOUTERBOUND(#1103,.T.);\n#1105=IFCFACE((#1104));\n#1106=IFCPOLYLOOP((#1094,#1005,#1009));\n#1107=IFCFACEOUTERBOUND(#1106,.T.);\n#1108=IFCFACE((#1107));\n#1109=IFCCARTESIANPOINT((-9.27557846119596,1.40511722150602,-3.96677330659526));\n#1110=IFCPOLYLOOP((#1109,#978,#1027));\n#1111=IFCFACEOUTERBOUND(#1110,.T.);\n#1112=IFCFACE((#1111));\n#1113=IFCPOLYLOOP((#1109,#1027,#1094));\n#1114=IFCFACEOUTERBOUND(#1113,.T.);\n#1115=IFCFACE((#1114));\n#1116=IFCPOLYLOOP((#1109,#1094,#1102));\n#1117=IFCFACEOUTERBOUND(#1116,.T.);\n#1118=IFCFACE((#1117));\n#1119=IFCCARTESIANPOINT((-9.52995203733221,1.92060949584813,-4.61137851821725));\n#1120=IFCCARTESIANPOINT((-10.,5.14119394621751,-6.63233318896722));\n#1121=IFCPOLYLOOP((#1119,#1120,#978));\n#1122=IFCFACEOUTERBOUND(#1121,.T.);\n#1123=IFCFACE((#1122));\n#1124=IFCPOLYLOOP((#1119,#978,#1109));\n#1125=IFCFACEOUTERBOUND(#1124,.T.);\n#1126=IFCFACE((#1125));\n#1127=IFCCARTESIANPOINT((-6.82357405685001,-0.520441873808217,-1.56306073312724));\n#1128=IFCPOLYLOOP((#1127,#1084,#406));\n#1129=IFCFACEOUTERBOUND(#1128,.T.);\n#1130=IFCFACE((#1129));\n#1131=IFCPOLYLOOP((#1127,#406,#425));\n#1132=IFCFACEOUTERBOUND(#1131,.T.);\n#1133=IFCFACE((#1132));\n#1134=IFCPOLYLOOP((#1127,#425,#430));\n#1135=IFCFACEOUTERBOUND(#1134,.T.);\n#1136=IFCFACE((#1135));\n#1137=IFCCARTESIANPOINT((-9.16568016501094,1.97269570868422,-3.90983010668618));\n#1138=IFCPOLYLOOP((#1137,#1109,#1102));\n#1139=IFCFACEOUTERBOUND(#1138,.T.);\n#1140=IFCFACE((#1139));\n#1141=IFCPOLYLOOP((#1137,#1119,#1109));\n#1142=IFCFACEOUTERBOUND(#1141,.T.);\n#1143=IFCFACE((#1142));\n#1144=IFCCARTESIANPOINT((-8.91107006019535,3.1170526045441,-3.81951258755636));\n#1145=IFCPOLYLOOP((#1137,#1144,#1119));\n#1146=IFCFACEOUTERBOUND(#1145,.T.);\n#1147=IFCFACE((#1146));\n#1148=IFCPOLYLOOP((#1137,#1102,#1144));\n#1149=IFCFACEOUTERBOUND(#1148,.T.);\n#1150=IFCFACE((#1149));\n#1151=IFCCARTESIANPOINT((-7.24728903960541,0.13961928711797,-1.78339249456487));\n#1152=IFCPOLYLOOP((#1151,#1009,#1084));\n#1153=IFCFACEOUTERBOUND(#1152,.T.);\n#1154=IFCFACE((#1153));\n#1155=IFCPOLYLOOP((#1151,#1084,#1127));\n#1156=IFCFACEOUTERBOUND(#1155,.T.);\n#1157=IFCFACE((#1156));\n#1158=IFCPOLYLOOP((#1151,#1127,#430));\n#1159=IFCFACEOUTERBOUND(#1158,.T.);\n#1160=IFCFACE((#1159));\n#1161=IFCCARTESIANPOINT((-7.38127789506495,0.357094058992203,-1.86907559809831));\n#1162=IFCPOLYLOOP((#1161,#1009,#1151));\n#1163=IFCFACEOUTERBOUND(#1162,.T.);\n#1164=IFCFACE((#1163));\n#1165=IFCPOLYLOOP((#1161,#1151,#430));\n#1166=IFCFACEOUTERBOUND(#1165,.T.);\n#1167=IFCFACE((#1166));\n#1168=IFCPOLYLOOP((#1161,#1098,#1009));\n#1169=IFCFACEOUTERBOUND(#1168,.T.);\n#1170=IFCFACE((#1169));\n#1171=IFCCARTESIANPOINT((1.85022324449002,-6.45485838803672,-2.03004571134619));\n#1172=IFCPOLYLOOP((#1171,#421,#1051));\n#1173=IFCFACEOUTERBOUND(#1172,.T.);\n#1174=IFCFACE((#1173));\n#1175=IFCPOLYLOOP((#1171,#414,#421));\n#1176=IFCFACEOUTERBOUND(#1175,.T.);\n#1177=IFCFACE((#1176));\n#1178=IFCCARTESIANPOINT((-7.34710304345816,2.03612228664881,-2.02112652785579));\n#1179=IFCPOLYLOOP((#1178,#1102,#1098));\n#1180=IFCFACEOUTERBOUND(#1179,.T.);\n#1181=IFCFACE((#1180));\n#1182=IFCCARTESIANPOINT((-9.03774781562444,4.21122102437521,-4.39486589500542));\n#1183=IFCPOLYLOOP((#1182,#1120,#1119));\n#1184=IFCFACEOUTERBOUND(#1183,.T.);\n#1185=IFCFACE((#1184));\n#1186=IFCPOLYLOOP((#1182,#1119,#1144));\n#1187=IFCFACEOUTERBOUND(#1186,.T.);\n#1188=IFCFACE((#1187));\n#1189=IFCPOLYLOOP((#1182,#1144,#474));\n#1190=IFCFACEOUTERBOUND(#1189,.T.);\n#1191=IFCFACE((#1190));\n#1192=IFCCARTESIANPOINT((1.74640125827095,-5.6970367121125,-1.56280619447048));\n#1193=IFCPOLYLOOP((#1192,#414,#1171));\n#1194=IFCFACEOUTERBOUND(#1193,.T.);\n#1195=IFCFACE((#1194));\n#1196=IFCPOLYLOOP((#1192,#357,#414));\n#1197=IFCFACEOUTERBOUND(#1196,.T.);\n#1198=IFCFACE((#1197));\n#1199=IFCPOLYLOOP((#1192,#445,#357));\n#1200=IFCFACEOUTERBOUND(#1199,.T.);\n#1201=IFCFACE((#1200));\n#1202=IFCPOLYLOOP((#1192,#469,#445));\n#1203=IFCFACEOUTERBOUND(#1202,.T.);\n#1204=IFCFACE((#1203));\n#1205=IFCCARTESIANPOINT((4.00836097420805,-8.50972312964825,-4.13048839822771));\n#1206=IFCPOLYLOOP((#1205,#1051,#450));\n#1207=IFCFACEOUTERBOUND(#1206,.T.);\n#1208=IFCFACE((#1207));\n#1209=IFCPOLYLOOP((#1205,#456,#1051));\n#1210=IFCFACEOUTERBOUND(#1209,.T.);\n#1211=IFCFACE((#1210));\n#1212=IFCCARTESIANPOINT((2.44146392443501,-6.15536108212793,-1.87806232181788));\n#1213=IFCPOLYLOOP((#1212,#1192,#1171));\n#1214=IFCFACEOUTERBOUND(#1213,.T.);\n#1215=IFCFACE((#1214));\n#1216=IFCPOLYLOOP((#1212,#469,#1192));\n#1217=IFCFACEOUTERBOUND(#1216,.T.);\n#1218=IFCFACE((#1217));\n#1219=IFCCARTESIANPOINT((-7.18055196225695,2.66233235747492,-2.01466552952007));\n#1220=IFCPOLYLOOP((#1219,#1102,#1178));\n#1221=IFCFACEOUTERBOUND(#1220,.T.);\n#1222=IFCFACE((#1221));\n#1223=IFCCARTESIANPOINT((-9.0977113785221,4.75690455408155,-4.70374277476908));\n#1224=IFCPOLYLOOP((#1223,#1120,#1182));\n#1225=IFCFACEOUTERBOUND(#1224,.T.);\n#1226=IFCFACE((#1225));\n#1227=IFCCARTESIANPOINT((2.88510900116164,-6.46388641683102,-2.12755504251113));\n#1228=IFCPOLYLOOP((#1227,#1212,#1171));\n#1229=IFCFACEOUTERBOUND(#1228,.T.);\n#1230=IFCFACE((#1229));\n#1231=IFCPOLYLOOP((#1227,#1171,#1051));\n#1232=IFCFACEOUTERBOUND(#1231,.T.);\n#1233=IFCFACE((#1232));\n#1234=IFCPOLYLOOP((#1227,#1051,#456));\n#1235=IFCFACEOUTERBOUND(#1234,.T.);\n#1236=IFCFACE((#1235));\n#1237=IFCCARTESIANPOINT((4.61786808855167,-8.51605468225143,-4.23981212952599));\n#1238=IFCPOLYLOOP((#1237,#1205,#450));\n#1239=IFCFACEOUTERBOUND(#1238,.T.);\n#1240=IFCFACE((#1239));\n#1241=IFCPOLYLOOP((#1237,#450,#449));\n#1242=IFCFACEOUTERBOUND(#1241,.T.);\n#1243=IFCFACE((#1242));\n#1244=IFCPOLYLOOP((#1237,#449,#457));\n#1245=IFCFACEOUTERBOUND(#1244,.T.);\n#1246=IFCFACE((#1245));\n#1247=IFCCARTESIANPOINT((4.33555307226507,-8.03428397440321,-3.69247428276088));\n#1248=IFCPOLYLOOP((#1247,#456,#1205));\n#1249=IFCFACEOUTERBOUND(#1248,.T.);\n#1250=IFCFACE((#1249));\n#1251=IFCPOLYLOOP((#1247,#1205,#1237));\n#1252=IFCFACEOUTERBOUND(#1251,.T.);\n#1253=IFCFACE((#1252));\n#1254=IFCPOLYLOOP((#1247,#1237,#457));\n#1255=IFCFACEOUTERBOUND(#1254,.T.);\n#1256=IFCFACE((#1255));\n#1257=IFCPOLYLOOP((#1247,#457,#456));\n#1258=IFCFACEOUTERBOUND(#1257,.T.);\n#1259=IFCFACE((#1258));\n#1260=IFCCARTESIANPOINT((-7.25432762006644,3.38652703084471,-2.20029073505893));\n#1261=IFCPOLYLOOP((#1260,#1219,#438));\n#1262=IFCFACEOUTERBOUND(#1261,.T.);\n#1263=IFCFACE((#1262));\n#1264=IFCPOLYLOOP((#1260,#475,#474));\n#1265=IFCFACEOUTERBOUND(#1264,.T.);\n#1266=IFCFACE((#1265));\n#1267=IFCCARTESIANPOINT((-8.74691129583695,5.02472007874072,-4.26775143748174));\n#1268=IFCPOLYLOOP((#1267,#474,#473));\n#1269=IFCFACEOUTERBOUND(#1268,.T.);\n#1270=IFCFACE((#1269));\n#1271=IFCPOLYLOOP((#1267,#1182,#474));\n#1272=IFCFACEOUTERBOUND(#1271,.T.);\n#1273=IFCFACE((#1272));\n#1274=IFCPOLYLOOP((#1267,#1223,#1182));\n#1275=IFCFACEOUTERBOUND(#1274,.T.);\n#1276=IFCFACE((#1275));\n#1277=IFCCARTESIANPOINT((-9.34716000649849,5.67764045442664,-5.48962889338229));\n#1278=IFCCARTESIANPOINT((-10.,7.48864707728979,-6.98888025773325));\n#1279=IFCPOLYLOOP((#1277,#1278,#1120));\n#1280=IFCFACEOUTERBOUND(#1279,.T.);\n#1281=IFCFACE((#1280));\n#1282=IFCCARTESIANPOINT((-9.04308333582316,5.36904720330309,-4.86174331308828));\n#1283=IFCPOLYLOOP((#1282,#1277,#1120));\n#1284=IFCFACEOUTERBOUND(#1283,.T.);\n#1285=IFCFACE((#1284));\n#1286=IFCPOLYLOOP((#1282,#1120,#1223));\n#1287=IFCFACEOUTERBOUND(#1286,.T.);\n#1288=IFCFACE((#1287));\n#1289=IFCPOLYLOOP((#1282,#1223,#1267));\n#1290=IFCFACEOUTERBOUND(#1289,.T.);\n#1291=IFCFACE((#1290));\n#1292=IFCCARTESIANPOINT((-8.9039594012134,5.6414122755279,-4.76370479490522));\n#1293=IFCPOLYLOOP((#1292,#473,#480));\n#1294=IFCFACEOUTERBOUND(#1293,.T.);\n#1295=IFCFACE((#1294));\n#1296=IFCPOLYLOOP((#1292,#1267,#473));\n#1297=IFCFACEOUTERBOUND(#1296,.T.);\n#1298=IFCFACE((#1297));\n#1299=IFCPOLYLOOP((#1292,#480,#1277));\n#1300=IFCFACEOUTERBOUND(#1299,.T.);\n#1301=IFCFACE((#1300));\n#1302=IFCPOLYLOOP((#1292,#1277,#1282));\n#1303=IFCFACEOUTERBOUND(#1302,.T.);\n#1304=IFCFACE((#1303));\n#1305=IFCPOLYLOOP((#1292,#1282,#1267));\n#1306=IFCFACEOUTERBOUND(#1305,.T.);\n#1307=IFCFACE((#1306));\n#1308=IFCCARTESIANPOINT((3.34581092659577,-5.71320614722157,-1.69622118301015));\n#1309=IFCPOLYLOOP((#1308,#469,#1212));\n#1310=IFCFACEOUTERBOUND(#1309,.T.);\n#1311=IFCFACE((#1310));\n#1312=IFCPOLYLOOP((#1308,#1212,#1227));\n#1313=IFCFACEOUTERBOUND(#1312,.T.);\n#1314=IFCFACE((#1313));\n#1315=IFCCARTESIANPOINT((3.68754544740715,-5.8687029854048,-1.83224574705951));\n#1316=IFCPOLYLOOP((#1315,#1227,#456));\n#1317=IFCFACEOUTERBOUND(#1316,.T.);\n#1318=IFCFACE((#1317));\n#1319=IFCPOLYLOOP((#1315,#1308,#1227));\n#1320=IFCFACEOUTERBOUND(#1319,.T.);\n#1321=IFCFACE((#1320));\n#1322=IFCPOLYLOOP((#1315,#456,#455));\n#1323=IFCFACEOUTERBOUND(#1322,.T.);\n#1324=IFCFACE((#1323));\n#1325=IFCCARTESIANPOINT((-6.65465392983592,4.05436252961744,-1.98062918263708));\n#1326=IFCPOLYLOOP((#1325,#438,#504));\n#1327=IFCFACEOUTERBOUND(#1326,.T.);\n#1328=IFCFACE((#1327));\n#1329=IFCPOLYLOOP((#1325,#475,#1260));\n#1330=IFCFACEOUTERBOUND(#1329,.T.);\n#1331=IFCFACE((#1330));\n#1332=IFCPOLYLOOP((#1325,#1260,#438));\n#1333=IFCFACEOUTERBOUND(#1332,.T.);\n#1334=IFCFACE((#1333));\n#1335=IFCPOLYLOOP((#1325,#504,#562));\n#1336=IFCFACEOUTERBOUND(#1335,.T.);\n#1337=IFCFACE((#1336));\n#1338=IFCCARTESIANPOINT((-7.15863219856347,6.00247343923351,-2.99175918994072));\n#1339=IFCPOLYLOOP((#1338,#479,#473));\n#1340=IFCFACEOUTERBOUND(#1339,.T.);\n#1341=IFCFACE((#1340));\n#1342=IFCPOLYLOOP((#1338,#473,#475));\n#1343=IFCFACEOUTERBOUND(#1342,.T.);\n#1344=IFCFACE((#1343));\n#1345=IFCCARTESIANPOINT((-6.82795051408118,5.66810994597095,-2.59386113102659));\n#1346=IFCPOLYLOOP((#1345,#475,#1325));\n#1347=IFCFACEOUTERBOUND(#1346,.T.);\n#1348=IFCFACE((#1347));\n#1349=IFCPOLYLOOP((#1345,#1325,#562));\n#1350=IFCFACEOUTERBOUND(#1349,.T.);\n#1351=IFCFACE((#1350));\n#1352=IFCPOLYLOOP((#1345,#1338,#475));\n#1353=IFCFACEOUTERBOUND(#1352,.T.);\n#1354=IFCFACE((#1353));\n#1355=IFCCARTESIANPOINT((5.31197922769667,-5.75244744672125,-2.06799193706778));\n#1356=IFCPOLYLOOP((#1355,#455,#457));\n#1357=IFCFACEOUTERBOUND(#1356,.T.);\n#1358=IFCFACE((#1357));\n#1359=IFCCARTESIANPOINT((4.60713208006776,-4.8417953141852,-1.40687199488705));\n#1360=IFCCARTESIANPOINT((5.585146484375,-4.42678466796875,-1.41629638671875));\n#1361=IFCPOLYLOOP((#1359,#1355,#1360));\n#1362=IFCFACEOUTERBOUND(#1361,.T.);\n#1363=IFCFACE((#1362));\n#1364=IFCPOLYLOOP((#1359,#455,#1355));\n#1365=IFCFACEOUTERBOUND(#1364,.T.);\n#1366=IFCFACE((#1365));\n#1367=IFCPOLYLOOP((#1359,#526,#469));\n#1368=IFCFACEOUTERBOUND(#1367,.T.);\n#1369=IFCFACE((#1368));\n#1370=IFCPOLYLOOP((#1359,#1360,#526));\n#1371=IFCFACEOUTERBOUND(#1370,.T.);\n#1372=IFCFACE((#1371));\n#1373=IFCPOLYLOOP((#1359,#469,#1308));\n#1374=IFCFACEOUTERBOUND(#1373,.T.);\n#1375=IFCFACE((#1374));\n#1376=IFCPOLYLOOP((#1359,#1308,#1315));\n#1377=IFCFACEOUTERBOUND(#1376,.T.);\n#1378=IFCFACE((#1377));\n#1379=IFCPOLYLOOP((#1359,#1315,#455));\n#1380=IFCFACEOUTERBOUND(#1379,.T.);\n#1381=IFCFACE((#1380));\n#1382=IFCCARTESIANPOINT((5.64842638879872,-6.21140177495042,-2.46692522528948));\n#1383=IFCPOLYLOOP((#1382,#457,#500));\n#1384=IFCFACEOUTERBOUND(#1383,.T.);\n#1385=IFCFACE((#1384));\n#1386=IFCPOLYLOOP((#1382,#1355,#457));\n#1387=IFCFACEOUTERBOUND(#1386,.T.);\n#1388=IFCFACE((#1387));\n#1389=IFCCARTESIANPOINT((-9.43659321086565,8.38604589670602,-6.51451274218262));\n#1390=IFCCARTESIANPOINT((-10.,10.,-7.13));\n#1391=IFCPOLYLOOP((#1389,#1390,#1278));\n#1392=IFCFACEOUTERBOUND(#1391,.T.);\n#1393=IFCFACE((#1392));\n#1394=IFCCARTESIANPOINT((-8.77626584587613,9.22952252042661,-6.27444294130975));\n#1395=IFCPOLYLOOP((#1389,#1394,#1390));\n#1396=IFCFACEOUTERBOUND(#1395,.T.);\n#1397=IFCFACE((#1396));\n#1398=IFCCARTESIANPOINT((-8.67522555117947,7.68543816623009,-5.44018687327973));\n#1399=IFCCARTESIANPOINT((-7.35248942492495,7.84076413543983,-4.25742265265999));\n#1400=IFCPOLYLOOP((#1398,#479,#1399));\n#1401=IFCFACEOUTERBOUND(#1400,.T.);\n#1402=IFCFACE((#1401));\n#1403=IFCPOLYLOOP((#1398,#480,#479));\n#1404=IFCFACEOUTERBOUND(#1403,.T.);\n#1405=IFCFACE((#1404));\n#1406=IFCPOLYLOOP((#1398,#1278,#1277));\n#1407=IFCFACEOUTERBOUND(#1406,.T.);\n#1408=IFCFACE((#1407));\n#1409=IFCPOLYLOOP((#1398,#1277,#480));\n#1410=IFCFACEOUTERBOUND(#1409,.T.);\n#1411=IFCFACE((#1410));\n#1412=IFCCARTESIANPOINT((-9.17664561709023,8.22905742598488,-6.20816012872));\n#1413=IFCPOLYLOOP((#1412,#1278,#1398));\n#1414=IFCFACEOUTERBOUND(#1413,.T.);\n#1415=IFCFACE((#1414));\n#1416=IFCPOLYLOOP((#1412,#1389,#1278));\n#1417=IFCFACEOUTERBOUND(#1416,.T.);\n#1418=IFCFACE((#1417));\n#1419=IFCCARTESIANPOINT((7.44857126287114,-8.193591026814,-4.64559421370549));\n#1420=IFCPOLYLOOP((#1419,#500,#449));\n#1421=IFCFACEOUTERBOUND(#1420,.T.);\n#1422=IFCFACE((#1421));\n#1423=IFCPOLYLOOP((#1419,#449,#451));\n#1424=IFCFACEOUTERBOUND(#1423,.T.);\n#1425=IFCFACE((#1424));\n#1426=IFCCARTESIANPOINT((-9.05054191372204,8.5660825646423,-6.21429467204169));\n#1427=IFCPOLYLOOP((#1426,#1394,#1389));\n#1428=IFCFACEOUTERBOUND(#1427,.T.);\n#1429=IFCFACE((#1428));\n#1430=IFCPOLYLOOP((#1426,#1389,#1412));\n#1431=IFCFACEOUTERBOUND(#1430,.T.);\n#1432=IFCFACE((#1431));\n#1433=IFCCARTESIANPOINT((-8.26024993237626,8.90001352516431,-5.73485822132526));\n#1434=IFCPOLYLOOP((#1426,#1433,#1394));\n#1435=IFCFACEOUTERBOUND(#1434,.T.);\n#1436=IFCFACE((#1435));\n#1437=IFCCARTESIANPOINT((-8.54492238495358,8.09444610638784,-5.51962571357029));\n#1438=IFCPOLYLOOP((#1437,#1398,#1399));\n#1439=IFCFACEOUTERBOUND(#1438,.T.);\n#1440=IFCFACE((#1439));\n#1441=IFCPOLYLOOP((#1437,#1412,#1398));\n#1442=IFCFACEOUTERBOUND(#1441,.T.);\n#1443=IFCFACE((#1442));\n#1444=IFCPOLYLOOP((#1437,#1426,#1412));\n#1445=IFCFACEOUTERBOUND(#1444,.T.);\n#1446=IFCFACE((#1445));\n#1447=IFCPOLYLOOP((#1437,#1433,#1426));\n#1448=IFCFACEOUTERBOUND(#1447,.T.);\n#1449=IFCFACE((#1448));\n#1450=IFCCARTESIANPOINT((-6.72695841282169,6.22358762790177,-2.76797106690877));\n#1451=IFCPOLYLOOP((#1450,#1345,#562));\n#1452=IFCFACEOUTERBOUND(#1451,.T.);\n#1453=IFCFACE((#1452));\n#1454=IFCPOLYLOOP((#1450,#1338,#1345));\n#1455=IFCFACEOUTERBOUND(#1454,.T.);\n#1456=IFCFACE((#1455));\n#1457=IFCCARTESIANPOINT((5.97027186894616,-5.77147624376931,-2.27105026946226));\n#1458=IFCPOLYLOOP((#1457,#1360,#1355));\n#1459=IFCFACEOUTERBOUND(#1458,.T.);\n#1460=IFCFACE((#1459));\n#1461=IFCPOLYLOOP((#1457,#1355,#1382));\n#1462=IFCFACEOUTERBOUND(#1461,.T.);\n#1463=IFCFACE((#1462));\n#1464=IFCCARTESIANPOINT((8.07279867875318,-8.72553331391621,-5.24091462274464));\n#1465=IFCPOLYLOOP((#1464,#1419,#451));\n#1466=IFCFACEOUTERBOUND(#1465,.T.);\n#1467=IFCFACE((#1466));\n#1468=IFCCARTESIANPOINT((-6.50569617091498,6.81752655528081,-2.92166429028506));\n#1469=IFCPOLYLOOP((#1468,#479,#1338));\n#1470=IFCFACEOUTERBOUND(#1469,.T.);\n#1471=IFCFACE((#1470));\n#1472=IFCPOLYLOOP((#1468,#1399,#479));\n#1473=IFCFACEOUTERBOUND(#1472,.T.);\n#1474=IFCFACE((#1473));\n#1475=IFCPOLYLOOP((#1468,#1338,#1450));\n#1476=IFCFACEOUTERBOUND(#1475,.T.);\n#1477=IFCFACE((#1476));\n#1478=IFCPOLYLOOP((#1468,#567,#1399));\n#1479=IFCFACEOUTERBOUND(#1478,.T.);\n#1480=IFCFACE((#1479));\n#1481=IFCCARTESIANPOINT((6.59914619267507,-5.74351521436717,-2.48481981320681));\n#1482=IFCPOLYLOOP((#1481,#1382,#500));\n#1483=IFCFACEOUTERBOUND(#1482,.T.);\n#1484=IFCFACE((#1483));\n#1485=IFCPOLYLOOP((#1481,#1457,#1382));\n#1486=IFCFACEOUTERBOUND(#1485,.T.);\n#1487=IFCFACE((#1486));\n#1488=IFCCARTESIANPOINT((7.61323958934521,-6.72653195706452,-3.63450410041427));\n#1489=IFCPOLYLOOP((#1488,#1481,#500));\n#1490=IFCFACEOUTERBOUND(#1489,.T.);\n#1491=IFCFACE((#1490));\n#1492=IFCCARTESIANPOINT((7.80993618576276,-5.69881971654383,-3.08833287267413));\n#1493=IFCPOLYLOOP((#1488,#1492,#1481));\n#1494=IFCFACEOUTERBOUND(#1493,.T.);\n#1495=IFCFACE((#1494));\n#1496=IFCPOLYLOOP((#1488,#500,#1419));\n#1497=IFCFACEOUTERBOUND(#1496,.T.);\n#1498=IFCFACE((#1497));\n#1499=IFCCARTESIANPOINT((-5.83631670883889,6.88236140181534,-2.53684487697602));\n#1500=IFCPOLYLOOP((#1499,#1450,#562));\n#1501=IFCFACEOUTERBOUND(#1500,.T.);\n#1502=IFCFACE((#1501));\n#1503=IFCPOLYLOOP((#1499,#1468,#1450));\n#1504=IFCFACEOUTERBOUND(#1503,.T.);\n#1505=IFCFACE((#1504));\n#1506=IFCCARTESIANPOINT((6.73367613797833,-5.05970148681632,-2.13247998701578));\n#1507=IFCPOLYLOOP((#1506,#1360,#1457));\n#1508=IFCFACEOUTERBOUND(#1507,.T.);\n#1509=IFCFACE((#1508));\n#1510=IFCPOLYLOOP((#1506,#1481,#1492));\n#1511=IFCFACEOUTERBOUND(#1510,.T.);\n#1512=IFCFACE((#1511));\n#1513=IFCPOLYLOOP((#1506,#1457,#1481));\n#1514=IFCFACEOUTERBOUND(#1513,.T.);\n#1515=IFCFACE((#1514));\n#1516=IFCCARTESIANPOINT((8.11140093115313,-6.99556618837935,-4.09540779617112));\n#1517=IFCCARTESIANPOINT((9.14150417151072,-7.53492138398366,-5.04947335753853));\n#1518=IFCPOLYLOOP((#1516,#1464,#1517));\n#1519=IFCFACEOUTERBOUND(#1518,.T.);\n#1520=IFCFACE((#1519));\n#1521=IFCPOLYLOOP((#1516,#1419,#1464));\n#1522=IFCFACEOUTERBOUND(#1521,.T.);\n#1523=IFCFACE((#1522));\n#1524=IFCPOLYLOOP((#1516,#1488,#1419));\n#1525=IFCFACEOUTERBOUND(#1524,.T.);\n#1526=IFCFACE((#1525));\n#1527=IFCPOLYLOOP((#1516,#1492,#1488));\n#1528=IFCFACEOUTERBOUND(#1527,.T.);\n#1529=IFCFACE((#1528));\n#1530=IFCCARTESIANPOINT((8.85724792366807,-6.34218288764684,-4.24904880287931));\n#1531=IFCPOLYLOOP((#1516,#1530,#1492));\n#1532=IFCFACEOUTERBOUND(#1531,.T.);\n#1533=IFCFACE((#1532));\n#1534=IFCPOLYLOOP((#1516,#1517,#1530));\n#1535=IFCFACEOUTERBOUND(#1534,.T.);\n#1536=IFCFACE((#1535));\n#1537=IFCCARTESIANPOINT((9.50633189233415,-8.7533085188268,-5.72191527232702));\n#1538=IFCPOLYLOOP((#1537,#1517,#1464));\n#1539=IFCFACEOUTERBOUND(#1538,.T.);\n#1540=IFCFACE((#1539));\n#1541=IFCCARTESIANPOINT((-7.77392973831147,9.41167343652016,-5.75848891869463));\n#1542=IFCPOLYLOOP((#1541,#1394,#1433));\n#1543=IFCFACEOUTERBOUND(#1542,.T.);\n#1544=IFCFACE((#1543));\n#1545=IFCCARTESIANPOINT((-7.30812506713516,8.94586774318046,-5.09236493200615));\n#1546=IFCPOLYLOOP((#1545,#1437,#1399));\n#1547=IFCFACEOUTERBOUND(#1546,.T.);\n#1548=IFCFACE((#1547));\n#1549=IFCPOLYLOOP((#1545,#1433,#1437));\n#1550=IFCFACEOUTERBOUND(#1549,.T.);\n#1551=IFCFACE((#1550));\n#1552=IFCPOLYLOOP((#1545,#1541,#1433));\n#1553=IFCFACEOUTERBOUND(#1552,.T.);\n#1554=IFCFACE((#1553));\n#1555=IFCCARTESIANPOINT((-5.11899060598054,6.97112556856096,-2.21583639352025));\n#1556=IFCPOLYLOOP((#1555,#1499,#562));\n#1557=IFCFACEOUTERBOUND(#1556,.T.);\n#1558=IFCFACE((#1557));\n#1559=IFCPOLYLOOP((#1555,#562,#558));\n#1560=IFCFACEOUTERBOUND(#1559,.T.);\n#1561=IFCFACE((#1560));\n#1562=IFCPOLYLOOP((#1555,#558,#568));\n#1563=IFCFACEOUTERBOUND(#1562,.T.);\n#1564=IFCFACE((#1563));\n#1565=IFCCARTESIANPOINT((-5.43201997204309,7.30654016557842,-2.56281458071381));\n#1566=IFCPOLYLOOP((#1565,#1468,#1499));\n#1567=IFCFACEOUTERBOUND(#1566,.T.);\n#1568=IFCFACE((#1567));\n#1569=IFCPOLYLOOP((#1565,#567,#1468));\n#1570=IFCFACEOUTERBOUND(#1569,.T.);\n#1571=IFCFACE((#1570));\n#1572=IFCPOLYLOOP((#1565,#1499,#1555));\n#1573=IFCFACEOUTERBOUND(#1572,.T.);\n#1574=IFCFACE((#1573));\n#1575=IFCPOLYLOOP((#1565,#568,#567));\n#1576=IFCFACEOUTERBOUND(#1575,.T.);\n#1577=IFCFACE((#1576));\n#1578=IFCPOLYLOOP((#1565,#1555,#568));\n#1579=IFCFACEOUTERBOUND(#1578,.T.);\n#1580=IFCFACE((#1579));\n#1581=IFCCARTESIANPOINT((-6.27261482556783,8.3151433535983,-3.83126199614358));\n#1582=IFCPOLYLOOP((#1581,#1399,#567));\n#1583=IFCFACEOUTERBOUND(#1582,.T.);\n#1584=IFCFACE((#1583));\n#1585=IFCPOLYLOOP((#1581,#567,#566));\n#1586=IFCFACEOUTERBOUND(#1585,.T.);\n#1587=IFCFACE((#1586));\n#1588=IFCCARTESIANPOINT((-7.52012073198568,9.65438568554864,-5.81325336048377));\n#1589=IFCCARTESIANPOINT((-6.27029378099855,10.,-5.65309973092067));\n#1590=IFCPOLYLOOP((#1588,#1589,#1390));\n#1591=IFCFACEOUTERBOUND(#1590,.T.);\n#1592=IFCFACE((#1591));\n#1593=IFCPOLYLOOP((#1588,#1390,#1394));\n#1594=IFCFACEOUTERBOUND(#1593,.T.);\n#1595=IFCFACE((#1594));\n#1596=IFCPOLYLOOP((#1588,#1394,#1541));\n#1597=IFCFACEOUTERBOUND(#1596,.T.);\n#1598=IFCFACE((#1597));\n#1599=IFCCARTESIANPOINT((-6.84371313874416,8.96628615319708,-4.80700937418496));\n#1600=IFCPOLYLOOP((#1599,#1545,#1399));\n#1601=IFCFACEOUTERBOUND(#1600,.T.);\n#1602=IFCFACE((#1601));\n#1603=IFCPOLYLOOP((#1599,#1399,#1581));\n#1604=IFCFACEOUTERBOUND(#1603,.T.);\n#1605=IFCFACE((#1604));\n#1606=IFCCARTESIANPOINT((6.17530526332934,-3.27895966058777,-1.118896648616));\n#1607=IFCPOLYLOOP((#1606,#526,#1360));\n#1608=IFCFACEOUTERBOUND(#1607,.T.);\n#1609=IFCFACE((#1608));\n#1610=IFCCARTESIANPOINT((-7.2812100226037,9.49307662222451,-5.55216480725473));\n#1611=IFCPOLYLOOP((#1610,#1589,#1588));\n#1612=IFCFACEOUTERBOUND(#1611,.T.);\n#1613=IFCFACE((#1612));\n#1614=IFCPOLYLOOP((#1610,#1599,#1589));\n#1615=IFCFACEOUTERBOUND(#1614,.T.);\n#1616=IFCFACE((#1615));\n#1617=IFCPOLYLOOP((#1610,#1541,#1545));\n#1618=IFCFACEOUTERBOUND(#1617,.T.);\n#1619=IFCFACE((#1618));\n#1620=IFCPOLYLOOP((#1610,#1545,#1599));\n#1621=IFCFACEOUTERBOUND(#1620,.T.);\n#1622=IFCFACE((#1621));\n#1623=IFCPOLYLOOP((#1610,#1588,#1541));\n#1624=IFCFACEOUTERBOUND(#1623,.T.);\n#1625=IFCFACE((#1624));\n#1626=IFCCARTESIANPOINT((7.22002376043364,-4.59555525600479,-2.11885236931153));\n#1627=IFCPOLYLOOP((#1626,#1506,#1492));\n#1628=IFCFACEOUTERBOUND(#1627,.T.);\n#1629=IFCFACE((#1628));\n#1630=IFCCARTESIANPOINT((6.90926721107077,-4.04476129410581,-1.70113142900951));\n#1631=IFCPOLYLOOP((#1630,#1360,#1506));\n#1632=IFCFACEOUTERBOUND(#1631,.T.);\n#1633=IFCFACE((#1632));\n#1634=IFCPOLYLOOP((#1630,#1606,#1360));\n#1635=IFCFACEOUTERBOUND(#1634,.T.);\n#1636=IFCFACE((#1635));\n#1637=IFCPOLYLOOP((#1630,#1506,#1626));\n#1638=IFCFACEOUTERBOUND(#1637,.T.);\n#1639=IFCFACE((#1638));\n#1640=IFCCARTESIANPOINT((6.08748829709309,-2.7885508978305,-0.935494073508391));\n#1641=IFCPOLYLOOP((#1640,#519,#518));\n#1642=IFCFACEOUTERBOUND(#1641,.T.);\n#1643=IFCFACE((#1642));\n#1644=IFCPOLYLOOP((#1640,#526,#1606));\n#1645=IFCFACEOUTERBOUND(#1644,.T.);\n#1646=IFCFACE((#1645));\n#1647=IFCPOLYLOOP((#1640,#518,#526));\n#1648=IFCFACEOUTERBOUND(#1647,.T.);\n#1649=IFCFACE((#1648));\n#1650=IFCCARTESIANPOINT((-6.28046087481535,8.88766386969201,-4.38109340004564));\n#1651=IFCPOLYLOOP((#1650,#1589,#1599));\n#1652=IFCFACEOUTERBOUND(#1651,.T.);\n#1653=IFCFACE((#1652));\n#1654=IFCPOLYLOOP((#1650,#1581,#566));\n#1655=IFCFACEOUTERBOUND(#1654,.T.);\n#1656=IFCFACE((#1655));\n#1657=IFCPOLYLOOP((#1650,#1599,#1581));\n#1658=IFCFACEOUTERBOUND(#1657,.T.);\n#1659=IFCFACE((#1658));\n#1660=IFCCARTESIANPOINT((7.08965576917782,-3.71605329976539,-1.64497496381168));\n#1661=IFCPOLYLOOP((#1660,#599,#598));\n#1662=IFCFACEOUTERBOUND(#1661,.T.);\n#1663=IFCFACE((#1662));\n#1664=IFCPOLYLOOP((#1660,#598,#1606));\n#1665=IFCFACEOUTERBOUND(#1664,.T.);\n#1666=IFCFACE((#1665));\n#1667=IFCPOLYLOOP((#1660,#1626,#599));\n#1668=IFCFACEOUTERBOUND(#1667,.T.);\n#1669=IFCFACE((#1668));\n#1670=IFCPOLYLOOP((#1660,#1606,#1630));\n#1671=IFCFACEOUTERBOUND(#1670,.T.);\n#1672=IFCFACE((#1671));\n#1673=IFCPOLYLOOP((#1660,#1630,#1626));\n#1674=IFCFACEOUTERBOUND(#1673,.T.);\n#1675=IFCFACE((#1674));\n#1676=IFCCARTESIANPOINT((7.23615299810435,-2.42460349497813,-1.26331352340013));\n#1677=IFCPOLYLOOP((#1676,#1606,#598));\n#1678=IFCFACEOUTERBOUND(#1677,.T.);\n#1679=IFCFACE((#1678));\n#1680=IFCPOLYLOOP((#1676,#598,#597));\n#1681=IFCFACEOUTERBOUND(#1680,.T.);\n#1682=IFCFACE((#1681));\n#1683=IFCPOLYLOOP((#1676,#1640,#1606));\n#1684=IFCFACEOUTERBOUND(#1683,.T.);\n#1685=IFCFACE((#1684));\n#1686=IFCCARTESIANPOINT((-2.99455440395077,7.29387057550375,-1.64094613228509));\n#1687=IFCPOLYLOOP((#1686,#637,#586));\n#1688=IFCFACEOUTERBOUND(#1687,.T.);\n#1689=IFCFACE((#1688));\n#1690=IFCPOLYLOOP((#1686,#558,#362));\n#1691=IFCFACEOUTERBOUND(#1690,.T.);\n#1692=IFCFACE((#1691));\n#1693=IFCPOLYLOOP((#1686,#586,#558));\n#1694=IFCFACEOUTERBOUND(#1693,.T.);\n#1695=IFCFACE((#1694));\n#1696=IFCCARTESIANPOINT((7.13134316918613,-2.10367593373974,-1.12402610135917));\n#1697=IFCPOLYLOOP((#1696,#1640,#1676));\n#1698=IFCFACEOUTERBOUND(#1697,.T.);\n#1699=IFCFACE((#1698));\n#1700=IFCPOLYLOOP((#1696,#519,#1640));\n#1701=IFCFACEOUTERBOUND(#1700,.T.);\n#1702=IFCFACE((#1701));\n#1703=IFCCARTESIANPOINT((-4.43572891828874,9.00867907343722,-3.57638084851729));\n#1704=IFCPOLYLOOP((#1703,#566,#568));\n#1705=IFCFACEOUTERBOUND(#1704,.T.);\n#1706=IFCFACE((#1705));\n#1707=IFCCARTESIANPOINT((-2.42861224107324,7.20883446561597,-1.46724893185791));\n#1708=IFCPOLYLOOP((#1707,#632,#637));\n#1709=IFCFACEOUTERBOUND(#1708,.T.);\n#1710=IFCFACE((#1709));\n#1711=IFCPOLYLOOP((#1707,#362,#632));\n#1712=IFCFACEOUTERBOUND(#1711,.T.);\n#1713=IFCFACE((#1712));\n#1714=IFCPOLYLOOP((#1707,#637,#1686));\n#1715=IFCFACEOUTERBOUND(#1714,.T.);\n#1716=IFCFACE((#1715));\n#1717=IFCPOLYLOOP((#1707,#1686,#362));\n#1718=IFCFACEOUTERBOUND(#1717,.T.);\n#1719=IFCFACE((#1718));\n#1720=IFCCARTESIANPOINT((-4.06828794514315,9.13496168859957,-3.59023790915396));\n#1721=IFCPOLYLOOP((#1720,#1703,#568));\n#1722=IFCFACEOUTERBOUND(#1721,.T.);\n#1723=IFCFACE((#1722));\n#1724=IFCCARTESIANPOINT((-4.3307938254193,9.42440045765249,-4.10016717638871));\n#1725=IFCPOLYLOOP((#1724,#1589,#1650));\n#1726=IFCFACEOUTERBOUND(#1725,.T.);\n#1727=IFCFACE((#1726));\n#1728=IFCCARTESIANPOINT((-3.49725143797288,10.,-4.89456992465945));\n#1729=IFCPOLYLOOP((#1724,#1728,#1589));\n#1730=IFCFACEOUTERBOUND(#1729,.T.);\n#1731=IFCFACE((#1730));\n#1732=IFCPOLYLOOP((#1724,#566,#1703));\n#1733=IFCFACEOUTERBOUND(#1732,.T.);\n#1734=IFCFACE((#1733));\n#1735=IFCPOLYLOOP((#1724,#1720,#1728));\n#1736=IFCFACEOUTERBOUND(#1735,.T.);\n#1737=IFCFACE((#1736));\n#1738=IFCPOLYLOOP((#1724,#1703,#1720));\n#1739=IFCFACEOUTERBOUND(#1738,.T.);\n#1740=IFCFACE((#1739));\n#1741=IFCPOLYLOOP((#1724,#1650,#566));\n#1742=IFCFACEOUTERBOUND(#1741,.T.);\n#1743=IFCFACE((#1742));\n#1744=IFCCARTESIANPOINT((7.19327211882919,-1.24280258239014,-0.961345032358077));\n#1745=IFCPOLYLOOP((#1744,#519,#1696));\n#1746=IFCFACEOUTERBOUND(#1745,.T.);\n#1747=IFCFACE((#1746));\n#1748=IFCCARTESIANPOINT((9.53488416393737,-4.68772290122735,-4.34564151253092));\n#1749=IFCCARTESIANPOINT((9.08157981340111,-4.35591562541041,-3.57243303776717));\n#1750=IFCCARTESIANPOINT((9.42511351073271,-5.61437809168869,-4.51697796645186));\n#1751=IFCPOLYLOOP((#1748,#1749,#1750));\n#1752=IFCFACEOUTERBOUND(#1751,.T.);\n#1753=IFCFACE((#1752));\n#1754=IFCCARTESIANPOINT((9.6738453355927,-5.26190925385743,-4.74242869149835));\n#1755=IFCCARTESIANPOINT((10.,-3.13481481481482,-4.96950617283951));\n#1756=IFCPOLYLOOP((#1748,#1754,#1755));\n#1757=IFCFACEOUTERBOUND(#1756,.T.);\n#1758=IFCFACE((#1757));\n#1759=IFCPOLYLOOP((#1748,#1750,#1754));\n#1760=IFCFACEOUTERBOUND(#1759,.T.);\n#1761=IFCFACE((#1760));\n#1762=IFCCARTESIANPOINT((7.56153455886919,-1.63754396419358,-1.23712199825569));\n#1763=IFCPOLYLOOP((#1762,#1696,#1676));\n#1764=IFCFACEOUTERBOUND(#1763,.T.);\n#1765=IFCFACE((#1764));\n#1766=IFCPOLYLOOP((#1762,#597,#642));\n#1767=IFCFACEOUTERBOUND(#1766,.T.);\n#1768=IFCFACE((#1767));\n#1769=IFCPOLYLOOP((#1762,#1676,#597));\n#1770=IFCFACEOUTERBOUND(#1769,.T.);\n#1771=IFCFACE((#1770));\n#1772=IFCPOLYLOOP((#1762,#1744,#1696));\n#1773=IFCFACEOUTERBOUND(#1772,.T.);\n#1774=IFCFACE((#1773));\n#1775=IFCCARTESIANPOINT((6.81843924434097,-0.558581892848015,-0.697390893086975));\n#1776=IFCPOLYLOOP((#1775,#519,#1744));\n#1777=IFCFACEOUTERBOUND(#1776,.T.);\n#1778=IFCFACE((#1777));\n#1779=IFCCARTESIANPOINT((-3.57288431677985,9.06102172009655,-3.31290926806071));\n#1780=IFCPOLYLOOP((#1779,#586,#636));\n#1781=IFCFACEOUTERBOUND(#1780,.T.);\n#1782=IFCFACE((#1781));\n#1783=IFCPOLYLOOP((#1779,#1728,#1720));\n#1784=IFCFACEOUTERBOUND(#1783,.T.);\n#1785=IFCFACE((#1784));\n#1786=IFCPOLYLOOP((#1779,#1720,#568));\n#1787=IFCFACEOUTERBOUND(#1786,.T.);\n#1788=IFCFACE((#1787));\n#1789=IFCPOLYLOOP((#1779,#568,#586));\n#1790=IFCFACEOUTERBOUND(#1789,.T.);\n#1791=IFCFACE((#1790));\n#1792=IFCCARTESIANPOINT((6.55292271500682,-0.100308754434876,-0.557007408020422));\n#1793=IFCCARTESIANPOINT((7.63935185185185,0.645254629629629,-0.988680555555557));\n#1794=IFCPOLYLOOP((#1792,#1793,#625));\n#1795=IFCFACEOUTERBOUND(#1794,.T.);\n#1796=IFCFACE((#1795));\n#1797=IFCPOLYLOOP((#1792,#519,#1775));\n#1798=IFCFACEOUTERBOUND(#1797,.T.);\n#1799=IFCFACE((#1798));\n#1800=IFCPOLYLOOP((#1792,#572,#519));\n#1801=IFCFACEOUTERBOUND(#1800,.T.);\n#1802=IFCFACE((#1801));\n#1803=IFCPOLYLOOP((#1792,#625,#572));\n#1804=IFCFACEOUTERBOUND(#1803,.T.);\n#1805=IFCFACE((#1804));\n#1806=IFCCARTESIANPOINT((8.87134505444308,-3.06627844699456,-2.81429928661014));\n#1807=IFCPOLYLOOP((#1806,#597,#599));\n#1808=IFCFACEOUTERBOUND(#1807,.T.);\n#1809=IFCFACE((#1808));\n#1810=IFCCARTESIANPOINT((9.16577067673206,-3.38632631432265,-3.3262418621406));\n#1811=IFCPOLYLOOP((#1810,#599,#1749));\n#1812=IFCFACEOUTERBOUND(#1811,.T.);\n#1813=IFCFACE((#1812));\n#1814=IFCPOLYLOOP((#1810,#1748,#1755));\n#1815=IFCFACEOUTERBOUND(#1814,.T.);\n#1816=IFCFACE((#1815));\n#1817=IFCPOLYLOOP((#1810,#1749,#1748));\n#1818=IFCFACEOUTERBOUND(#1817,.T.);\n#1819=IFCFACE((#1818));\n#1820=IFCPOLYLOOP((#1810,#1806,#599));\n#1821=IFCFACEOUTERBOUND(#1820,.T.);\n#1822=IFCFACE((#1821));\n#1823=IFCCARTESIANPOINT((7.35676710532362,-0.610429334878431,-0.939120625293558));\n#1824=IFCPOLYLOOP((#1823,#642,#1793));\n#1825=IFCFACEOUTERBOUND(#1824,.T.);\n#1826=IFCFACE((#1825));\n#1827=IFCPOLYLOOP((#1823,#1775,#1744));\n#1828=IFCFACEOUTERBOUND(#1827,.T.);\n#1829=IFCFACE((#1828));\n#1830=IFCPOLYLOOP((#1823,#1762,#642));\n#1831=IFCFACEOUTERBOUND(#1830,.T.);\n#1832=IFCFACE((#1831));\n#1833=IFCPOLYLOOP((#1823,#1744,#1762));\n#1834=IFCFACEOUTERBOUND(#1833,.T.);\n#1835=IFCFACE((#1834));\n#1836=IFCPOLYLOOP((#1823,#1793,#1792));\n#1837=IFCFACEOUTERBOUND(#1836,.T.);\n#1838=IFCFACE((#1837));\n#1839=IFCPOLYLOOP((#1823,#1792,#1775));\n#1840=IFCFACEOUTERBOUND(#1839,.T.);\n#1841=IFCFACE((#1840));\n#1842=IFCCARTESIANPOINT((9.20333535987884,-2.76748638440622,-3.18658536230913));\n#1843=IFCPOLYLOOP((#1842,#1810,#1755));\n#1844=IFCFACEOUTERBOUND(#1843,.T.);\n#1845=IFCFACE((#1844));\n#1846=IFCPOLYLOOP((#1842,#1806,#1810));\n#1847=IFCFACEOUTERBOUND(#1846,.T.);\n#1848=IFCFACE((#1847));\n#1849=IFCCARTESIANPOINT((9.12500296522367,-2.36172083536144,-2.94946452042389));\n#1850=IFCPOLYLOOP((#1849,#641,#597));\n#1851=IFCFACEOUTERBOUND(#1850,.T.);\n#1852=IFCFACE((#1851));\n#1853=IFCPOLYLOOP((#1849,#597,#1806));\n#1854=IFCFACEOUTERBOUND(#1853,.T.);\n#1855=IFCFACE((#1854));\n#1856=IFCPOLYLOOP((#1849,#1806,#1842));\n#1857=IFCFACEOUTERBOUND(#1856,.T.);\n#1858=IFCFACE((#1857));\n#1859=IFCPOLYLOOP((#1849,#1842,#1755));\n#1860=IFCFACEOUTERBOUND(#1859,.T.);\n#1861=IFCFACE((#1860));\n#1862=IFCCARTESIANPOINT((1.16032684422279,6.56529016434118,-0.807248738289168));\n#1863=IFCPOLYLOOP((#1862,#367,#603));\n#1864=IFCFACEOUTERBOUND(#1863,.T.);\n#1865=IFCFACE((#1864));\n#1866=IFCPOLYLOOP((#1862,#676,#367));\n#1867=IFCFACEOUTERBOUND(#1866,.T.);\n#1868=IFCFACE((#1867));\n#1869=IFCPOLYLOOP((#1862,#603,#611));\n#1870=IFCFACEOUTERBOUND(#1869,.T.);\n#1871=IFCFACE((#1870));\n#1872=IFCCARTESIANPOINT((6.55292271500682,1.64157872990017,-0.496101952381885));\n#1873=IFCPOLYLOOP((#1872,#590,#625));\n#1874=IFCFACEOUTERBOUND(#1873,.T.);\n#1875=IFCFACE((#1874));\n#1876=IFCPOLYLOOP((#1872,#625,#1793));\n#1877=IFCFACEOUTERBOUND(#1876,.T.);\n#1878=IFCFACE((#1877));\n#1879=IFCPOLYLOOP((#1872,#669,#590));\n#1880=IFCFACEOUTERBOUND(#1879,.T.);\n#1881=IFCFACE((#1880));\n#1882=IFCCARTESIANPOINT((-1.89762440256387,9.13984408716134,-2.96699802545375));\n#1883=IFCPOLYLOOP((#1882,#636,#637));\n#1884=IFCFACEOUTERBOUND(#1883,.T.);\n#1885=IFCFACE((#1884));\n#1886=IFCCARTESIANPOINT((-1.57071972682337,9.24664590877725,-3.0623926037298));\n#1887=IFCPOLYLOOP((#1886,#1882,#637));\n#1888=IFCFACEOUTERBOUND(#1887,.T.);\n#1889=IFCFACE((#1888));\n#1890=IFCCARTESIANPOINT((-1.8392646813747,9.5118935511272,-3.55001911581324));\n#1891=IFCCARTESIANPOINT((-1.17363885472244,10.,-4.58359337618914));\n#1892=IFCPOLYLOOP((#1890,#1891,#1728));\n#1893=IFCFACEOUTERBOUND(#1892,.T.);\n#1894=IFCFACE((#1893));\n#1895=IFCPOLYLOOP((#1890,#636,#1882));\n#1896=IFCFACEOUTERBOUND(#1895,.T.);\n#1897=IFCFACE((#1896));\n#1898=IFCPOLYLOOP((#1890,#1728,#1779));\n#1899=IFCFACEOUTERBOUND(#1898,.T.);\n#1900=IFCFACE((#1899));\n#1901=IFCPOLYLOOP((#1890,#1886,#1891));\n#1902=IFCFACEOUTERBOUND(#1901,.T.);\n#1903=IFCFACE((#1902));\n#1904=IFCPOLYLOOP((#1890,#1882,#1886));\n#1905=IFCFACEOUTERBOUND(#1904,.T.);\n#1906=IFCFACE((#1905));\n#1907=IFCPOLYLOOP((#1890,#1779,#636));\n#1908=IFCFACEOUTERBOUND(#1907,.T.);\n#1909=IFCFACE((#1908));\n#1910=IFCCARTESIANPOINT((6.81843924434097,2.04381399770578,-0.606890523602934));\n#1911=IFCPOLYLOOP((#1910,#669,#1872));\n#1912=IFCFACEOUTERBOUND(#1911,.T.);\n#1913=IFCFACE((#1912));\n#1914=IFCCARTESIANPOINT((-1.08645224889949,9.16695996554692,-2.87186524136129));\n#1915=IFCPOLYLOOP((#1914,#646,#687));\n#1916=IFCFACEOUTERBOUND(#1915,.T.);\n#1917=IFCFACE((#1916));\n#1918=IFCPOLYLOOP((#1914,#1886,#637));\n#1919=IFCFACEOUTERBOUND(#1918,.T.);\n#1920=IFCFACE((#1919));\n#1921=IFCPOLYLOOP((#1914,#637,#646));\n#1922=IFCFACEOUTERBOUND(#1921,.T.);\n#1923=IFCFACE((#1922));\n#1924=IFCCARTESIANPOINT((9.18494562685119,-0.720718597481389,-2.71754837459515));\n#1925=IFCPOLYLOOP((#1924,#642,#641));\n#1926=IFCFACEOUTERBOUND(#1925,.T.);\n#1927=IFCFACE((#1926));\n#1928=IFCCARTESIANPOINT((7.35676710532363,1.97044810396899,-0.851097678127514));\n#1929=IFCPOLYLOOP((#1928,#1872,#1793));\n#1930=IFCFACEOUTERBOUND(#1929,.T.);\n#1931=IFCFACE((#1930));\n#1932=IFCPOLYLOOP((#1928,#1793,#696));\n#1933=IFCFACEOUTERBOUND(#1932,.T.);\n#1934=IFCFACE((#1933));\n#1935=IFCPOLYLOOP((#1928,#1910,#1872));\n#1936=IFCFACEOUTERBOUND(#1935,.T.);\n#1937=IFCFACE((#1936));\n#1938=IFCCARTESIANPOINT((9.69895115528596,-1.54057532875274,-3.90632344831512));\n#1939=IFCCARTESIANPOINT((10.,0.,-4.755));\n#1940=IFCPOLYLOOP((#1938,#1755,#1939));\n#1941=IFCFACEOUTERBOUND(#1940,.T.);\n#1942=IFCFACE((#1941));\n#1943=IFCPOLYLOOP((#1938,#1924,#641));\n#1944=IFCFACEOUTERBOUND(#1943,.T.);\n#1945=IFCFACE((#1944));\n#1946=IFCPOLYLOOP((#1938,#641,#1849));\n#1947=IFCFACEOUTERBOUND(#1946,.T.);\n#1948=IFCFACE((#1947));\n#1949=IFCPOLYLOOP((#1938,#1849,#1755));\n#1950=IFCFACEOUTERBOUND(#1949,.T.);\n#1951=IFCFACE((#1950));\n#1952=IFCCARTESIANPOINT((7.19327211882919,2.63451388432877,-0.830178758630065));\n#1953=IFCPOLYLOOP((#1952,#669,#1910));\n#1954=IFCFACEOUTERBOUND(#1953,.T.);\n#1955=IFCFACE((#1954));\n#1956=IFCPOLYLOOP((#1952,#1910,#1928));\n#1957=IFCFACEOUTERBOUND(#1956,.T.);\n#1958=IFCFACE((#1957));\n#1959=IFCCARTESIANPOINT((9.54529562906159,-0.732004139804299,-3.43150720399627));\n#1960=IFCCARTESIANPOINT((9.2851796688512,-0.314799235470127,-2.85392415063223));\n#1961=IFCPOLYLOOP((#1959,#1939,#1960));\n#1962=IFCFACEOUTERBOUND(#1961,.T.);\n#1963=IFCFACE((#1962));\n#1964=IFCPOLYLOOP((#1959,#1924,#1938));\n#1965=IFCFACEOUTERBOUND(#1964,.T.);\n#1966=IFCFACE((#1965));\n#1967=IFCPOLYLOOP((#1959,#1960,#1924));\n#1968=IFCFACEOUTERBOUND(#1967,.T.);\n#1969=IFCFACE((#1968));\n#1970=IFCPOLYLOOP((#1959,#1938,#1939));\n#1971=IFCFACEOUTERBOUND(#1970,.T.);\n#1972=IFCFACE((#1971));\n#1973=IFCCARTESIANPOINT((-0.674855497052797,9.53689534665799,-3.44581648227862));\n#1974=IFCCARTESIANPOINT((0.942227298706008,10.,-4.62955073267135));\n#1975=IFCPOLYLOOP((#1973,#1974,#1891));\n#1976=IFCFACEOUTERBOUND(#1975,.T.);\n#1977=IFCFACE((#1976));\n#1978=IFCPOLYLOOP((#1973,#1891,#1886));\n#1979=IFCFACEOUTERBOUND(#1978,.T.);\n#1980=IFCFACE((#1979));\n#1981=IFCPOLYLOOP((#1973,#1914,#687));\n#1982=IFCFACEOUTERBOUND(#1981,.T.);\n#1983=IFCFACE((#1982));\n#1984=IFCPOLYLOOP((#1973,#1886,#1914));\n#1985=IFCFACEOUTERBOUND(#1984,.T.);\n#1986=IFCFACE((#1985));\n#1987=IFCCARTESIANPOINT((9.00205439814815,0.379254918981481,-2.34029947916667));\n#1988=IFCPOLYLOOP((#1987,#642,#1924));\n#1989=IFCFACEOUTERBOUND(#1988,.T.);\n#1990=IFCFACE((#1989));\n#1991=IFCPOLYLOOP((#1987,#1924,#1960));\n#1992=IFCFACEOUTERBOUND(#1991,.T.);\n#1993=IFCFACE((#1992));\n#1994=IFCCARTESIANPOINT((3.58754972281922,6.38447640621999,-0.820738947557059));\n#1995=IFCPOLYLOOP((#1994,#611,#650));\n#1996=IFCFACEOUTERBOUND(#1995,.T.);\n#1997=IFCFACE((#1996));\n#1998=IFCCARTESIANPOINT((7.56894078350354,2.8142500570908,-1.07599750174521));\n#1999=IFCPOLYLOOP((#1998,#1952,#1928));\n#2000=IFCFACEOUTERBOUND(#1999,.T.);\n#2001=IFCFACE((#2000));\n#2002=IFCPOLYLOOP((#1998,#696,#695));\n#2003=IFCFACEOUTERBOUND(#2002,.T.);\n#2004=IFCFACE((#2003));\n#2005=IFCPOLYLOOP((#1998,#1928,#696));\n#2006=IFCFACEOUTERBOUND(#2005,.T.);\n#2007=IFCFACE((#2006));\n#2008=IFCCARTESIANPOINT((7.14519211050001,3.32338718528929,-0.914413679878129));\n#2009=IFCPOLYLOOP((#2008,#669,#1952));\n#2010=IFCFACEOUTERBOUND(#2009,.T.);\n#2011=IFCFACE((#2010));\n#2012=IFCPOLYLOOP((#2008,#1952,#1998));\n#2013=IFCFACEOUTERBOUND(#2012,.T.);\n#2014=IFCFACE((#2013));\n#2015=IFCCARTESIANPOINT((3.93865580521004,6.49720944264734,-0.915574459721597));\n#2016=IFCPOLYLOOP((#2015,#1994,#650));\n#2017=IFCFACEOUTERBOUND(#2016,.T.);\n#2018=IFCFACE((#2017));\n#2019=IFCCARTESIANPOINT((6.34857614929695,4.53202323085393,-0.835876048955446));\n#2020=IFCPOLYLOOP((#2019,#669,#2008));\n#2021=IFCFACEOUTERBOUND(#2020,.T.);\n#2022=IFCFACE((#2021));\n#2023=IFCPOLYLOOP((#2019,#662,#661));\n#2024=IFCFACEOUTERBOUND(#2023,.T.);\n#2025=IFCFACE((#2024));\n#2026=IFCPOLYLOOP((#2019,#661,#669));\n#2027=IFCFACEOUTERBOUND(#2026,.T.);\n#2028=IFCFACE((#2027));\n#2029=IFCCARTESIANPOINT((7.2531325872205,3.59003499218134,-1.03136368530428));\n#2030=IFCPOLYLOOP((#2029,#1998,#695));\n#2031=IFCFACEOUTERBOUND(#2030,.T.);\n#2032=IFCFACE((#2031));\n#2033=IFCPOLYLOOP((#2029,#695,#707));\n#2034=IFCFACEOUTERBOUND(#2033,.T.);\n#2035=IFCFACE((#2034));\n#2036=IFCPOLYLOOP((#2029,#707,#2019));\n#2037=IFCFACEOUTERBOUND(#2036,.T.);\n#2038=IFCFACE((#2037));\n#2039=IFCPOLYLOOP((#2029,#2019,#2008));\n#2040=IFCFACEOUTERBOUND(#2039,.T.);\n#2041=IFCFACE((#2040));\n#2042=IFCPOLYLOOP((#2029,#2008,#1998));\n#2043=IFCFACEOUTERBOUND(#2042,.T.);\n#2044=IFCFACE((#2043));\n#2045=IFCCARTESIANPOINT((4.4941041026953,6.32937643249591,-0.941901707344667));\n#2046=IFCPOLYLOOP((#2045,#2015,#650));\n#2047=IFCFACEOUTERBOUND(#2046,.T.);\n#2048=IFCFACE((#2047));\n#2049=IFCPOLYLOOP((#2045,#657,#662));\n#2050=IFCFACEOUTERBOUND(#2049,.T.);\n#2051=IFCFACE((#2050));\n#2052=IFCPOLYLOOP((#2045,#650,#657));\n#2053=IFCFACEOUTERBOUND(#2052,.T.);\n#2054=IFCFACE((#2053));\n#2055=IFCPOLYLOOP((#2045,#712,#2015));\n#2056=IFCFACEOUTERBOUND(#2055,.T.);\n#2057=IFCFACE((#2056));\n#2058=IFCCARTESIANPOINT((9.2851796688512,0.906793070875574,-2.82948960089823));\n#2059=IFCPOLYLOOP((#2058,#1960,#1939));\n#2060=IFCFACEOUTERBOUND(#2059,.T.);\n#2061=IFCFACE((#2060));\n#2062=IFCPOLYLOOP((#2058,#1987,#1960));\n#2063=IFCFACEOUTERBOUND(#2062,.T.);\n#2064=IFCFACE((#2063));\n#2065=IFCCARTESIANPOINT((0.467719592102535,9.55025838901758,-3.43809523552361));\n#2066=IFCPOLYLOOP((#2065,#1973,#687));\n#2067=IFCFACEOUTERBOUND(#2066,.T.);\n#2068=IFCFACE((#2067));\n#2069=IFCPOLYLOOP((#2065,#1974,#1973));\n#2070=IFCFACEOUTERBOUND(#2069,.T.);\n#2071=IFCFACE((#2070));\n#2072=IFCCARTESIANPOINT((9.18635252450546,1.29883834294331,-2.67009231059277));\n#2073=IFCPOLYLOOP((#2072,#696,#1987));\n#2074=IFCFACEOUTERBOUND(#2073,.T.);\n#2075=IFCFACE((#2074));\n#2076=IFCPOLYLOOP((#2072,#1987,#2058));\n#2077=IFCFACEOUTERBOUND(#2076,.T.);\n#2078=IFCFACE((#2077));\n#2079=IFCPOLYLOOP((#2072,#697,#696));\n#2080=IFCFACEOUTERBOUND(#2079,.T.);\n#2081=IFCFACE((#2080));\n#2082=IFCCARTESIANPOINT((1.06142292093934,9.38151501695194,-3.14074259586667));\n#2083=IFCPOLYLOOP((#2082,#687,#680));\n#2084=IFCFACEOUTERBOUND(#2083,.T.);\n#2085=IFCFACE((#2084));\n#2086=IFCPOLYLOOP((#2082,#1974,#2065));\n#2087=IFCFACEOUTERBOUND(#2086,.T.);\n#2088=IFCFACE((#2087));\n#2089=IFCPOLYLOOP((#2082,#680,#691));\n#2090=IFCFACEOUTERBOUND(#2089,.T.);\n#2091=IFCFACE((#2090));\n#2092=IFCPOLYLOOP((#2082,#2065,#687));\n#2093=IFCFACEOUTERBOUND(#2092,.T.);\n#2094=IFCFACE((#2093));\n#2095=IFCPOLYLOOP((#2082,#691,#701));\n#2096=IFCFACEOUTERBOUND(#2095,.T.);\n#2097=IFCFACE((#2096));\n#2098=IFCCARTESIANPOINT((9.54653439793015,1.03807751393521,-3.40063402655209));\n#2099=IFCPOLYLOOP((#2098,#2058,#1939));\n#2100=IFCFACEOUTERBOUND(#2099,.T.);\n#2101=IFCFACE((#2100));\n#2102=IFCPOLYLOOP((#2098,#2072,#2058));\n#2103=IFCFACEOUTERBOUND(#2102,.T.);\n#2104=IFCFACE((#2103));\n#2105=IFCCARTESIANPOINT((7.0604775039919,5.23702883961869,-1.45872244373669));\n#2106=IFCPOLYLOOP((#2105,#662,#2019));\n#2107=IFCFACEOUTERBOUND(#2106,.T.);\n#2108=IFCFACE((#2107));\n#2109=IFCPOLYLOOP((#2105,#2019,#707));\n#2110=IFCFACEOUTERBOUND(#2109,.T.);\n#2111=IFCFACE((#2110));\n#2112=IFCPOLYLOOP((#2105,#707,#706));\n#2113=IFCFACEOUTERBOUND(#2112,.T.);\n#2114=IFCFACE((#2113));\n#2115=IFCPOLYLOOP((#2105,#706,#717));\n#2116=IFCFACEOUTERBOUND(#2115,.T.);\n#2117=IFCFACE((#2116));\n#2118=IFCCARTESIANPOINT((1.59984896525763,9.55131729830246,-3.52489605462407));\n#2119=IFCCARTESIANPOINT((3.06834680727272,10.,-5.014553557514));\n#2120=IFCPOLYLOOP((#2118,#2119,#1974));\n#2121=IFCFACEOUTERBOUND(#2120,.T.);\n#2122=IFCFACE((#2121));\n#2123=IFCPOLYLOOP((#2118,#2082,#701));\n#2124=IFCFACEOUTERBOUND(#2123,.T.);\n#2125=IFCFACE((#2124));\n#2126=IFCPOLYLOOP((#2118,#1974,#2082));\n#2127=IFCFACEOUTERBOUND(#2126,.T.);\n#2128=IFCFACE((#2127));\n#2129=IFCCARTESIANPOINT((9.70300151540916,1.54472505490958,-3.85560773732705));\n#2130=IFCCARTESIANPOINT((10.,2.51644585791691,-4.9369676273256));\n#2131=IFCPOLYLOOP((#2129,#1939,#2130));\n#2132=IFCFACEOUTERBOUND(#2131,.T.);\n#2133=IFCFACE((#2132));\n#2134=IFCPOLYLOOP((#2129,#2098,#1939));\n#2135=IFCFACEOUTERBOUND(#2134,.T.);\n#2136=IFCFACE((#2135));\n#2137=IFCPOLYLOOP((#2129,#697,#2072));\n#2138=IFCFACEOUTERBOUND(#2137,.T.);\n#2139=IFCFACE((#2138));\n#2140=IFCPOLYLOOP((#2129,#2072,#2098));\n#2141=IFCFACEOUTERBOUND(#2140,.T.);\n#2142=IFCFACE((#2141));\n#2143=IFCCARTESIANPOINT((9.1440763650256,2.61678006802339,-2.79722371674537));\n#2144=IFCPOLYLOOP((#2143,#695,#697));\n#2145=IFCFACEOUTERBOUND(#2144,.T.);\n#2146=IFCFACE((#2145));\n#2147=IFCCARTESIANPOINT((5.11177144488803,7.29182488158978,-1.68299793554793));\n#2148=IFCPOLYLOOP((#2147,#2045,#662));\n#2149=IFCFACEOUTERBOUND(#2148,.T.);\n#2150=IFCFACE((#2149));\n#2151=IFCPOLYLOOP((#2147,#712,#2045));\n#2152=IFCFACEOUTERBOUND(#2151,.T.);\n#2153=IFCFACE((#2152));\n#2154=IFCPOLYLOOP((#2147,#722,#711));\n#2155=IFCFACEOUTERBOUND(#2154,.T.);\n#2156=IFCFACE((#2155));\n#2157=IFCPOLYLOOP((#2147,#711,#712));\n#2158=IFCFACEOUTERBOUND(#2157,.T.);\n#2159=IFCFACE((#2158));\n#2160=IFCCARTESIANPOINT((6.84389658900384,5.78747693084088,-1.58869471813653));\n#2161=IFCPOLYLOOP((#2160,#662,#2105));\n#2162=IFCFACEOUTERBOUND(#2161,.T.);\n#2163=IFCFACE((#2162));\n#2164=IFCCARTESIANPOINT((9.5208800107573,2.33424035647539,-3.51288214079829));\n#2165=IFCPOLYLOOP((#2164,#2129,#2130));\n#2166=IFCFACEOUTERBOUND(#2165,.T.);\n#2167=IFCFACE((#2166));\n#2168=IFCPOLYLOOP((#2164,#697,#2129));\n#2169=IFCFACEOUTERBOUND(#2168,.T.);\n#2170=IFCFACE((#2169));\n#2171=IFCPOLYLOOP((#2164,#2143,#697));\n#2172=IFCFACEOUTERBOUND(#2171,.T.);\n#2173=IFCFACE((#2172));\n#2174=IFCCARTESIANPOINT((9.22787834436207,2.87604174583961,-3.01608918283702));\n#2175=IFCPOLYLOOP((#2174,#695,#2143));\n#2176=IFCFACEOUTERBOUND(#2175,.T.);\n#2177=IFCFACE((#2176));\n#2178=IFCPOLYLOOP((#2174,#2164,#2130));\n#2179=IFCFACEOUTERBOUND(#2178,.T.);\n#2180=IFCFACE((#2179));\n#2181=IFCPOLYLOOP((#2174,#2143,#2164));\n#2182=IFCFACEOUTERBOUND(#2181,.T.);\n#2183=IFCFACE((#2182));\n#2184=IFCCARTESIANPOINT((5.69613925630973,7.10486850393657,-1.79154290719563));\n#2185=IFCPOLYLOOP((#2184,#722,#2147));\n#2186=IFCFACEOUTERBOUND(#2185,.T.);\n#2187=IFCFACE((#2186));\n#2188=IFCPOLYLOOP((#2184,#2147,#662));\n#2189=IFCFACEOUTERBOUND(#2188,.T.);\n#2190=IFCFACE((#2189));\n#2191=IFCCARTESIANPOINT((6.75100016220125,6.30926490438307,-1.8320023630533));\n#2192=IFCPOLYLOOP((#2191,#2184,#662));\n#2193=IFCFACEOUTERBOUND(#2192,.T.);\n#2194=IFCFACE((#2193));\n#2195=IFCPOLYLOOP((#2191,#662,#2160));\n#2196=IFCFACEOUTERBOUND(#2195,.T.);\n#2197=IFCFACE((#2196));\n#2198=IFCCARTESIANPOINT((9.10054517976487,3.43691112371838,-2.94772611466433));\n#2199=IFCPOLYLOOP((#2198,#695,#2174));\n#2200=IFCFACEOUTERBOUND(#2199,.T.);\n#2201=IFCFACE((#2200));\n#2202=IFCPOLYLOOP((#2198,#706,#695));\n#2203=IFCFACEOUTERBOUND(#2202,.T.);\n#2204=IFCFACE((#2203));\n#2205=IFCPOLYLOOP((#2198,#716,#706));\n#2206=IFCFACEOUTERBOUND(#2205,.T.);\n#2207=IFCFACE((#2206));\n#2208=IFCCARTESIANPOINT((3.26396308200168,9.16793538805331,-3.14522832834111));\n#2209=IFCPOLYLOOP((#2208,#701,#702));\n#2210=IFCFACEOUTERBOUND(#2209,.T.);\n#2211=IFCFACE((#2210));\n#2212=IFCPOLYLOOP((#2208,#702,#711));\n#2213=IFCFACEOUTERBOUND(#2212,.T.);\n#2214=IFCFACE((#2213));\n#2215=IFCCARTESIANPOINT((7.18488651309743,6.06230613221037,-1.9846063979699));\n#2216=IFCPOLYLOOP((#2215,#2105,#717));\n#2217=IFCFACEOUTERBOUND(#2216,.T.);\n#2218=IFCFACE((#2217));\n#2219=IFCPOLYLOOP((#2215,#2160,#2105));\n#2220=IFCFACEOUTERBOUND(#2219,.T.);\n#2221=IFCFACE((#2220));\n#2222=IFCPOLYLOOP((#2215,#2191,#2160));\n#2223=IFCFACEOUTERBOUND(#2222,.T.);\n#2224=IFCFACE((#2223));\n#2225=IFCPOLYLOOP((#2215,#717,#727));\n#2226=IFCFACEOUTERBOUND(#2225,.T.);\n#2227=IFCFACE((#2226));\n#2228=IFCCARTESIANPOINT((2.73162812871312,9.53888472910354,-3.70926673667609));\n#2229=IFCPOLYLOOP((#2228,#701,#2208));\n#2230=IFCFACEOUTERBOUND(#2229,.T.);\n#2231=IFCFACE((#2230));\n#2232=IFCPOLYLOOP((#2228,#2118,#701));\n#2233=IFCFACEOUTERBOUND(#2232,.T.);\n#2234=IFCFACE((#2233));\n#2235=IFCPOLYLOOP((#2228,#2119,#2118));\n#2236=IFCFACEOUTERBOUND(#2235,.T.);\n#2237=IFCFACE((#2236));\n#2238=IFCCARTESIANPOINT((6.43234384971733,6.95488278003622,-2.07968997000246));\n#2239=IFCPOLYLOOP((#2238,#722,#2184));\n#2240=IFCFACEOUTERBOUND(#2239,.T.);\n#2241=IFCFACE((#2240));\n#2242=IFCPOLYLOOP((#2238,#2184,#2191));\n#2243=IFCFACEOUTERBOUND(#2242,.T.);\n#2244=IFCFACE((#2243));\n#2245=IFCPOLYLOOP((#2238,#2191,#2215));\n#2246=IFCFACEOUTERBOUND(#2245,.T.);\n#2247=IFCFACE((#2246));\n#2248=IFCCARTESIANPOINT((3.7248233696665,9.24521857655041,-3.42389380278945));\n#2249=IFCPOLYLOOP((#2248,#2208,#711));\n#2250=IFCFACEOUTERBOUND(#2249,.T.);\n#2251=IFCFACE((#2250));\n#2252=IFCPOLYLOOP((#2248,#2228,#2208));\n#2253=IFCFACEOUTERBOUND(#2252,.T.);\n#2254=IFCFACE((#2253));\n#2255=IFCPOLYLOOP((#2248,#2119,#2228));\n#2256=IFCFACEOUTERBOUND(#2255,.T.);\n#2257=IFCFACE((#2256));\n#2258=IFCCARTESIANPOINT((9.47667617468398,3.54910125647961,-3.73588847927795));\n#2259=IFCPOLYLOOP((#2258,#716,#2198));\n#2260=IFCFACEOUTERBOUND(#2259,.T.);\n#2261=IFCFACE((#2260));\n#2262=IFCPOLYLOOP((#2258,#2174,#2130));\n#2263=IFCFACEOUTERBOUND(#2262,.T.);\n#2264=IFCFACE((#2263));\n#2265=IFCPOLYLOOP((#2258,#2198,#2174));\n#2266=IFCFACEOUTERBOUND(#2265,.T.);\n#2267=IFCFACE((#2266));\n#2268=IFCCARTESIANPOINT((4.09279992765225,9.13458953992449,-3.36431103346481));\n#2269=IFCPOLYLOOP((#2268,#711,#721));\n#2270=IFCFACEOUTERBOUND(#2269,.T.);\n#2271=IFCFACE((#2270));\n#2272=IFCPOLYLOOP((#2268,#2248,#711));\n#2273=IFCFACEOUTERBOUND(#2272,.T.);\n#2274=IFCFACE((#2273));\n#2275=IFCCARTESIANPOINT((6.82460399869234,7.41772872964233,-2.7495626198666));\n#2276=IFCPOLYLOOP((#2275,#722,#2238));\n#2277=IFCFACEOUTERBOUND(#2276,.T.);\n#2278=IFCFACE((#2277));\n#2279=IFCPOLYLOOP((#2275,#2215,#727));\n#2280=IFCFACEOUTERBOUND(#2279,.T.);\n#2281=IFCFACE((#2280));\n#2282=IFCCARTESIANPOINT((7.20929052697411,7.86582258901123,-3.52651909547784));\n#2283=IFCPOLYLOOP((#2275,#2282,#722));\n#2284=IFCFACEOUTERBOUND(#2283,.T.);\n#2285=IFCFACE((#2284));\n#2286=IFCPOLYLOOP((#2275,#2238,#2215));\n#2287=IFCFACEOUTERBOUND(#2286,.T.);\n#2288=IFCFACE((#2287));\n#2289=IFCCARTESIANPOINT((3.93881533572506,9.50925512166422,-4.018660184446));\n#2290=IFCPOLYLOOP((#2289,#2268,#721));\n#2291=IFCFACEOUTERBOUND(#2290,.T.);\n#2292=IFCFACE((#2291));\n#2293=IFCPOLYLOOP((#2289,#2248,#2268));\n#2294=IFCFACEOUTERBOUND(#2293,.T.);\n#2295=IFCFACE((#2294));\n#2296=IFCCARTESIANPOINT((5.68782579151555,10.,-5.90760377123967));\n#2297=IFCPOLYLOOP((#2289,#2296,#2119));\n#2298=IFCFACEOUTERBOUND(#2297,.T.);\n#2299=IFCFACE((#2298));\n#2300=IFCPOLYLOOP((#2289,#2119,#2248));\n#2301=IFCFACEOUTERBOUND(#2300,.T.);\n#2302=IFCFACE((#2301));\n#2303=IFCCARTESIANPOINT((9.64373267766084,3.95792994230555,-4.26861390205416));\n#2304=IFCCARTESIANPOINT((10.,4.75823867008481,-5.45327128399318));\n#2305=IFCPOLYLOOP((#2303,#2130,#2304));\n#2306=IFCFACEOUTERBOUND(#2305,.T.);\n#2307=IFCFACE((#2306));\n#2308=IFCPOLYLOOP((#2303,#2258,#2130));\n#2309=IFCFACEOUTERBOUND(#2308,.T.);\n#2310=IFCFACE((#2309));\n#2311=IFCPOLYLOOP((#2303,#716,#2258));\n#2312=IFCFACEOUTERBOUND(#2311,.T.);\n#2313=IFCFACE((#2312));\n#2314=IFCCARTESIANPOINT((7.96621617916866,6.97823118830386,-3.40754307857302));\n#2315=IFCPOLYLOOP((#2314,#2275,#727));\n#2316=IFCFACEOUTERBOUND(#2315,.T.);\n#2317=IFCFACE((#2316));\n#2318=IFCPOLYLOOP((#2314,#2282,#2275));\n#2319=IFCFACEOUTERBOUND(#2318,.T.);\n#2320=IFCFACE((#2319));\n#2321=IFCPOLYLOOP((#2314,#727,#726));\n#2322=IFCFACEOUTERBOUND(#2321,.T.);\n#2323=IFCFACE((#2322));\n#2324=IFCCARTESIANPOINT((9.37120114827863,5.41822907982549,-4.33444938064896));\n#2325=IFCPOLYLOOP((#2324,#2303,#2304));\n#2326=IFCFACEOUTERBOUND(#2325,.T.);\n#2327=IFCFACE((#2326));\n#2328=IFCPOLYLOOP((#2324,#716,#2303));\n#2329=IFCFACEOUTERBOUND(#2328,.T.);\n#2330=IFCFACE((#2329));\n#2331=IFCCARTESIANPOINT((10.,6.93951228852737,-6.31474303761694));\n#2332=IFCPOLYLOOP((#2324,#2304,#2331));\n#2333=IFCFACEOUTERBOUND(#2332,.T.);\n#2334=IFCFACE((#2333));\n#2335=IFCPOLYLOOP((#2324,#726,#716));\n#2336=IFCFACEOUTERBOUND(#2335,.T.);\n#2337=IFCFACE((#2336));\n#2338=IFCCARTESIANPOINT((6.24133212894416,8.74255298950008,-3.87685756457508));\n#2339=IFCPOLYLOOP((#2338,#722,#2282));\n#2340=IFCFACEOUTERBOUND(#2339,.T.);\n#2341=IFCFACE((#2340));\n#2342=IFCCARTESIANPOINT((5.73958451968616,9.13094689195209,-4.16611946602962));\n#2343=IFCPOLYLOOP((#2342,#2296,#2289));\n#2344=IFCFACEOUTERBOUND(#2343,.T.);\n#2345=IFCFACE((#2344));\n#2346=IFCPOLYLOOP((#2342,#722,#2338));\n#2347=IFCFACEOUTERBOUND(#2346,.T.);\n#2348=IFCFACE((#2347));\n#2349=IFCPOLYLOOP((#2342,#721,#722));\n#2350=IFCFACEOUTERBOUND(#2349,.T.);\n#2351=IFCFACE((#2350));\n#2352=IFCPOLYLOOP((#2342,#2289,#721));\n#2353=IFCFACEOUTERBOUND(#2352,.T.);\n#2354=IFCFACE((#2353));\n#2355=IFCCARTESIANPOINT((7.98354004810011,7.57826546309029,-3.9860645949887));\n#2356=IFCPOLYLOOP((#2355,#2282,#2314));\n#2357=IFCFACEOUTERBOUND(#2356,.T.);\n#2358=IFCFACE((#2357));\n#2359=IFCCARTESIANPOINT((6.48797897208109,9.02650191192725,-4.48010629309703));\n#2360=IFCPOLYLOOP((#2359,#2342,#2338));\n#2361=IFCFACEOUTERBOUND(#2360,.T.);\n#2362=IFCFACE((#2361));\n#2363=IFCPOLYLOOP((#2359,#2296,#2342));\n#2364=IFCFACEOUTERBOUND(#2363,.T.);\n#2365=IFCFACE((#2364));\n#2366=IFCCARTESIANPOINT((8.36223563578413,7.42031665300137,-4.26795999460325));\n#2367=IFCPOLYLOOP((#2366,#2355,#2314));\n#2368=IFCFACEOUTERBOUND(#2367,.T.);\n#2369=IFCFACE((#2368));\n#2370=IFCPOLYLOOP((#2366,#736,#2355));\n#2371=IFCFACEOUTERBOUND(#2370,.T.);\n#2372=IFCFACE((#2371));\n#2373=IFCPOLYLOOP((#2366,#2314,#726));\n#2374=IFCFACEOUTERBOUND(#2373,.T.);\n#2375=IFCFACE((#2374));\n#2376=IFCCARTESIANPOINT((6.99294353710344,8.94706009608038,-4.73021934194737));\n#2377=IFCPOLYLOOP((#2376,#2338,#2282));\n#2378=IFCFACEOUTERBOUND(#2377,.T.);\n#2379=IFCFACE((#2378));\n#2380=IFCPOLYLOOP((#2376,#2282,#2355));\n#2381=IFCFACEOUTERBOUND(#2380,.T.);\n#2382=IFCFACE((#2381));\n#2383=IFCPOLYLOOP((#2376,#2355,#736));\n#2384=IFCFACEOUTERBOUND(#2383,.T.);\n#2385=IFCFACE((#2384));\n#2386=IFCPOLYLOOP((#2376,#2359,#2338));\n#2387=IFCFACEOUTERBOUND(#2386,.T.);\n#2388=IFCFACE((#2387));\n#2389=IFCCARTESIANPOINT((8.91559596954348,7.2793274357246,-4.85832166148246));\n#2390=IFCPOLYLOOP((#2389,#2324,#2331));\n#2391=IFCFACEOUTERBOUND(#2390,.T.);\n#2392=IFCFACE((#2391));\n#2393=IFCPOLYLOOP((#2389,#726,#2324));\n#2394=IFCFACEOUTERBOUND(#2393,.T.);\n#2395=IFCFACE((#2394));\n#2396=IFCPOLYLOOP((#2389,#2366,#726));\n#2397=IFCFACEOUTERBOUND(#2396,.T.);\n#2398=IFCFACE((#2397));\n#2399=IFCCARTESIANPOINT((6.86959694444614,9.45761666142187,-5.4537893522932));\n#2400=IFCPOLYLOOP((#2399,#2296,#2359));\n#2401=IFCFACEOUTERBOUND(#2400,.T.);\n#2402=IFCFACE((#2401));\n#2403=IFCPOLYLOOP((#2399,#2359,#2376));\n#2404=IFCFACEOUTERBOUND(#2403,.T.);\n#2405=IFCFACE((#2404));\n#2406=IFCCARTESIANPOINT((7.21774256195473,9.37235232209269,-5.55085468416389));\n#2407=IFCPOLYLOOP((#2406,#2376,#736));\n#2408=IFCFACEOUTERBOUND(#2407,.T.);\n#2409=IFCFACE((#2408));\n#2410=IFCPOLYLOOP((#2406,#2399,#2376));\n#2411=IFCFACEOUTERBOUND(#2410,.T.);\n#2412=IFCFACE((#2411));\n#2413=IFCPOLYLOOP((#2406,#735,#748));\n#2414=IFCFACEOUTERBOUND(#2413,.T.);\n#2415=IFCFACE((#2414));\n#2416=IFCPOLYLOOP((#2406,#736,#735));\n#2417=IFCFACEOUTERBOUND(#2416,.T.);\n#2418=IFCFACE((#2417));\n#2419=IFCCARTESIANPOINT((6.80346359480084,9.64241128142266,-5.7356589883809));\n#2420=IFCPOLYLOOP((#2419,#748,#2296));\n#2421=IFCFACEOUTERBOUND(#2420,.T.);\n#2422=IFCFACE((#2421));\n#2423=IFCPOLYLOOP((#2419,#2399,#2406));\n#2424=IFCFACEOUTERBOUND(#2423,.T.);\n#2425=IFCFACE((#2424));\n#2426=IFCPOLYLOOP((#2419,#2406,#748));\n#2427=IFCFACEOUTERBOUND(#2426,.T.);\n#2428=IFCFACE((#2427));\n#2429=IFCPOLYLOOP((#2419,#2296,#2399));\n#2430=IFCFACEOUTERBOUND(#2429,.T.);\n#2431=IFCFACE((#2430));\n#2432=IFCCARTESIANPOINT((9.07548286509334,7.91798558880134,-5.60980448809762));\n#2433=IFCPOLYLOOP((#2432,#736,#2366));\n#2434=IFCFACEOUTERBOUND(#2433,.T.);\n#2435=IFCFACE((#2434));\n#2436=IFCPOLYLOOP((#2432,#2366,#2389));\n#2437=IFCFACEOUTERBOUND(#2436,.T.);\n#2438=IFCFACE((#2437));\n#2439=IFCCARTESIANPOINT((9.04961499158356,8.14997560768381,-5.78073882680869));\n#2440=IFCPOLYLOOP((#2439,#744,#734));\n#2441=IFCFACEOUTERBOUND(#2440,.T.);\n#2442=IFCFACE((#2441));\n#2443=IFCPOLYLOOP((#2439,#736,#2432));\n#2444=IFCFACEOUTERBOUND(#2443,.T.);\n#2445=IFCFACE((#2444));\n#2446=IFCPOLYLOOP((#2439,#734,#736));\n#2447=IFCFACEOUTERBOUND(#2446,.T.);\n#2448=IFCFACE((#2447));\n#2449=IFCCARTESIANPOINT((9.53755428452182,7.8041890627945,-6.12282120890463));\n#2450=IFCPOLYLOOP((#2449,#753,#744));\n#2451=IFCFACEOUTERBOUND(#2450,.T.);\n#2452=IFCFACE((#2451));\n#2453=IFCPOLYLOOP((#2449,#2331,#753));\n#2454=IFCFACEOUTERBOUND(#2453,.T.);\n#2455=IFCFACE((#2454));\n#2456=IFCPOLYLOOP((#2449,#744,#2439));\n#2457=IFCFACEOUTERBOUND(#2456,.T.);\n#2458=IFCFACE((#2457));\n#2459=IFCPOLYLOOP((#2449,#2389,#2331));\n#2460=IFCFACEOUTERBOUND(#2459,.T.);\n#2461=IFCFACE((#2460));\n#2462=IFCPOLYLOOP((#2449,#2432,#2389));\n#2463=IFCFACEOUTERBOUND(#2462,.T.);\n#2464=IFCFACE((#2463));\n#2465=IFCPOLYLOOP((#2449,#2439,#2432));\n#2466=IFCFACEOUTERBOUND(#2465,.T.);\n#2467=IFCFACE((#2466));\n#2468=IFCCARTESIANPOINT((-8.58877892522778,-9.51996241277546,-4.1832991149625));\n#2469=IFCPOLYLOOP((#2468,#776,#772));\n#2470=IFCFACEOUTERBOUND(#2469,.T.);\n#2471=IFCFACE((#2470));\n#2472=IFCCARTESIANPOINT((-9.24500268122647,-9.2899457324743,-4.1166204318426));\n#2473=IFCCARTESIANPOINT((-10.,-10.,-4.13));\n#2474=IFCPOLYLOOP((#2468,#2472,#2473));\n#2475=IFCFACEOUTERBOUND(#2474,.T.);\n#2476=IFCFACE((#2475));\n#2477=IFCPOLYLOOP((#2468,#772,#2472));\n#2478=IFCFACEOUTERBOUND(#2477,.T.);\n#2479=IFCFACE((#2478));\n#2480=IFCCARTESIANPOINT((-9.38276715633794,-7.49654080493779,-3.89293654982009));\n#2481=IFCPOLYLOOP((#2480,#794,#2473));\n#2482=IFCFACEOUTERBOUND(#2481,.T.);\n#2483=IFCFACE((#2482));\n#2484=IFCPOLYLOOP((#2480,#2473,#2472));\n#2485=IFCFACEOUTERBOUND(#2484,.T.);\n#2486=IFCFACE((#2485));\n#2487=IFCPOLYLOOP((#2480,#2472,#772));\n#2488=IFCFACEOUTERBOUND(#2487,.T.);\n#2489=IFCFACE((#2488));\n#2490=IFCCARTESIANPOINT((-9.19670877209541,-7.26684625984806,-3.77356048824769));\n#2491=IFCPOLYLOOP((#2490,#794,#2480));\n#2492=IFCFACEOUTERBOUND(#2491,.T.);\n#2493=IFCFACE((#2492));\n#2494=IFCPOLYLOOP((#2490,#780,#794));\n#2495=IFCFACEOUTERBOUND(#2494,.T.);\n#2496=IFCFACE((#2495));\n#2497=IFCPOLYLOOP((#2490,#772,#780));\n#2498=IFCFACEOUTERBOUND(#2497,.T.);\n#2499=IFCFACE((#2498));\n#2500=IFCPOLYLOOP((#2490,#2480,#772));\n#2501=IFCFACEOUTERBOUND(#2500,.T.);\n#2502=IFCFACE((#2501));\n#2503=IFCCARTESIANPOINT((-3.93125598556019,-9.49340532827569,-4.47000917194718));\n#2504=IFCPOLYLOOP((#2503,#919,#918));\n#2505=IFCFACEOUTERBOUND(#2504,.T.);\n#2506=IFCFACE((#2505));\n#2507=IFCPOLYLOOP((#2503,#918,#923));\n#2508=IFCFACEOUTERBOUND(#2507,.T.);\n#2509=IFCFACE((#2508));\n#2510=IFCCARTESIANPOINT((-3.83948849896211,-7.40187764634,-2.65601560859763));\n#2511=IFCPOLYLOOP((#2510,#384,#951));\n#2512=IFCFACEOUTERBOUND(#2511,.T.);\n#2513=IFCFACE((#2512));\n#2514=IFCPOLYLOOP((#2510,#923,#384));\n#2515=IFCFACEOUTERBOUND(#2514,.T.);\n#2516=IFCFACE((#2515));\n#2517=IFCPOLYLOOP((#2510,#883,#813));\n#2518=IFCFACEOUTERBOUND(#2517,.T.);\n#2519=IFCFACE((#2518));\n#2520=IFCPOLYLOOP((#2510,#813,#2503));\n#2521=IFCFACEOUTERBOUND(#2520,.T.);\n#2522=IFCFACE((#2521));\n#2523=IFCPOLYLOOP((#2510,#2503,#923));\n#2524=IFCFACEOUTERBOUND(#2523,.T.);\n#2525=IFCFACE((#2524));\n#2526=IFCPOLYLOOP((#2510,#951,#883));\n#2527=IFCFACEOUTERBOUND(#2526,.T.);\n#2528=IFCFACE((#2527));\n#2529=IFCCARTESIANPOINT((-8.12967619613619,3.26562298669175,-2.89403795475305));\n#2530=IFCPOLYLOOP((#2529,#1260,#474));\n#2531=IFCFACEOUTERBOUND(#2530,.T.);\n#2532=IFCFACE((#2531));\n#2533=IFCPOLYLOOP((#2529,#1219,#1260));\n#2534=IFCFACEOUTERBOUND(#2533,.T.);\n#2535=IFCFACE((#2534));\n#2536=IFCPOLYLOOP((#2529,#1102,#1219));\n#2537=IFCFACEOUTERBOUND(#2536,.T.);\n#2538=IFCFACE((#2537));\n#2539=IFCPOLYLOOP((#2529,#1144,#1102));\n#2540=IFCFACEOUTERBOUND(#2539,.T.);\n#2541=IFCFACE((#2540));\n#2542=IFCPOLYLOOP((#2529,#474,#1144));\n#2543=IFCFACEOUTERBOUND(#2542,.T.);\n#2544=IFCFACE((#2543));\n#2545=IFCCARTESIANPOINT((-6.25626914885146,1.84858291015222,-1.46392330203646));\n#2546=IFCPOLYLOOP((#2545,#438,#1219));\n#2547=IFCFACEOUTERBOUND(#2546,.T.);\n#2548=IFCFACE((#2547));\n#2549=IFCPOLYLOOP((#2545,#1161,#430));\n#2550=IFCFACEOUTERBOUND(#2549,.T.);\n#2551=IFCFACE((#2550));\n#2552=IFCPOLYLOOP((#2545,#1098,#1161));\n#2553=IFCFACEOUTERBOUND(#2552,.T.);\n#2554=IFCFACE((#2553));\n#2555=IFCPOLYLOOP((#2545,#1178,#1098));\n#2556=IFCFACEOUTERBOUND(#2555,.T.);\n#2557=IFCFACE((#2556));\n#2558=IFCPOLYLOOP((#2545,#1219,#1178));\n#2559=IFCFACEOUTERBOUND(#2558,.T.);\n#2560=IFCFACE((#2559));\n#2561=IFCPOLYLOOP((#2545,#434,#438));\n#2562=IFCFACEOUTERBOUND(#2561,.T.);\n#2563=IFCFACE((#2562));\n#2564=IFCPOLYLOOP((#2545,#430,#434));\n#2565=IFCFACEOUTERBOUND(#2564,.T.);\n#2566=IFCFACE((#2565));\n#2567=IFCCARTESIANPOINT((8.95425238548131,-9.36511289390634,-5.82647010683197));\n#2568=IFCCARTESIANPOINT((10.,-10.,-6.13));\n#2569=IFCPOLYLOOP((#2567,#451,#2568));\n#2570=IFCFACEOUTERBOUND(#2569,.T.);\n#2571=IFCFACE((#2570));\n#2572=IFCPOLYLOOP((#2567,#2568,#1537));\n#2573=IFCFACEOUTERBOUND(#2572,.T.);\n#2574=IFCFACE((#2573));\n#2575=IFCPOLYLOOP((#2567,#1464,#451));\n#2576=IFCFACEOUTERBOUND(#2575,.T.);\n#2577=IFCFACE((#2576));\n#2578=IFCPOLYLOOP((#2567,#1537,#1464));\n#2579=IFCFACEOUTERBOUND(#2578,.T.);\n#2580=IFCFACE((#2579));\n#2581=IFCCARTESIANPOINT((9.78552887689799,-7.56218406598153,-5.51967531496118));\n#2582=IFCPOLYLOOP((#2581,#1750,#1530));\n#2583=IFCFACEOUTERBOUND(#2582,.T.);\n#2584=IFCFACE((#2583));\n#2585=IFCPOLYLOOP((#2581,#2568,#1755));\n#2586=IFCFACEOUTERBOUND(#2585,.T.);\n#2587=IFCFACE((#2586));\n#2588=IFCPOLYLOOP((#2581,#1755,#1754));\n#2589=IFCFACEOUTERBOUND(#2588,.T.);\n#2590=IFCFACE((#2589));\n#2591=IFCPOLYLOOP((#2581,#1537,#2568));\n#2592=IFCFACEOUTERBOUND(#2591,.T.);\n#2593=IFCFACE((#2592));\n#2594=IFCPOLYLOOP((#2581,#1530,#1517));\n#2595=IFCFACEOUTERBOUND(#2594,.T.);\n#2596=IFCFACE((#2595));\n#2597=IFCPOLYLOOP((#2581,#1754,#1750));\n#2598=IFCFACEOUTERBOUND(#2597,.T.);\n#2599=IFCFACE((#2598));\n#2600=IFCPOLYLOOP((#2581,#1517,#1537));\n#2601=IFCFACEOUTERBOUND(#2600,.T.);\n#2602=IFCFACE((#2601));\n#2603=IFCCARTESIANPOINT((8.48456491609014,-5.01445881216711,-3.2276751667918));\n#2604=IFCPOLYLOOP((#2603,#1530,#1750));\n#2605=IFCFACEOUTERBOUND(#2604,.T.);\n#2606=IFCFACE((#2605));\n#2607=IFCPOLYLOOP((#2603,#1750,#1749));\n#2608=IFCFACEOUTERBOUND(#2607,.T.);\n#2609=IFCFACE((#2608));\n#2610=IFCPOLYLOOP((#2603,#1749,#599));\n#2611=IFCFACEOUTERBOUND(#2610,.T.);\n#2612=IFCFACE((#2611));\n#2613=IFCPOLYLOOP((#2603,#1492,#1530));\n#2614=IFCFACEOUTERBOUND(#2613,.T.);\n#2615=IFCFACE((#2614));\n#2616=IFCPOLYLOOP((#2603,#1626,#1492));\n#2617=IFCFACEOUTERBOUND(#2616,.T.);\n#2618=IFCFACE((#2617));\n#2619=IFCPOLYLOOP((#2603,#599,#1626));\n#2620=IFCFACEOUTERBOUND(#2619,.T.);\n#2621=IFCFACE((#2620));\n#2622=IFCCARTESIANPOINT((8.36173828125,0.52494873046875,-1.55491943359375));\n#2623=IFCPOLYLOOP((#2622,#1987,#696));\n#2624=IFCFACEOUTERBOUND(#2623,.T.);\n#2625=IFCFACE((#2624));\n#2626=IFCPOLYLOOP((#2622,#1793,#642));\n#2627=IFCFACEOUTERBOUND(#2626,.T.);\n#2628=IFCFACE((#2627));\n#2629=IFCPOLYLOOP((#2622,#696,#1793));\n#2630=IFCFACEOUTERBOUND(#2629,.T.);\n#2631=IFCFACE((#2630));\n#2632=IFCPOLYLOOP((#2622,#642,#1987));\n#2633=IFCFACEOUTERBOUND(#2632,.T.);\n#2634=IFCFACE((#2633));\n#2635=IFCCARTESIANPOINT((2.67063505502873,7.5205692437348,-1.29495133204325));\n#2636=IFCPOLYLOOP((#2635,#611,#1994));\n#2637=IFCFACEOUTERBOUND(#2636,.T.);\n#2638=IFCFACE((#2637));\n#2639=IFCPOLYLOOP((#2635,#691,#676));\n#2640=IFCFACEOUTERBOUND(#2639,.T.);\n#2641=IFCFACE((#2640));\n#2642=IFCPOLYLOOP((#2635,#702,#691));\n#2643=IFCFACEOUTERBOUND(#2642,.T.);\n#2644=IFCFACE((#2643));\n#2645=IFCPOLYLOOP((#2635,#676,#1862));\n#2646=IFCFACEOUTERBOUND(#2645,.T.);\n#2647=IFCFACE((#2646));\n#2648=IFCPOLYLOOP((#2635,#1862,#611));\n#2649=IFCFACEOUTERBOUND(#2648,.T.);\n#2650=IFCFACE((#2649));\n#2651=IFCCARTESIANPOINT((3.35920839876876,7.41441842456308,-1.3151495578287));\n#2652=IFCPOLYLOOP((#2651,#2635,#1994));\n#2653=IFCFACEOUTERBOUND(#2652,.T.);\n#2654=IFCFACE((#2653));\n#2655=IFCPOLYLOOP((#2651,#1994,#2015));\n#2656=IFCFACEOUTERBOUND(#2655,.T.);\n#2657=IFCFACE((#2656));\n#2658=IFCPOLYLOOP((#2651,#2015,#712));\n#2659=IFCFACEOUTERBOUND(#2658,.T.);\n#2660=IFCFACE((#2659));\n#2661=IFCPOLYLOOP((#2651,#712,#702));\n#2662=IFCFACEOUTERBOUND(#2661,.T.);\n#2663=IFCFACE((#2662));\n#2664=IFCPOLYLOOP((#2651,#702,#2635));\n#2665=IFCFACEOUTERBOUND(#2664,.T.);\n#2666=IFCFACE((#2665));\n#2667=IFCCARTESIANPOINT((-6.68194696517004,-9.75254460185546,-4.44665225504588));\n#2668=IFCPOLYLOOP((#2667,#2473,#919));\n#2669=IFCFACEOUTERBOUND(#2668,.T.);\n#2670=IFCFACE((#2669));\n#2671=IFCPOLYLOOP((#2667,#776,#2468));\n#2672=IFCFACEOUTERBOUND(#2671,.T.);\n#2673=IFCFACE((#2672));\n#2674=IFCPOLYLOOP((#2667,#919,#2503));\n#2675=IFCFACEOUTERBOUND(#2674,.T.);\n#2676=IFCFACE((#2675));\n#2677=IFCPOLYLOOP((#2667,#813,#801));\n#2678=IFCFACEOUTERBOUND(#2677,.T.);\n#2679=IFCFACE((#2678));\n#2680=IFCPOLYLOOP((#2667,#770,#776));\n#2681=IFCFACEOUTERBOUND(#2680,.T.);\n#2682=IFCFACE((#2681));\n#2683=IFCPOLYLOOP((#2667,#801,#770));\n#2684=IFCFACEOUTERBOUND(#2683,.T.);\n#2685=IFCFACE((#2684));\n#2686=IFCPOLYLOOP((#2667,#2468,#2473));\n#2687=IFCFACEOUTERBOUND(#2686,.T.);\n#2688=IFCFACE((#2687));\n#2689=IFCPOLYLOOP((#2667,#2503,#813));\n#2690=IFCFACEOUTERBOUND(#2689,.T.);\n#2691=IFCFACE((#2690));\n#2692=IFCOPENSHELL((#356,#360,#366,#370,#376,#381,#387,#391,#394,#398,#401,#405,#409,#413,#417,#420,#424,#428,#433,#437,#441,#444,#448,#454,#460,#464,#468,#472,#478,#483,#487,#490,#493,#496,#499,#503,#507,#510,#514,#517,#522,#525,#529,#533,#537,#540,#543,#547,#551,#554,#557,#561,#565,#571,#575,#578,#581,#585,#589,#593,#596,#602,#606,#610,#614,#618,#621,#624,#628,#631,#635,#640,#645,#649,#653,#656,#660,#665,#668,#672,#675,#679,#683,#686,#690,#694,#700,#705,#710,#715,#720,#725,#730,#733,#739,#743,#747,#751,#756,#759,#763,#766,#769,#775,#779,#783,#787,#790,#793,#797,#800,#804,#808,#811,#816,#819,#822,#827,#830,#834,#837,#841,#844,#848,#852,#856,#859,#862,#865,#869,#872,#875,#879,#882,#886,#891,#894,#897,#900,#904,#907,#910,#913,#916,#922,#926,#930,#933,#936,#940,#943,#947,#950,#954,#957,#960,#964,#967,#970,#973,#976,#981,#984,#987,#991,#995,#998,#1001,#1004,#1008,#1012,#1016,#1019,#1022,#1026,#1030,#1033,#1037,#1040,#1043,#1047,#1050,#1054,#1058,#1061,#1064,#1067,#1071,#1074,#1077,#1080,#1083,#1087,#1090,#1093,#1097,#1101,#1105,#1108,#1112,#1115,#1118,#1123,#1126,#1130,#1133,#1136,#1140,#1143,#1147,#1150,#1154,#1157,#1160,#1164,#1167,#1170,#1174,#1177,#1181,#1185,#1188,#1191,#1195,#1198,#1201,#1204,#1208,#1211,#1215,#1218,#1222,#1226,#1230,#1233,#1236,#1240,#1243,#1246,#1250,#1253,#1256,#1259,#1263,#1266,#1270,#1273,#1276,#1281,#1285,#1288,#1291,#1295,#1298,#1301,#1304,#1307,#1311,#1314,#1318,#1321,#1324,#1328,#1331,#1334,#1337,#1341,#1344,#1348,#1351,#1354,#1358,#1363,#1366,#1369,#1372,#1375,#1378,#1381,#1385,#1388,#1393,#1397,#1402,#1405,#1408,#1411,#1415,#1418,#1422,#1425,#1429,#1432,#1436,#1440,#1443,#1446,#1449,#1453,#1456,#1460,#1463,#1467,#1471,#1474,#1477,#1480,#1484,#1487,#1491,#1495,#1498,#1502,#1505,#1509,#1512,#1515,#1520,#1523,#1526,#1529,#1533,#1536,#1540,#1544,#1548,#1551,#1554,#1558,#1561,#1564,#1568,#1571,#1574,#1577,#1580,#1584,#1587,#1592,#1595,#1598,#1602,#1605,#1609,#1613,#1616,#1619,#1622,#1625,#1629,#1633,#1636,#1639,#1643,#1646,#1649,#1653,#1656,#1659,#1663,#1666,#1669,#1672,#1675,#1679,#1682,#1685,#1689,#1692,#1695,#1699,#1702,#1706,#1710,#1713,#1716,#1719,#1723,#1727,#1731,#1734,#1737,#1740,#1743,#1747,#1753,#1758,#1761,#1765,#1768,#1771,#1774,#1778,#1782,#1785,#1788,#1791,#1796,#1799,#1802,#1805,#1809,#1813,#1816,#1819,#1822,#1826,#1829,#1832,#1835,#1838,#1841,#1845,#1848,#1852,#1855,#1858,#1861,#1865,#1868,#1871,#1875,#1878,#1881,#1885,#1889,#1894,#1897,#1900,#1903,#1906,#1909,#1913,#1917,#1920,#1923,#1927,#1931,#1934,#1937,#1942,#1945,#1948,#1951,#1955,#1958,#1963,#1966,#1969,#1972,#1977,#1980,#1983,#1986,#1990,#1993,#1997,#2001,#2004,#2007,#2011,#2014,#2018,#2022,#2025,#2028,#2032,#2035,#2038,#2041,#2044,#2048,#2051,#2054,#2057,#2061,#2064,#2068,#2071,#2075,#2078,#2081,#2085,#2088,#2091,#2094,#2097,#2101,#2104,#2108,#2111,#2114,#2117,#2122,#2125,#2128,#2133,#2136,#2139,#2142,#2146,#2150,#2153,#2156,#2159,#2163,#2167,#2170,#2173,#2177,#2180,#2183,#2187,#2190,#2194,#2197,#2201,#2204,#2207,#2211,#2214,#2218,#2221,#2224,#2227,#2231,#2234,#2237,#2241,#2244,#2247,#2251,#2254,#2257,#2261,#2264,#2267,#2271,#2274,#2278,#2281,#2285,#2288,#2292,#2295,#2299,#2302,#2307,#2310,#2313,#2317,#2320,#2323,#2327,#2330,#2334,#2337,#2341,#2345,#2348,#2351,#2354,#2358,#2362,#2365,#2369,#2372,#2375,#2379,#2382,#2385,#2388,#2392,#2395,#2398,#2402,#2405,#2409,#2412,#2415,#2418,#2422,#2425,#2428,#2431,#2435,#2438,#2442,#2445,#2448,#2452,#2455,#2458,#2461,#2464,#2467,#2471,#2476,#2479,#2483,#2486,#2489,#2493,#2496,#2499,#2502,#2506,#2509,#2513,#2516,#2519,#2522,#2525,#2528,#2532,#2535,#2538,#2541,#2544,#2548,#2551,#2554,#2557,#2560,#2563,#2566,#2571,#2574,#2577,#2580,#2584,#2587,#2590,#2593,#2596,#2599,#2602,#2606,#2609,#2612,#2615,#2618,#2621,#2625,#2628,#2631,#2634,#2638,#2641,#2644,#2647,#2650,#2654,#2657,#2660,#2663,#2666,#2670,#2673,#2676,#2679,#2682,#2685,#2688,#2691));\n#2693=IFCFACEBASEDSURFACEMODEL((#2692));\n#2694=IFCSHAPEREPRESENTATION(#12,'Facetation','SurfaceModel',(#2693));\n#2695=IFCPRODUCTDEFINITIONSHAPE($,$,(#2694));\n#2696=IFCSURFACESTYLE($,.BOTH.,(#2697));\n#2697=IFCSURFACESTYLESHADING(#2698,0.);\n#2698=IFCCOLOURRGB($,0.15,0.25,0.05);\n#2699=IFCSTYLEDITEM(#2693,(#2696),$);\n#2700=IFCMATERIAL('Brick',$,'brick');\n#2701=IFCMATERIALLAYERSET((#2702),'Wall',$);\n#2702=IFCMATERIALLAYER(#2700,0.36,$,$,$,$,$);\n#2703=IFCMATERIALLAYERSET($,$,$);\n#2704=IFCMATERIALLAYERSETUSAGE(#2701,.AXIS2.,.POSITIVE.,0.,$);\n#2705=IFCRELASSOCIATESMATERIAL('2B3jSMCzr52RnROrydw$9K',$,$,$,(#41),#2704);\n#2706=IFCMATERIALLAYERSET($,$,$);\n#2707=IFCMATERIALLAYERSETUSAGE(#2701,.AXIS2.,.POSITIVE.,0.,$);\n#2708=IFCRELASSOCIATESMATERIAL('36JIAeoOv3fQlE6DhSQ_rd',$,$,$,(#214),#2707);\n#2709=IFCMATERIALLAYERSET($,$,$);\n#2710=IFCMATERIALLAYERSETUSAGE(#2701,.AXIS2.,.POSITIVE.,0.,$);\n#2711=IFCRELASSOCIATESMATERIAL('3DTNQObzPFogDTbKjGhZxq',$,$,$,(#222),#2710);\n#2712=IFCMATERIALLAYERSET($,$,$);\n#2713=IFCMATERIALLAYERSETUSAGE(#2701,.AXIS2.,.POSITIVE.,0.,$);\n#2714=IFCRELASSOCIATESMATERIAL('2lnKwfXnr6$Pw5XYUN$BHC',$,$,$,(#254),#2713);\n#2715=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677455,#19,#16,1682677455);\n#2716=IFCSTAIRFLIGHT('2pRloCeBj4jeY2D8pisUtH',#2715,$,$,$,#2732,#2727,$,2,2,0.2,0.25,.STRAIGHT.);\n#2717=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.25,0.),(0.25,0.2),(0.5,0.2),(0.5,0.4),(0.,0.4)),$);\n#2718=IFCINDEXEDPOLYCURVE(#2717,$,.F.);\n#2719=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2718);\n#2720=IFCCARTESIANPOINT((0.,0.,0.));\n#2721=IFCDIRECTION((0.,0.,1.));\n#2722=IFCDIRECTION((1.,0.,0.));\n#2723=IFCAXIS2PLACEMENT3D(#2720,#2721,#2722);\n#2724=IFCDIRECTION((0.,0.,1.));\n#2725=IFCEXTRUDEDAREASOLID(#2719,#2723,#2724,1.2);\n#2726=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2725));\n#2727=IFCPRODUCTDEFINITIONSHAPE($,$,(#2726));\n#2728=IFCCARTESIANPOINT((5.41,1.5,0.));\n#2729=IFCDIRECTION((0.,1.,0.));\n#2730=IFCDIRECTION((1.,0.,0.));\n#2731=IFCAXIS2PLACEMENT3D(#2728,#2729,#2730);\n#2732=IFCLOCALPLACEMENT($,#2731);\n#2733=IFCSTYLEDITEM(#2725,(#82),$);\n#2734=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677456,#19,#16,1682677456);\n#2735=IFCDOOR('0QlRifYUP2pwzaF3f7HQnN',#2734,'Main door',$,$,#2838,#2839,$,$,$,.DOOR.,$,$);\n#2736=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677456,#19,#16,1682677456);\n#2737=IFCOPENINGELEMENT('2I41WpEJD1vemCI7PmMJUY',#2736,$,$,$,#2760,#2748,$,$);\n#2738=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,1.),(1.,1.),(1.,0.),(0.,0.)),$);\n#2739=IFCINDEXEDPOLYCURVE(#2738,$,.F.);\n#2740=IFCDIRECTION((0.,0.,1.));\n#2741=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2739);\n#2742=IFCCARTESIANPOINT((0.,0.,0.));\n#2743=IFCDIRECTION((0.,0.,1.));\n#2744=IFCDIRECTION((1.,0.,0.));\n#2745=IFCAXIS2PLACEMENT3D(#2742,#2743,#2744);\n#2746=IFCEXTRUDEDAREASOLID(#2741,#2745,#2740,2.2);\n#2747=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2746));\n#2748=IFCPRODUCTDEFINITIONSHAPE($,$,(#2747));\n#2754=IFCOWNERHISTORY(#19,#16,.READWRITE.,.ADDED.,1682677456,#19,#16,1682677456);\n#2755=IFCRELVOIDSELEMENT('2FS3Z$3ED7nv_v_B18vctJ',#2754,$,$,#222,#2737);\n#2756=IFCCARTESIANPOINT((-0.5,1.78,0.));\n#2757=IFCDIRECTION((0.,0.,1.));\n#2758=IFCDIRECTION((1.,0.,0.));\n#2759=IFCAXIS2PLACEMENT3D(#2756,#2757,#2758);\n#2760=IFCLOCALPLACEMENT(#252,#2759);\n#2761=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,2.20000004768372),(1.,2.20000004768372),(1.,0.),(0.975000023841858,0.),(0.975000023841858,2.17499995231628),(0.025000000372529,2.17499995231628),(0.025000000372529,0.)),$);\n#2762=IFCINDEXEDPOLYCURVE(#2761,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,7)),IFCLINEINDEX((7,8)),IFCLINEINDEX((8,1))),$);\n#2763=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2762);\n#2764=IFCCARTESIANPOINT((0.,0.025000000372529,0.));\n#2765=IFCDIRECTION((0.,-1.,0.));\n#2766=IFCDIRECTION((1.,0.,0.));\n#2767=IFCAXIS2PLACEMENT3D(#2764,#2765,#2766);\n#2768=IFCDIRECTION((0.,0.,-1.));\n#2769=IFCEXTRUDEDAREASOLID(#2763,#2767,#2768,0.025000000372529);\n#2770=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,2.20000004768372),(1.,2.20000004768372),(1.,0.),(0.949999988079071,0.),(0.949999988079071,2.15000009536743),(0.0500000007450581,2.15000009536743),(0.0500000007450581,0.)),$);\n#2771=IFCINDEXEDPOLYCURVE(#2770,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,7)),IFCLINEINDEX((7,8)),IFCLINEINDEX((8,1))),$);\n#2772=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2771);\n#2773=IFCCARTESIANPOINT((0.,0.,0.));\n#2774=IFCDIRECTION((0.,-1.,0.));\n#2775=IFCDIRECTION((1.,0.,0.));\n#2776=IFCAXIS2PLACEMENT3D(#2773,#2774,#2775);\n#2777=IFCDIRECTION((0.,0.,-1.));\n#2778=IFCEXTRUDEDAREASOLID(#2772,#2776,#2777,0.025000000372529);\n#2779=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.899999976158142,0.),(0.899999976158142,0.100000001490116),(0.,0.100000001490116)),$);\n#2780=IFCINDEXEDPOLYCURVE(#2779,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2781=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2780);\n#2782=IFCCARTESIANPOINT((0.0500000007450581,0.,0.));\n#2783=IFCDIRECTION((0.,0.,1.));\n#2784=IFCDIRECTION((1.,0.,0.));\n#2785=IFCAXIS2PLACEMENT3D(#2782,#2783,#2784);\n#2786=IFCDIRECTION((0.,0.,1.));\n#2787=IFCEXTRUDEDAREASOLID(#2781,#2785,#2786,0.025000000372529);\n#2788=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,2.22499990463257),(1.04999995231628,2.22499990463257),(1.04999995231628,0.),(0.974999964237213,0.),(0.974999964237213,2.14999985694885),(0.0750000029802322,2.14999985694885),(0.0750000029802322,0.)),$);\n#2789=IFCINDEXEDPOLYCURVE(#2788,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,7)),IFCLINEINDEX((7,8)),IFCLINEINDEX((8,1))),$);\n#2790=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2789);\n#2791=IFCCARTESIANPOINT((-0.025000000372529,-0.00499999988824129,0.));\n#2792=IFCDIRECTION((0.,-1.,0.));\n#2793=IFCDIRECTION((1.,0.,0.));\n#2794=IFCAXIS2PLACEMENT3D(#2791,#2792,#2793);\n#2795=IFCDIRECTION((0.,0.,-1.));\n#2796=IFCEXTRUDEDAREASOLID(#2790,#2794,#2795,0.00499999988824129);\n#2797=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,2.22499990463257),(1.04999995231628,2.22499990463257),(1.04999995231628,0.),(0.999999940395355,0.),(0.999999940395355,2.17499995231628),(0.0500000007450581,2.17499995231628),(0.0500000007450581,0.)),$);\n#2798=IFCINDEXEDPOLYCURVE(#2797,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,7)),IFCLINEINDEX((7,8)),IFCLINEINDEX((8,1))),$);\n#2799=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2798);\n#2800=IFCCARTESIANPOINT((-0.025000000372529,0.0500000007450581,0.));\n#2801=IFCDIRECTION((0.,-1.,0.));\n#2802=IFCDIRECTION((1.,0.,0.));\n#2803=IFCAXIS2PLACEMENT3D(#2800,#2801,#2802);\n#2804=IFCDIRECTION((0.,0.,-1.));\n#2805=IFCEXTRUDEDAREASOLID(#2799,#2803,#2804,0.00499999988824129);\n#2806=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.949999988079071,0.),(0.949999988079071,0.0350000001490116),(0.,0.0350000001490116)),$);\n#2807=IFCINDEXEDPOLYCURVE(#2806,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2808=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2807);\n#2809=IFCCARTESIANPOINT((0.025000000372529,0.025000000372529,0.025000000372529));\n#2810=IFCDIRECTION((0.,0.,1.));\n#2811=IFCDIRECTION((1.,0.,0.));\n#2812=IFCAXIS2PLACEMENT3D(#2809,#2810,#2811);\n#2813=IFCDIRECTION((0.,0.,1.));\n#2814=IFCEXTRUDEDAREASOLID(#2808,#2812,#2813,2.15000009536743);\n#2815=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,-0.0400000028312206),(-0.120000004768372,-0.0400000028312206),(-0.120000004768372,-0.0200000014156103),(-0.0200000405311584,-0.0200000014156103),(-0.0200000405311584,0.)),$);\n#2816=IFCINDEXEDPOLYCURVE(#2815,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,1))),$);\n#2817=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2816);\n#2818=IFCCARTESIANPOINT((0.925000011920929,0.025000000372529,1.01499998569489));\n#2819=IFCDIRECTION((0.,0.,1.));\n#2820=IFCDIRECTION((1.,0.,0.));\n#2821=IFCAXIS2PLACEMENT3D(#2818,#2819,#2820);\n#2822=IFCDIRECTION((0.,0.,1.));\n#2823=IFCEXTRUDEDAREASOLID(#2817,#2821,#2822,0.0200000014156103);\n#2824=IFCEXTRUDEDAREASOLID(#2825,#2828,#2832,0.0200000014156103);\n#2825=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2826);\n#2826=IFCINDEXEDPOLYCURVE(#2827,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,5)),IFCLINEINDEX((5,6)),IFCLINEINDEX((6,1))),$);\n#2827=IFCCARTESIANPOINTLIST2D(((0.,0.),(0.,0.0400000065565109),(-0.120000004768372,0.0400000065565109),(-0.120000004768372,0.0199999958276749),(-0.0200000405311584,0.0199999958276749),(-0.0200000405311584,0.)),$);\n#2828=IFCAXIS2PLACEMENT3D(#2829,#2830,#2831);\n#2829=IFCCARTESIANPOINT((0.925000011920929,0.0600000023841858,1.01499998569489));\n#2830=IFCDIRECTION((0.,0.,1.));\n#2831=IFCDIRECTION((1.,0.,0.));\n#2832=IFCDIRECTION((0.,0.,1.));\n#2833=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2769,#2778,#2814,#2823,#2824,#2787,#2796,#2805));\n#2834=IFCCARTESIANPOINT((5.09,1.6,0.));\n#2835=IFCDIRECTION((0.,0.,1.));\n#2836=IFCDIRECTION((0.,1.,0.));\n#2837=IFCAXIS2PLACEMENT3D(#2834,#2835,#2836);\n#2838=IFCLOCALPLACEMENT($,#2837);\n#2839=IFCPRODUCTDEFINITIONSHAPE($,$,(#2833));\n#2840=IFCRELFILLSELEMENT('0BUFJ2OpzAfwNpv1YO2S2t',$,$,$,#2737,#2735);\n#2841=IFCSURFACESTYLE($,.BOTH.,(#2842));\n#2842=IFCSURFACESTYLESHADING(#2843,0.);\n#2843=IFCCOLOURRGB($,0.8,0.8,0.8);\n#2844=IFCSTYLEDITEM(#2769,(#2841),$);\n#2845=IFCSTYLEDITEM(#2778,(#2841),$);\n#2846=IFCSTYLEDITEM(#2814,(#2841),$);\n#2847=IFCSTYLEDITEM(#2823,(#2841),$);\n#2848=IFCSTYLEDITEM(#2824,(#2841),$);\n#2849=IFCSTYLEDITEM(#2787,(#2841),$);\n#2850=IFCSTYLEDITEM(#2796,(#2841),$);\n#2851=IFCSTYLEDITEM(#2805,(#2841),$);\n#2852=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677457,#19,#16,1682677457);\n#2853=IFCWINDOW('3STMaeBvz7kh0yZpyDqQ0s',#2852,'Right window',$,$,#2901,#2902,$,$,$,.WINDOW.,.SINGLE_PANEL.,$);\n#2854=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.86000001430511,0.),(1.86000001430511,1.60000002384186),(0.,1.60000002384186)),$);\n#2855=IFCINDEXEDPOLYCURVE(#2854,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2856=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.025000000372529),(1.83500003814697,0.025000000372529),(1.83500003814697,1.57500004768372),(0.025000000372529,1.57500004768372)),$);\n#2857=IFCINDEXEDPOLYCURVE(#2856,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2858=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2855,(#2857));\n#2859=IFCCARTESIANPOINT((0.,0.0900000035762787,0.));\n#2860=IFCDIRECTION((0.,-1.,0.));\n#2861=IFCDIRECTION((1.,0.,0.));\n#2862=IFCAXIS2PLACEMENT3D(#2859,#2860,#2861);\n#2863=IFCDIRECTION((0.,0.,-1.));\n#2864=IFCEXTRUDEDAREASOLID(#2858,#2862,#2863,0.0100000007078052);\n#2865=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.86000001430511,0.),(1.86000001430511,1.60000002384186),(0.,1.60000002384186)),$);\n#2866=IFCINDEXEDPOLYCURVE(#2865,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2867=IFCCARTESIANPOINTLIST2D(((0.0500000007450581,0.0500000007450581),(1.80999994277954,0.0500000007450581),(1.80999994277954,1.54999995231628),(0.0500000007450581,1.54999995231628)),$);\n#2868=IFCINDEXEDPOLYCURVE(#2867,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2869=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2866,(#2868));\n#2870=IFCCARTESIANPOINT((0.,0.0500000007450581,0.));\n#2871=IFCDIRECTION((0.,-1.,0.));\n#2872=IFCDIRECTION((1.,0.,0.));\n#2873=IFCAXIS2PLACEMENT3D(#2870,#2871,#2872);\n#2874=IFCDIRECTION((0.,0.,-1.));\n#2875=IFCEXTRUDEDAREASOLID(#2869,#2873,#2874,0.0399999991059303);\n#2876=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.80999994277954,0.),(1.80999994277954,1.54999995231628),(0.,1.54999995231628)),$);\n#2877=IFCINDEXEDPOLYCURVE(#2876,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2878=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.77499985694885,0.0350000001490116),(1.77499985694885,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2879=IFCINDEXEDPOLYCURVE(#2878,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2880=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2877,(#2879));\n#2881=IFCCARTESIANPOINT((0.025000000372529,0.0900000035762787,0.025000000372529));\n#2882=IFCDIRECTION((0.,-1.,0.));\n#2883=IFCDIRECTION((1.,0.,0.));\n#2884=IFCAXIS2PLACEMENT3D(#2881,#2882,#2883);\n#2885=IFCDIRECTION((0.,0.,-1.));\n#2886=IFCEXTRUDEDAREASOLID(#2880,#2884,#2885,0.0350000001490116);\n#2887=IFCINDEXEDPOLYCURVE(#2888,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2888=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.77499985694885,0.0350000001490116),(1.77499985694885,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2889=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2887);\n#2890=IFCCARTESIANPOINT((0.025000000372529,0.10249999910593,0.025000000372529));\n#2891=IFCDIRECTION((0.,-1.,0.));\n#2892=IFCDIRECTION((1.,0.,0.));\n#2893=IFCAXIS2PLACEMENT3D(#2890,#2891,#2892);\n#2894=IFCDIRECTION((0.,0.,-1.));\n#2895=IFCEXTRUDEDAREASOLID(#2889,#2893,#2894,0.01);\n#2896=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2864,#2875,#2886,#2895));\n#2897=IFCCARTESIANPOINT((2.,-0.12,0.4));\n#2898=IFCDIRECTION((0.,0.,1.));\n#2899=IFCDIRECTION((1.,0.,0.));\n#2900=IFCAXIS2PLACEMENT3D(#2897,#2898,#2899);\n#2901=IFCLOCALPLACEMENT($,#2900);\n#2902=IFCPRODUCTDEFINITIONSHAPE($,$,(#2896));\n#2903=IFCRELFILLSELEMENT('3bGFOQWgLB6gsgqHZTBEue',$,$,$,#112,#2853);\n#2904=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677457,#19,#16,1682677457);\n#2905=IFCWINDOW('16$bvz4NTEV8ZRpseE39jT',#2904,'West window',$,$,#2953,#2954,$,$,$,.WINDOW.,.SINGLE_PANEL.,$);\n#2906=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.86000001430511,0.),(1.86000001430511,1.60000002384186),(0.,1.60000002384186)),$);\n#2907=IFCINDEXEDPOLYCURVE(#2906,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2908=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.025000000372529),(1.83500003814697,0.025000000372529),(1.83500003814697,1.57500004768372),(0.025000000372529,1.57500004768372)),$);\n#2909=IFCINDEXEDPOLYCURVE(#2908,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2910=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2907,(#2909));\n#2911=IFCCARTESIANPOINT((0.,0.0900000035762787,0.));\n#2912=IFCDIRECTION((0.,-1.,0.));\n#2913=IFCDIRECTION((1.,0.,0.));\n#2914=IFCAXIS2PLACEMENT3D(#2911,#2912,#2913);\n#2915=IFCDIRECTION((0.,0.,-1.));\n#2916=IFCEXTRUDEDAREASOLID(#2910,#2914,#2915,0.0100000007078052);\n#2917=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.86000001430511,0.),(1.86000001430511,1.60000002384186),(0.,1.60000002384186)),$);\n#2918=IFCINDEXEDPOLYCURVE(#2917,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2919=IFCCARTESIANPOINTLIST2D(((0.0500000007450581,0.0500000007450581),(1.80999994277954,0.0500000007450581),(1.80999994277954,1.54999995231628),(0.0500000007450581,1.54999995231628)),$);\n#2920=IFCINDEXEDPOLYCURVE(#2919,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2921=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2918,(#2920));\n#2922=IFCCARTESIANPOINT((0.,0.0500000007450581,0.));\n#2923=IFCDIRECTION((0.,-1.,0.));\n#2924=IFCDIRECTION((1.,0.,0.));\n#2925=IFCAXIS2PLACEMENT3D(#2922,#2923,#2924);\n#2926=IFCDIRECTION((0.,0.,-1.));\n#2927=IFCEXTRUDEDAREASOLID(#2921,#2925,#2926,0.0399999991059303);\n#2928=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.80999994277954,0.),(1.80999994277954,1.54999995231628),(0.,1.54999995231628)),$);\n#2929=IFCINDEXEDPOLYCURVE(#2928,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2930=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.77499985694885,0.0350000001490116),(1.77499985694885,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2931=IFCINDEXEDPOLYCURVE(#2930,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2932=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2929,(#2931));\n#2933=IFCCARTESIANPOINT((0.025000000372529,0.0900000035762787,0.025000000372529));\n#2934=IFCDIRECTION((0.,-1.,0.));\n#2935=IFCDIRECTION((1.,0.,0.));\n#2936=IFCAXIS2PLACEMENT3D(#2933,#2934,#2935);\n#2937=IFCDIRECTION((0.,0.,-1.));\n#2938=IFCEXTRUDEDAREASOLID(#2932,#2936,#2937,0.0350000001490116);\n#2939=IFCINDEXEDPOLYCURVE(#2940,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2940=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.77499985694885,0.0350000001490116),(1.77499985694885,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2941=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2939);\n#2942=IFCCARTESIANPOINT((0.025000000372529,0.10249999910593,0.025000000372529));\n#2943=IFCDIRECTION((0.,-1.,0.));\n#2944=IFCDIRECTION((1.,0.,0.));\n#2945=IFCAXIS2PLACEMENT3D(#2942,#2943,#2944);\n#2946=IFCDIRECTION((0.,0.,-1.));\n#2947=IFCEXTRUDEDAREASOLID(#2941,#2945,#2946,0.01);\n#2948=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2916,#2927,#2938,#2947));\n#2949=IFCCARTESIANPOINT((-5.36,1.84,0.4));\n#2950=IFCDIRECTION((0.,0.,1.));\n#2951=IFCDIRECTION((0.,-1.,0.));\n#2952=IFCAXIS2PLACEMENT3D(#2949,#2950,#2951);\n#2953=IFCLOCALPLACEMENT($,#2952);\n#2954=IFCPRODUCTDEFINITIONSHAPE($,$,(#2948));\n#2955=IFCRELFILLSELEMENT('3U5$pn7Ef7HQ$1EKY3RVKd',$,$,$,#291,#2905);\n#2956=IFCOWNERHISTORY(#19,#16,.READWRITE.,.MODIFIED.,1682677457,#19,#16,1682677457);\n#2957=IFCWINDOW('3mU$JU8Z1CIfPQgHKe0Nrd',#2956,'Left Window',$,$,#3089,#3090,$,$,$,.WINDOW.,.TRIPLE_PANEL_VERTICAL.,$);\n#2958=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#2959=IFCINDEXEDPOLYCURVE(#2958,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2960=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.025000000372529),(1.80833339691162,0.025000000372529),(1.80833339691162,1.57500004768372),(0.025000000372529,1.57500004768372)),$);\n#2961=IFCINDEXEDPOLYCURVE(#2960,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2962=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2959,(#2961));\n#2963=IFCCARTESIANPOINT((0.,0.0900000035762787,0.));\n#2964=IFCDIRECTION((0.,-1.,0.));\n#2965=IFCDIRECTION((1.,0.,0.));\n#2966=IFCAXIS2PLACEMENT3D(#2963,#2964,#2965);\n#2967=IFCDIRECTION((0.,0.,-1.));\n#2968=IFCEXTRUDEDAREASOLID(#2962,#2966,#2967,0.0100000007078052);\n#2969=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#2970=IFCINDEXEDPOLYCURVE(#2969,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2971=IFCCARTESIANPOINTLIST2D(((0.0500000007450581,0.0500000007450581),(1.80833327770233,0.0500000007450581),(1.80833327770233,1.54999995231628),(0.0500000007450581,1.54999995231628)),$);\n#2972=IFCINDEXEDPOLYCURVE(#2971,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2973=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2970,(#2972));\n#2974=IFCCARTESIANPOINT((0.,0.0500000007450581,0.));\n#2975=IFCDIRECTION((0.,-1.,0.));\n#2976=IFCDIRECTION((1.,0.,0.));\n#2977=IFCAXIS2PLACEMENT3D(#2974,#2975,#2976);\n#2978=IFCDIRECTION((0.,0.,-1.));\n#2979=IFCEXTRUDEDAREASOLID(#2973,#2977,#2978,0.0399999991059303);\n#2980=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.78333330154419,0.),(1.78333330154419,1.54999995231628),(0.,1.54999995231628)),$);\n#2981=IFCINDEXEDPOLYCURVE(#2980,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2982=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2983=IFCINDEXEDPOLYCURVE(#2982,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2984=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#2981,(#2983));\n#2985=IFCCARTESIANPOINT((0.025000000372529,0.0900000035762787,0.025000000372529));\n#2986=IFCDIRECTION((0.,-1.,0.));\n#2987=IFCDIRECTION((1.,0.,0.));\n#2988=IFCAXIS2PLACEMENT3D(#2985,#2986,#2987);\n#2989=IFCDIRECTION((0.,0.,-1.));\n#2990=IFCEXTRUDEDAREASOLID(#2984,#2988,#2989,0.0350000001490116);\n#2991=IFCINDEXEDPOLYCURVE(#2992,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#2992=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#2993=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#2991);\n#2994=IFCCARTESIANPOINT((0.025000000372529,0.10249999910593,0.025000000372529));\n#2995=IFCDIRECTION((0.,-1.,0.));\n#2996=IFCDIRECTION((1.,0.,0.));\n#2997=IFCAXIS2PLACEMENT3D(#2994,#2995,#2996);\n#2998=IFCDIRECTION((0.,0.,-1.));\n#2999=IFCEXTRUDEDAREASOLID(#2993,#2997,#2998,0.01);\n#3000=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#3001=IFCINDEXEDPOLYCURVE(#3000,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3002=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.025000000372529),(1.80833339691162,0.025000000372529),(1.80833339691162,1.57500004768372),(0.025000000372529,1.57500004768372)),$);\n#3003=IFCINDEXEDPOLYCURVE(#3002,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3004=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3001,(#3003));\n#3005=IFCCARTESIANPOINT((1.83333337306976,0.0900000035762787,0.));\n#3006=IFCDIRECTION((0.,-1.,0.));\n#3007=IFCDIRECTION((1.,0.,0.));\n#3008=IFCAXIS2PLACEMENT3D(#3005,#3006,#3007);\n#3009=IFCDIRECTION((0.,0.,-1.));\n#3010=IFCEXTRUDEDAREASOLID(#3004,#3008,#3009,0.0100000007078052);\n#3011=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#3012=IFCINDEXEDPOLYCURVE(#3011,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3013=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.0500000007450581),(1.80833339691162,0.0500000007450581),(1.80833339691162,1.54999995231628),(0.025000000372529,1.54999995231628)),$);\n#3014=IFCINDEXEDPOLYCURVE(#3013,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3015=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3012,(#3014));\n#3016=IFCCARTESIANPOINT((1.83333337306976,0.0500000007450581,0.));\n#3017=IFCDIRECTION((0.,-1.,0.));\n#3018=IFCDIRECTION((1.,0.,0.));\n#3019=IFCAXIS2PLACEMENT3D(#3016,#3017,#3018);\n#3020=IFCDIRECTION((0.,0.,-1.));\n#3021=IFCEXTRUDEDAREASOLID(#3015,#3019,#3020,0.0399999991059303);\n#3022=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.78333330154419,0.),(1.78333330154419,1.54999995231628),(0.,1.54999995231628)),$);\n#3023=IFCINDEXEDPOLYCURVE(#3022,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3024=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#3025=IFCINDEXEDPOLYCURVE(#3024,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3026=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3023,(#3025));\n#3027=IFCCARTESIANPOINT((1.85833334922791,0.0900000035762787,0.025000000372529));\n#3028=IFCDIRECTION((0.,-1.,0.));\n#3029=IFCDIRECTION((1.,0.,0.));\n#3030=IFCAXIS2PLACEMENT3D(#3027,#3028,#3029);\n#3031=IFCDIRECTION((0.,0.,-1.));\n#3032=IFCEXTRUDEDAREASOLID(#3026,#3030,#3031,0.0350000001490116);\n#3033=IFCINDEXEDPOLYCURVE(#3034,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3034=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#3035=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#3033);\n#3036=IFCCARTESIANPOINT((1.85833334922791,0.10249999910593,0.025000000372529));\n#3037=IFCDIRECTION((0.,-1.,0.));\n#3038=IFCDIRECTION((1.,0.,0.));\n#3039=IFCAXIS2PLACEMENT3D(#3036,#3037,#3038);\n#3040=IFCDIRECTION((0.,0.,-1.));\n#3041=IFCEXTRUDEDAREASOLID(#3035,#3039,#3040,0.01);\n#3042=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#3043=IFCINDEXEDPOLYCURVE(#3042,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3044=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.025000000372529),(1.80833339691162,0.025000000372529),(1.80833339691162,1.57500004768372),(0.025000000372529,1.57500004768372)),$);\n#3045=IFCINDEXEDPOLYCURVE(#3044,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3046=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3043,(#3045));\n#3047=IFCCARTESIANPOINT((3.66666674613953,0.0900000035762787,0.));\n#3048=IFCDIRECTION((0.,-1.,0.));\n#3049=IFCDIRECTION((1.,0.,0.));\n#3050=IFCAXIS2PLACEMENT3D(#3047,#3048,#3049);\n#3051=IFCDIRECTION((0.,0.,-1.));\n#3052=IFCEXTRUDEDAREASOLID(#3046,#3050,#3051,0.0100000007078052);\n#3053=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.83333337306976,0.),(1.83333337306976,1.60000002384186),(0.,1.60000002384186)),$);\n#3054=IFCINDEXEDPOLYCURVE(#3053,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3055=IFCCARTESIANPOINTLIST2D(((0.025000000372529,0.0500000007450581),(1.78333330154419,0.0500000007450581),(1.78333330154419,1.54999995231628),(0.025000000372529,1.54999995231628)),$);\n#3056=IFCINDEXEDPOLYCURVE(#3055,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3057=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3054,(#3056));\n#3058=IFCCARTESIANPOINT((3.66666674613953,0.0500000007450581,0.));\n#3059=IFCDIRECTION((0.,-1.,0.));\n#3060=IFCDIRECTION((1.,0.,0.));\n#3061=IFCAXIS2PLACEMENT3D(#3058,#3059,#3060);\n#3062=IFCDIRECTION((0.,0.,-1.));\n#3063=IFCEXTRUDEDAREASOLID(#3057,#3061,#3062,0.0399999991059303);\n#3064=IFCCARTESIANPOINTLIST2D(((0.,0.),(1.78333330154419,0.),(1.78333330154419,1.54999995231628),(0.,1.54999995231628)),$);\n#3065=IFCINDEXEDPOLYCURVE(#3064,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3066=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#3067=IFCINDEXEDPOLYCURVE(#3066,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3068=IFCARBITRARYPROFILEDEFWITHVOIDS(.AREA.,$,#3065,(#3067));\n#3069=IFCCARTESIANPOINT((3.69166684150696,0.0900000035762787,0.025000000372529));\n#3070=IFCDIRECTION((0.,-1.,0.));\n#3071=IFCDIRECTION((1.,0.,0.));\n#3072=IFCAXIS2PLACEMENT3D(#3069,#3070,#3071);\n#3073=IFCDIRECTION((0.,0.,-1.));\n#3074=IFCEXTRUDEDAREASOLID(#3068,#3072,#3073,0.0350000001490116);\n#3075=IFCINDEXEDPOLYCURVE(#3076,(IFCLINEINDEX((1,2)),IFCLINEINDEX((2,3)),IFCLINEINDEX((3,4)),IFCLINEINDEX((4,1))),$);\n#3076=IFCCARTESIANPOINTLIST2D(((0.0350000001490116,0.0350000001490116),(1.7483332157135,0.0350000001490116),(1.7483332157135,1.5149998664856),(0.0350000001490116,1.5149998664856)),$);\n#3077=IFCARBITRARYCLOSEDPROFILEDEF(.AREA.,$,#3075);\n#3078=IFCCARTESIANPOINT((3.69166684150696,0.10249999910593,0.025000000372529));\n#3079=IFCDIRECTION((0.,-1.,0.));\n#3080=IFCDIRECTION((1.,0.,0.));\n#3081=IFCAXIS2PLACEMENT3D(#3078,#3079,#3080);\n#3082=IFCDIRECTION((0.,0.,-1.));\n#3083=IFCEXTRUDEDAREASOLID(#3077,#3081,#3082,0.01);\n#3084=IFCSHAPEREPRESENTATION(#12,'Body','SweptSolid',(#2968,#2979,#2990,#2999,#3010,#3021,#3032,#3041,#3052,#3063,#3074,#3083));\n#3085=IFCCARTESIANPOINT((-5.31,-0.12,0.4));\n#3086=IFCDIRECTION((0.,0.,1.));\n#3087=IFCDIRECTION((1.,0.,0.));\n#3088=IFCAXIS2PLACEMENT3D(#3085,#3086,#3087);\n#3089=IFCLOCALPLACEMENT($,#3088);\n#3090=IFCPRODUCTDEFINITIONSHAPE($,$,(#3084));\n#3091=IFCRELFILLSELEMENT('3nI0lpdkn43h$HclzFv0KO',$,$,$,#87,#2957);\n#3092=IFCSURFACESTYLE($,.BOTH.,(#3093));\n#3093=IFCSURFACESTYLESHADING(#3094,0.8);\n#3094=IFCCOLOURRGB($,0.5,0.4,0.3);\n#3095=IFCSTYLEDITEM(#2864,(#3092),$);\n#3096=IFCSTYLEDITEM(#2875,(#3092),$);\n#3097=IFCSTYLEDITEM(#2886,(#3092),$);\n#3098=IFCSTYLEDITEM(#2895,(#3092),$);\n#3099=IFCSTYLEDITEM(#2916,(#3092),$);\n#3100=IFCSTYLEDITEM(#2927,(#3092),$);\n#3101=IFCSTYLEDITEM(#2938,(#3092),$);\n#3102=IFCSTYLEDITEM(#2947,(#3092),$);\n#3103=IFCSTYLEDITEM(#2968,(#3092),$);\n#3104=IFCSTYLEDITEM(#2979,(#3092),$);\n#3105=IFCSTYLEDITEM(#2990,(#3092),$);\n#3106=IFCSTYLEDITEM(#2999,(#3092),$);\n#3107=IFCSTYLEDITEM(#3010,(#3092),$);\n#3108=IFCSTYLEDITEM(#3021,(#3092),$);\n#3109=IFCSTYLEDITEM(#3032,(#3092),$);\n#3110=IFCSTYLEDITEM(#3041,(#3092),$);\n#3111=IFCSTYLEDITEM(#3052,(#3092),$);\n#3112=IFCSTYLEDITEM(#3063,(#3092),$);\n#3113=IFCSTYLEDITEM(#3074,(#3092),$);\n#3114=IFCSTYLEDITEM(#3083,(#3092),$);\nENDSEC;\nEND-ISO-10303-21;\n"""

If browsing through the website version, the preview has been already shown at the top of the page. Do you fancy trying with a 60º roof? or taller walls? Chek out [the notebook version](https://github.com/cvillagrasa/IfcOpenHouse/blob/master/nbs/00_generation.ipynb) in order to be able to visualize those changes.

## Saving our IFC file

Finally, we can dump the programmatically generated file into disk:

In [ ]:
#| exports

ifc_path = Path('..') / 'ifc' / 'IfcOpenHouse.ifc'
file.write(ifc_path)

And that was it! we have programmatically and procedurally generated a simple house in IFC. What are you thinking to build next? 😎

## Known missing features

If you feel something is missing or can be improve, kindly [file an issue](https://github.com/cvillagrasa/IfcOpenHouse/issues/new). This is a list of known missing features at the moment:

- Using IFC Types

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()